# Дообучение encoder'а для классификации последовательностей.

In [ ]:
!pip install evaluate
import pandas as pd
import numpy as np
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
import evaluate
from datasets import Dataset
from tqdm.auto import tqdm

class WeightedCETrainer(Trainer):
    """
    Кастомный Trainer с поддержкой взвешенной CrossEntropy.
    Приоритет источников весов:
      1) class_weights (готовый тензор весов)
      2) train_labels (массив меток train-сплита)
      3) train_data_df + target_column_name (DataFrame и имя колонки меток)
    Формула: weight_i = N / (K * n_i), где K — число классов, n_i — кол-во примеров класса i.
    Отсутствующие классы получают вес 0.
    """

    def __init__(
        self,
        *args,
        train_data_df=None,
        target_column_name=None,
        num_labels=None,
        train_labels=None,
        class_weights=None,
        **kwargs
    ):
        # Убираем future warning: tokenizer -> processing_class (если передан)
        processing = kwargs.pop("tokenizer", None)
        if processing is not None and "processing_class" not in kwargs:
            kwargs["processing_class"] = processing

        super().__init__(*args, **kwargs)

        self.num_labels = num_labels or getattr(self.model.config, "num_labels", None)

        if class_weights is not None:
            # Готовые веса
            w = torch.as_tensor(class_weights, dtype=torch.float32)
        else:
            # Считаем веса из train_labels или train_data_df
            labels_arr = None
            if train_labels is not None:
                labels_arr = np.asarray(train_labels)
            elif train_data_df is not None and target_column_name is not None:
                labels_arr = np.asarray(train_data_df[target_column_name].values)

            w = None
            if labels_arr is not None and self.num_labels is not None:
                if not np.issubdtype(labels_arr.dtype, np.integer):
                    _, labels_arr = np.unique(labels_arr, return_inverse=True)
                labels_arr = labels_arr.astype(int)
                counts = np.bincount(labels_arr, minlength=self.num_labels)
                n_samples = counts.sum()

                weights = np.zeros(self.num_labels, dtype=np.float32)
                nonzero = counts > 0
                weights[nonzero] = n_samples / (self.num_labels * counts[nonzero].astype(np.float32))
                w = torch.tensor(weights, dtype=torch.float32)

        self.class_weights = w  # может быть None

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits if hasattr(outputs, "logits") else outputs["logits"]

        weight = self.class_weights.to(logits.device) if self.class_weights is not None else None
        loss_fct = torch.nn.CrossEntropyLoss(weight=weight)
        loss = loss_fct(logits, labels.long())

        return (loss, outputs) if return_outputs else loss

class SequenceClassification:
    """
    Pipeline, благодаря которому можно быстро и удобно загружать encoder из transformers и дообучать его для
    классификации последовательностей. Проблема дисбаланса классов решается (взвешенная CrossEntropy).

    Необходимые импорты:
    import pandas as pd
    import numpy as np
    import torch
    from transformers import AutoModelForSequenceClassification, AutoTokenizer, Trainer, TrainingArguments
    import evaluate
    from datasets import Dataset
    from tqdm.auto import tqdm
    """

    def __init__(self, checkpoint, num_labels, target_column_name, texts_columns_names):
        """
        :param checkpoint: Название предобученной модели из Hugging Face Hub (например, 'bert-base-uncased').
        :param num_labels: Количество классов в задаче классификации.
        :param target_column_name: Имя столбца с целевой переменной.
        :param texts_columns_names: Список с именами текстовых столбцов, которые нужно объединить и использовать.
        """

        self.tokenizer = AutoTokenizer.from_pretrained(checkpoint)
        self.model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=num_labels)
        self.target_column_name = target_column_name
        self.texts_columns_names = texts_columns_names
        self.num_labels = num_labels

        # Маппинги меток (заполняются в fit при необходимости)
        self.label2id = None
        self.id2label = None

        self.trainer = None
        self.compute_metrics = None

    def _prepare_dataset(self, df):
        """Внутренний метод для подготовки датасета."""
        df_copy = df.copy()

        # Используем специальный SEP-токен токенайзера (если есть), чтобы не склеивать строки буквальным "[SEP]"
        sep_tok = self.tokenizer.sep_token if self.tokenizer.sep_token is not None else "[SEP]"
        df_copy['text'] = df_copy[self.texts_columns_names].fillna('').agg(sep_tok.join, axis=1)

        # Применяем mapping меток к id, если он уже построен
        if self.label2id is not None:
            df_copy[self.target_column_name] = df_copy[self.target_column_name].map(self.label2id).astype(int)

        dataset = Dataset.from_pandas(df_copy[[self.target_column_name, 'text']])

        def tokenize_function(examples):
            return self.tokenizer(
                examples['text'],
                truncation=True,
                padding='max_length',
                max_length=self.model.config.max_position_embeddings
            )

        tokenized_dataset = dataset.map(tokenize_function, batched=True)
        tokenized_dataset = tokenized_dataset.rename_column(self.target_column_name, 'labels')

        return tokenized_dataset

    def _setup_compute_metrics(self, metric_name):
        """Внутренний метод для настройки функции подсчета метрик."""
        if metric_name == 'f1':
            metric = evaluate.load('f1')
            def compute_f1(pred):
                if isinstance(pred, (tuple, list)):
                    logits, labels = pred
                else:
                    logits, labels = pred.predictions, pred.label_ids
                predictions = np.argmax(logits, axis=-1)
                return metric.compute(predictions=predictions, references=labels, average='weighted')
            self.compute_metrics = compute_f1

        elif metric_name == 'accuracy':
            metric = evaluate.load('accuracy')
            def compute_accuracy(pred):
                if isinstance(pred, (tuple, list)):
                    logits, labels = pred
                else:
                    logits, labels = pred.predictions, pred.label_ids
                predictions = np.argmax(logits, axis=-1)
                return metric.compute(predictions=predictions, references=labels)
            self.compute_metrics = compute_accuracy
        else:
            raise ValueError('Параметр "metric_name" может быть только "f1" или "accuracy".')

    def fit(self, train_data, epochs=3, test_size=0.2, per_device_train_batch_size=32, 
            gradient_accumulation_steps=1, learning_rate=2e-5, metric_name='f1', fp16=True,
            logging_steps=50, eval_steps=100, output_dir='./result'):
        """
        Дообучает модель на предоставленных данных.
        """

        # Если метки не 0..K-1 (или строковые), построим mapping по train_data
        classes = sorted(train_data[self.target_column_name].unique().tolist())
        if self.num_labels != len(classes):
            print(f"Warning: num_labels={self.num_labels} != len(classes)={len(classes)}")
        self.label2id = {c: i for i, c in enumerate(classes)}
        self.id2label = {i: str(c) for c, i in self.label2id.items()}
        # Сохраняем в конфиг модели
        self.model.config.label2id = {str(k): int(v) for k, v in self.label2id.items()}
        self.model.config.id2label = {int(k): str(v) for k, v in self.id2label.items()}

        # Готовим датасет и делим его на train/test
        full_dataset = self._prepare_dataset(train_data)
        data = full_dataset.train_test_split(test_size=test_size)

        # Считаем веса классов по train-сплиту (после split)
        train_labels = np.array(data['train']['labels'])
        counts = np.bincount(train_labels, minlength=self.num_labels)
        n_samples = counts.sum()
        class_weights = np.zeros(self.num_labels, dtype=np.float32)
        nonzero = counts > 0
        class_weights[nonzero] = n_samples / (self.num_labels * counts[nonzero].astype(np.float32))

        self._setup_compute_metrics(metric_name=metric_name.lower())

        training_arguments = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=epochs,
            per_device_train_batch_size=per_device_train_batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            learning_rate=learning_rate,
            warmup_ratio=0.1,
            lr_scheduler_type='cosine',
            weight_decay=0.01,
            eval_strategy='steps',
            eval_steps=eval_steps,
            save_strategy='steps',
            save_steps=eval_steps,
            load_best_model_at_end=True,
            metric_for_best_model=metric_name,
            save_total_limit=1,
            logging_strategy='steps',
            logging_steps=logging_steps,
            report_to='none',
            fp16=fp16 and torch.cuda.is_available()
        )

        # Используем WeightedCETrainer и передаем готовые class_weights
        self.trainer = WeightedCETrainer(
            model=self.model,
            args=training_arguments,
            train_dataset=data['train'],
            eval_dataset=data['test'],
            tokenizer=self.tokenizer,
            compute_metrics=self.compute_metrics,
            num_labels=self.num_labels,
            train_labels=train_labels,
            class_weights=class_weights
        )

        self.trainer.train()

        return self

    def predict(self, df):
        """
        Делает предсказания для новых данных.
        """
        if self.trainer is None:
            raise RuntimeError("Модель еще не обучена. Вызовите метод .fit() перед предсказанием.")

        df_copy = df.copy()
        df_copy[self.target_column_name] = 0  # фиктивный таргет
        predict_dataset = self._prepare_dataset(df_copy)
        predictions = self.trainer.predict(predict_dataset)

        return np.argmax(predictions.predictions, axis=-1)

    def get_embeddings(self, df, batch_size=32):
        """
        Извлекает эмбеддинги [CLS] токена для каждой строки текста.
        Этот метод идеально подходит для создания признаков для других моделей (CatBoost, LightGBM).
        """
        if self.trainer is None:
            raise RuntimeError("Модель должна быть обучена. Вызовите .fit() перед извлечением эмбеддингов.")

        device = self.trainer.model.device
        self.trainer.model.eval()

        df_copy = df.copy()
        sep_tok = self.tokenizer.sep_token if self.tokenizer.sep_token is not None else "[SEP]"
        df_copy['text'] = df_copy[self.texts_columns_names].fillna('').agg(sep_tok.join, axis=1)
        texts = df_copy['text'].tolist()

        all_embeddings = []

        for i in tqdm(range(0, len(texts), batch_size), desc="Извлечение эмбеддингов"):
            batch_texts = texts[i:i + batch_size]

            inputs = self.tokenizer(
                batch_texts, 
                truncation=True,
                padding='max_length',
                max_length=self.model.config.max_position_embeddings,
                return_tensors="pt"
            ).to(device)

            with torch.no_grad():
                base_model = getattr(self.trainer.model, self.trainer.model.base_model_prefix)
                outputs = base_model(**inputs)

            cls_embeddings = outputs.last_hidden_state[:, 0, :]
            all_embeddings.append(cls_embeddings.cpu().numpy())

        return np.vstack(all_embeddings)

Пример использования.

In [ ]:
# создание данных
train_data = pd.DataFrame({
    'name': ['Федя', 'Одиссей', 'Маргулан', 'Андриус'],
    'user_description': ['Я люблю отдыхать.', 'Ненавижу работать дома!', 'Я ГЕЙМЕР.', 'Осуждаю курение.'],
    'target': [1, 1, 1, 0]
})
submission_data = pd.DataFrame({
    'name': ['Женя', 'Людмила'],
    'user_description': ['Нет амбиций.', 'Кумирница Арнольда Шварцнеггера.',]
})

# создание и обучение модели
model = SequenceClassification(
    checkpoint='DeepPavlov/rubert-base-cased',
    num_labels=2,
    target_column_name='target',
    texts_columns_names=['name', 'user_description']
)
model.fit(
    train_data,
    epochs=3,
    test_size=0.1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    learning_rate=2e-5,
    metric_name='f1'
)

# прогнозирование и получение эмбеддингов
labels = model.predict(submission_data)
print(labels)
embeddings = model.get_embeddings(submission_data, batch_size=2)
print(embeddings)

# Дообучение encoder'а для регрессии на основе последовательностей.

In [ ]:
class SequenceRegression:
    """
    Pipeline, благодаря которому можно быстро и удобно загружать encoder из transformers и дообучать его для
    задачи регрессии (предсказания непрерывного значения).
    """

    def __init__(self, checkpoint, target_column_name, texts_columns_names):
        """
        :param checkpoint: Название предобученной модели из Hugging Face Hub.
        :param target_column_name: Имя столбца с целевой переменной (непрерывным значением).
        :param texts_columns_names: Список с именами текстовых столбцов для объединения.
        """

        self.tokenizer = AutoTokenizer.from_pretrained(checkpoint)
        self.model = AutoModelForSequenceClassification.from_pretrained(checkpoint, num_labels=1)
        self.target_column_name = target_column_name
        self.texts_columns_names = texts_columns_names

        self.trainer = None
        self.compute_metrics = None

    def _prepare_dataset(self, df):
        """Внутренний метод для подготовки датасета."""

        df_copy = df.copy()
        df_copy[self.target_column_name] = np.log1p(df_copy[self.target_column_name])
        df_copy['text'] = df_copy[self.texts_columns_names].fillna('').agg('[SEP]'.join, axis=1)
        df_copy[self.target_column_name] = df_copy[self.target_column_name].astype(np.float32)
        
        dataset = Dataset.from_pandas(df_copy[[self.target_column_name, 'text']])

        def tokenize_function(examples):
            return self.tokenizer(
                examples['text'], 
                truncation=True,
                padding='max_length',
                max_length=self.model.config.max_position_embeddings
            )

        tokenized_dataset = dataset.map(tokenize_function, batched=True)
        tokenized_dataset = tokenized_dataset.rename_column(self.target_column_name, 'labels')
        return tokenized_dataset

    def _setup_compute_metrics(self, metric_name):
        """Внутренний метод для настройки функции подсчета метрик для регрессии."""

        if metric_name == 'mape':
            metric = evaluate.load("mape")
            def compute_mape(pred):
                logits, labels = pred
                predictions = logits.flatten()

                # исключаем нулевые значения из labels, чтобы избежать деления на ноль
                mask = labels != 0
                if not np.any(mask): return {'mape': 0.0}
                
                filtered_predictions = predictions[mask]
                filtered_labels = labels[mask]
                return metric.compute(predictions=filtered_predictions, references=filtered_labels)
            self.compute_metrics = compute_mape
        else:
            try:
                metric = evaluate.load(metric_name)

                def compute_generic_metric(pred):
                    logits, labels = pred
                    predictions = logits.flatten()
                    return metric.compute(predictions=predictions, references=labels)

                self.compute_metrics = compute_generic_metric
            except FileNotFoundError:
                raise ValueError(f'Метрика "{metric_name}" не найдена.')

    def fit(self, train_data, epochs=3, test_size=0.2, per_device_train_batch_size=32, 
            gradient_accumulation_steps=1, learning_rate=2e-5, metric_name='mse', fp16=True,
            logging_steps=50, eval_steps=100, output_dir='./result'):
        """Дообучает модель на предоставленных данных для задачи регрессии."""

        full_dataset = self._prepare_dataset(train_data)
        data = full_dataset.train_test_split(test_size=test_size)
        self._setup_compute_metrics(metric_name=metric_name.lower())

        training_arguments = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=epochs,
            per_device_train_batch_size=per_device_train_batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            learning_rate=learning_rate,
            warmup_ratio=0.1,
            lr_scheduler_type='cosine',
            weight_decay=0.01,
            eval_strategy='steps',
            eval_steps=eval_steps,
            save_strategy='steps',
            save_steps=eval_steps,
            load_best_model_at_end=True,
            metric_for_best_model=metric_name,
            save_total_limit=1,
            logging_strategy='steps',
            logging_steps=logging_steps,
            report_to='none',
            fp16=fp16
        )

        self.trainer = Trainer(
            model=self.model, args=training_arguments, train_dataset=data['train'],
            eval_dataset=data['test'], tokenizer=self.tokenizer, compute_metrics=self.compute_metrics
        )

        self.trainer.train()

        return self

    def predict(self, df):
        """
        Делает предсказания для новых данных.
        """

        if self.trainer is None:
            raise RuntimeError("Модель еще не обучена. Вызовите метод .fit() перед предсказанием.")

        df_copy = df.copy()
        df_copy[self.target_column_name] = 0.0  # фиктивный таргет
        predict_dataset = self._prepare_dataset(df_copy)
        predictions = self.trainer.predict(predict_dataset)
        predictions =  np.expm1(predictions.predictions.flatten())
        return predictions

    def get_embeddings(self, df, batch_size=32):
        """
        Извлекает эмбеддинги [CLS] токена для каждой строки текста.
        Этот метод идеально подходит для создания признаков для других моделей (CatBoost, LightGBM).

        :param df: pd.DataFrame с текстами для обработки.
        :param batch_size: Размер батча для обработки. Подбирайте для оптимальной скорости.
        :return: np.array размером (n_samples, hidden_size) с эмбеддингами.
        """

        if self.trainer is None:
            raise RuntimeError("Модель должна быть обучена. Вызовите .fit() перед извлечением эмбеддингов.")
        
        model = self.trainer.model
        device = next(model.parameters()).device
        model.eval()
        
        df_copy = df.copy()
        df_copy['text'] = df_copy[self.texts_columns_names].fillna('').agg('[SEP]'.join, axis=1)
        texts = df_copy['text'].tolist()
        
        max_len = min(getattr(self.tokenizer, "model_max_length", 512),
                      getattr(self.model.config, "max_position_embeddings", 512))
        
        all_embeddings = []
        
        for i in range(0, len(texts), batch_size):
            batch_texts = texts[i:i + batch_size]
        
            inputs = self.tokenizer(
                batch_texts,
                truncation=True,
                padding='max_length',
                max_length=max_len,
                return_tensors='pt'
            ).to(device)
        
            with torch.no_grad():
                outputs = model(**inputs, output_hidden_states=True, return_dict=True)
        
            # Берём последнюю скрытую матрицу и [CLS]
            last_hidden = outputs.hidden_states[-1] if outputs.hidden_states is not None else outputs.last_hidden_state
            cls_embeddings = last_hidden[:, 0, :]
            all_embeddings.append(cls_embeddings.cpu().numpy())
        
        return np.vstack(all_embeddings)

Пример использования.

In [ ]:
# создание данных
train_data = pd.DataFrame({
    'name': ['Федя', 'Одиссей', 'Маргулан', 'Андриус'],
    'user_description': ['Я люблю отдыхать.', 'Ненавижу работать дома!', 'Я киберспортсмен мира.', 'Осуждаю курение.'],
    'target': [10000, 40000, 1000000, 55000]
})
submission_data = pd.DataFrame({
    'name': ['Женя', 'Людмила'],
    'user_description': ['Нет амбиций.', 'Кумирница Арнольда Шварцнеггера.',]
})

# создание и обучение модели
model = SequenceRegression(
    checkpoint='DeepPavlov/rubert-base-cased',
    target_column_name='target',
    texts_columns_names=['name', 'user_description']
)
model.fit(
    train_data,
    epochs=3,
    test_size=0.1,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    learning_rate=2e-5,
    metric_name='mape'
    # fp16=True  # если используется GPU
)

# прогнозирование и получение эмбеддингов
labels = model.predict(submission_data)  # аргумента batch_size нет
print(labels)
embeddings = model.get_embeddings(submission_data, batch_size=2)
print(embeddings)

# Дообучение encoder'а для классификации токенов.

In [ ]:
!pip install evaluate seqeval
import numpy as np
import torch
import evaluate
from transformers import (
    AutoModelForTokenClassification,
    AutoTokenizer,
    Trainer,
    TrainingArguments,
    DataCollatorForTokenClassification,
)
import pandas as pd
from datasets import Dataset
from sklearn.model_selection import train_test_split
from tqdm.auto import tqdm

class WeightedTokenCETrainer(Trainer):
    """
    Кастомный Trainer для токен-классификации с взвешенной CrossEntropy.
    Веса считаются с учетом числа токенов класса (после разметки и чанкинга):
      weight_i = N / (K * n_i),
    где K — число классов, n_i — число токенов класса i, N — сумма всех n_i.
    Отсутствующие классы получают вес 0. Метки -100 игнорируются в потере.
    """

    def __init__(self, *args, class_weights=None, **kwargs):
        # Тихо переводим устаревший аргумент tokenizer в processing_class
        processing = kwargs.pop("tokenizer", None)
        if processing is not None and "processing_class" not in kwargs:
            kwargs["processing_class"] = processing

        super().__init__(*args, **kwargs)

        self.class_weights = None
        if class_weights is not None:
            self.class_weights = torch.as_tensor(class_weights, dtype=torch.float32)

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits if hasattr(outputs, "logits") else outputs["logits"]
        # logits: (batch, seq_len, num_labels) -> (batch*seq_len, num_labels)
        # labels: (batch, seq_len) -> (batch*seq_len)
        loss_fct = torch.nn.CrossEntropyLoss(
            weight=(self.class_weights.to(logits.device) if self.class_weights is not None else None),
            ignore_index=-100,
        )
        loss = loss_fct(logits.view(-1, logits.size(-1)), labels.view(-1))
        return (loss, outputs) if return_outputs else loss


class TokenClassification:
    """
    Необходимые импорты:
    import numpy as np
    import torch
    import evaluate
    from transformers import (
        AutoModelForTokenClassification,
        AutoTokenizer,
        Trainer,
        TrainingArguments,
        DataCollatorForTokenClassification,
    )
    import pandas as pd
    from datasets import Dataset
    from sklearn.model_selection import train_test_split
    from tqdm.auto import tqdm
    """
    
    def __init__(self, checkpoint, label2id, tokens_column_name, tags_column_name):
        self.id2label = {v: k for k, v in label2id.items()}
        self.label2id = label2id
        
        self.model = AutoModelForTokenClassification.from_pretrained(
            checkpoint,
            num_labels=len(self.id2label),
            id2label=self.id2label,
            label2id=self.label2id,
            ignore_mismatched_sizes=True
        )
        
        self.tokenizer = AutoTokenizer.from_pretrained(checkpoint)
        self.tokens_column_name = tokens_column_name
        self.tags_column_name = tags_column_name
        self.trainer = None
        self.compute_metrics = None
        self.data_collator = DataCollatorForTokenClassification(tokenizer=self.tokenizer)

    def _align_labels_with_tokens(self, labels, word_ids):
        new_labels = []
        current_word = None
        
        for word_id in word_ids:
            if word_id != current_word:
                current_word = word_id
                label = -100 if word_id is None else labels[word_id]
                new_labels.append(label)
            elif word_id is None:
                new_labels.append(-100)
            else:
                new_labels.append(-100)
        
        return new_labels

    def _prepare_dataset_with_sliding_window(self, df, max_length, stride):
        df = df.copy()
        
        all_chunked_input_ids = []
        all_chunked_attention_masks = []
        all_chunked_labels = []

        for _, row in tqdm(df.iterrows(), total=len(df), desc="Нарезка данных на чанки"):
            tokens = row[self.tokens_column_name]
            labels = row[self.tags_column_name]
            
            tokenized_chunks = self.tokenizer(
                [tokens],
                is_split_into_words=True,
                return_overflowing_tokens=True,
                max_length=max_length,
                stride=stride,
                truncation=True
            )
            
            tokenized_chunks.pop("overflow_to_sample_mapping", None)
            
            num_chunks = len(tokenized_chunks['input_ids'])
            for i in range(num_chunks):
                chunk_word_ids = self.tokenizer.word_ids(batch_index=i) if hasattr(self.tokenizer, "word_ids") else tokenized_chunks.word_ids(batch_index=i)
                # На случай, если метод доступен только у BatchEncoding:
                if chunk_word_ids is None:
                    chunk_word_ids = tokenized_chunks.word_ids(batch_index=i)
                aligned_labels = self._align_labels_with_tokens(labels, chunk_word_ids)
                
                all_chunked_input_ids.append(tokenized_chunks['input_ids'][i])
                all_chunked_attention_masks.append(tokenized_chunks['attention_mask'][i])
                all_chunked_labels.append(aligned_labels)

        return Dataset.from_dict({
            'input_ids': all_chunked_input_ids,
            'attention_mask': all_chunked_attention_masks,
            'labels': all_chunked_labels
        })

    def _setup_compute_metrics(self):
        metric = evaluate.load("seqeval")
        
        def compute_seqeval_metrics(p):
            # Поддержка EvalPrediction и tuple
            if isinstance(p, (tuple, list)):
                predictions, labels = p
            else:
                predictions, labels = p.predictions, p.label_ids

            predictions = np.argmax(predictions, axis=2)
            
            true_predictions = [
                [self.id2label[p] for (p, l) in zip(prediction, label) if l != -100]
                for prediction, label in zip(predictions, labels)
            ]
            
            true_labels = [
                [self.id2label[l] for (p, l) in zip(prediction, label) if l != -100]
                for prediction, label in zip(predictions, labels)
            ]
            
            results = metric.compute(predictions=true_predictions, references=true_labels)
            
            return {
                "precision": results.get("overall_precision", 0.0),
                "recall": results.get("overall_recall", 0.0),
                "f1": results.get("overall_f1", 0.0),
                "accuracy": results.get("overall_accuracy", 0.0),
            }
            
        self.compute_metrics = compute_seqeval_metrics

    def fit(self, train_data, epochs=3, per_device_train_batch_size=16, gradient_accumulation_steps=1,
            test_size=0.2, learning_rate=2e-5, fp16=True, stride=128, logging_steps=50,
            eval_steps=100, output_dir='./result'):
        
        max_length = self.model.config.max_position_embeddings
        
        train_data_prepared = train_data.copy()
        # Маппинг строковых тегов -> id при необходимости
        if isinstance(train_data_prepared[self.tags_column_name].iloc[0][0], str):
            train_data_prepared[self.tags_column_name] = train_data_prepared[self.tags_column_name].apply(
                lambda tags: [self.label2id[tag] for tag in tags]
            )
        
        # Сплит по документам, затем чанкуем отдельно
        train_df, eval_df = train_test_split(train_data_prepared, test_size=test_size, random_state=42)
        train_dataset = self._prepare_dataset_with_sliding_window(train_df, max_length, stride)
        eval_dataset = self._prepare_dataset_with_sliding_window(eval_df, max_length, stride)

        # Считаем веса классов по токенам train_dataset (игнорируя -100)
        labels_list = train_dataset["labels"]  # список списков
        flat = np.concatenate([np.asarray(x, dtype=np.int64) for x in labels_list]) if len(labels_list) > 0 else np.array([], dtype=np.int64)
        flat = flat[flat >= 0]  # убираем -100
        num_labels = len(self.id2label)
        counts = np.bincount(flat, minlength=num_labels) if flat.size > 0 else np.zeros(num_labels, dtype=np.int64)
        n_samples = counts.sum()
        class_weights = np.zeros(num_labels, dtype=np.float32)
        nonzero = counts > 0
        if n_samples > 0:
            class_weights[nonzero] = n_samples / (num_labels * counts[nonzero].astype(np.float32))
        # Отсутствующие классы останутся с весом 0

        self._setup_compute_metrics()

        training_arguments = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=epochs,
            per_device_train_batch_size=per_device_train_batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            learning_rate=learning_rate,
            warmup_ratio=0.1,
            lr_scheduler_type='cosine',
            weight_decay=0.01,
            eval_strategy='steps',
            eval_steps=eval_steps,
            save_strategy='steps',
            save_steps=eval_steps,
            load_best_model_at_end=True,
            metric_for_best_model='f1',
            save_total_limit=1,
            logging_strategy='steps',
            logging_steps=logging_steps,
            report_to='none',
            fp16=fp16
        )

        # Используем взвешенную CrossEntropy через кастомный Trainer
        self.trainer = WeightedTokenCETrainer(
            model=self.model,
            args=training_arguments,
            data_collator=self.data_collator,
            compute_metrics=self.compute_metrics,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            tokenizer=self.tokenizer,
            class_weights=class_weights
        )
        
        self.trainer.train()
        
        return self

    def _predict_single_document(self, tokens, stride):
        max_length = self.model.config.max_position_embeddings
        
        tokenized_inputs = self.tokenizer(
            [tokens],
            is_split_into_words=True,
            return_overflowing_tokens=True,
            max_length=max_length,
            stride=stride, 
            truncation=True,
        )
        
        tokenized_inputs.pop("overflow_to_sample_mapping", None)
        chunk_dataset = Dataset.from_dict(tokenized_inputs)
        
        outputs = self.trainer.predict(chunk_dataset)
        predictions = np.argmax(outputs.predictions, axis=2)

        num_original_words = len(tokens)
        final_predictions = np.full(num_original_words, -1, dtype=np.int32)

        for i, chunk_preds in enumerate(predictions):
            chunk_word_ids = tokenized_inputs.word_ids(batch_index=i)
            for token_pos, word_id in enumerate(chunk_word_ids):
                if word_id is not None and final_predictions[word_id] == -1:
                    final_predictions[word_id] = chunk_preds[token_pos]

        return [self.id2label.get(pid, 'O') for pid in final_predictions]

    def predict(self, df, stride=128):
        all_final_labels = []
        
        for _, row in tqdm(df.iterrows(), total=len(df), desc="Предсказание (sliding window)"):
            original_tokens = row[self.tokens_column_name]
            if not original_tokens:
                all_final_labels.append([])
                continue
            
            document_labels = self._predict_single_document(original_tokens, stride)
            all_final_labels.append(document_labels)
            
        return all_final_labels

    def _get_embeddings_single_document(self, tokens, stride, device):
        max_length = self.model.config.max_position_embeddings
        num_original_words = len(tokens)

        chunk_inputs = self.tokenizer(
            [tokens],
            is_split_into_words=True,
            return_overflowing_tokens=True,
            max_length=max_length,
            stride=stride,
            truncation=True,
            return_tensors="pt"
        ).to(device)

        chunk_inputs.pop("overflow_to_sample_mapping")
        
        with torch.no_grad():
            base_model = getattr(self.trainer.model, self.trainer.model.base_model_prefix)
            outputs = base_model(**chunk_inputs)
        
        chunk_embeddings = outputs.last_hidden_state
        
        hidden_size = self.model.config.hidden_size
        final_word_embeddings = torch.zeros(num_original_words, hidden_size, device=device)
        word_counts = torch.zeros(num_original_words, device=device)

        for i in range(len(chunk_embeddings)):
            chunk_embeds = chunk_embeddings[i]
            chunk_word_ids = chunk_inputs.word_ids(batch_index=i)

            for token_pos, word_id in enumerate(chunk_word_ids):
                if word_id is not None:
                    final_word_embeddings[word_id] += chunk_embeds[token_pos]
                    if token_pos == 0 or chunk_word_ids[token_pos - 1] != word_id:
                        word_counts[word_id] += 1
        
        average_embeddings = final_word_embeddings / (word_counts.unsqueeze(1) + 1e-8)
        
        return average_embeddings.cpu().numpy()

    def get_embeddings(self, df, stride=128):
        self.trainer.model.eval()
        device = self.trainer.model.device
        all_final_embeddings = []

        for _, row in tqdm(df.iterrows(), total=len(df), desc="Генерация эмбеддингов (sliding window)"):
            original_tokens = row[self.tokens_column_name]
            if not original_tokens:
                all_final_embeddings.append(np.array([]))
                continue
            
            document_embeddings = self._get_embeddings_single_document(original_tokens, stride, device)
            all_final_embeddings.append(document_embeddings)
            
        return all_final_embeddings


Пример использования.

In [ ]:
# Для классификации токенов нам нужны списки токенов и списки тегов
train_data = pd.DataFrame({
    'tokens': [
        ['Федор', 'Достоевский', 'родился', 'в', 'Москве', '.'],
        ['Анна', 'Керн', 'была', 'музой', 'Пушкина', '.'],
        ['Компания', 'Яндекс', 'представила', 'Алису', '.'],
        ['Илон', 'Маск', 'основал', 'SpaceX', 'и', 'Tesla', '.']
    ],
    'ner_tags': [
        ['B-PER', 'I-PER', 'O', 'O', 'B-LOC', 'O'],
        ['B-PER', 'I-PER', 'O', 'O', 'B-PER', 'O'],
        ['O', 'B-ORG', 'O', 'B-PER', 'O'],
        ['B-PER', 'I-PER', 'O', 'B-ORG', 'O', 'B-ORG', 'O']
    ]
})

# Для предсказания нам нужны только токены
submission_data = pd.DataFrame({
    'tokens': [
        ['Лев', 'Толстой', 'написал', 'роман', '"', 'Война', 'и', 'мир', '"', '.'],
        ['Сергей', 'Королев', 'работал', 'в', 'РКК', '"', 'Энергия', '"', '.']
    ]
})

# 2. Создание и обучение модели
# Создаем маппинг из тегов в ID
tags = train_data['ner_tags'].explode().unique()
label2id = {tag: i for i, tag in enumerate(tags)}

model = TokenClassification(
    checkpoint='DeepPavlov/rubert-base-cased',
    label2id=label2id,
    tokens_column_name='tokens',
    tags_column_name='ner_tags'
)

model.fit(
    train_data,
    epochs=3,
    test_size=0.25,
    per_device_train_batch_size=2,
    gradient_accumulation_steps=1,
    learning_rate=2e-5,
    fp16=False
)

# 3. Прогнозирование и получение эмбеддингов
labels = model.predict(submission_data)
print(labels)
embeddings = model.get_embeddings(submission_data)
print(embeddings)

# Дообучение ViT'а и AST'а для классификации звуков.

In [ ]:
!pip install evaluate
import pandas as pd
import numpy as np
import torch
import librosa
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification, Trainer, TrainingArguments, DataCollatorWithPadding
import evaluate
from datasets import Dataset, Audio, ClassLabel
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import f1_score

class WeightedCETrainer(Trainer):
    """
    Кастомный Trainer с автоматическим вычислением весов классов.
    weight_i = N / (K * n_i), где:
      N — число обучающих примеров,
      K — число классов,
      n_i — число примеров класса i.
    Отсутствующие классы получают вес 0.
    """

    def __init__(
        self,
        *args,
        train_data_df=None,
        target_column_name=None,
        num_labels=None,
        **kwargs
    ):
        # тихо переводим устаревший аргумент tokenizer в processing_class,
        # чтобы убрать FutureWarning и ничего не менять в вызывающем коде
        processing = kwargs.pop("tokenizer", None)
        if processing is not None and "processing_class" not in kwargs:
            kwargs["processing_class"] = processing

        super().__init__(*args, **kwargs)

        self.num_labels = num_labels or getattr(self.model.config, "num_labels", None)
        self.class_weights = None

        if train_data_df is not None and target_column_name is not None and self.num_labels is not None:
            labels = np.asarray(train_data_df[target_column_name].values)

            # на всякий: если метки не целочисленные — факторизуем
            if not np.issubdtype(labels.dtype, np.integer):
                _, labels = np.unique(labels, return_inverse=True)

            labels = labels.astype(int)
            counts = np.bincount(labels, minlength=self.num_labels)
            n_samples = counts.sum()

            weights = np.zeros(self.num_labels, dtype=np.float32)
            nonzero = counts > 0
            weights[nonzero] = n_samples / (self.num_labels * counts[nonzero].astype(np.float32))

            self.class_weights = torch.tensor(weights, dtype=torch.float32)

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits if hasattr(outputs, "logits") else outputs["logits"]

        weight = self.class_weights.to(logits.device) if self.class_weights is not None else None
        loss_fct = torch.nn.CrossEntropyLoss(weight=weight)
        loss = loss_fct(logits, labels.long())

        return (loss, outputs) if return_outputs else loss

class AudioClassification:
    """
    Pipeline, благодаря которому можно быстро и удобно загружать аудио-энкодер из transformers и дообучать его для
    классификации аудиофайлов. Проблема дисбаланса классов частично решается с помощью метрики F1-weighted и
    взвешенной функции потерь (WeightedCETrainer). Добавлена возможность аугментации данных "на лету".
    """

    def __init__(self, checkpoint, num_labels, target_column_name, audio_path_column_name, audio_freq=16000, use_augmentation=False):
        """
        :param checkpoint: Название предобученной модели из Hugging Face Hub (например, 'MIT/ast-finetuned-audioset-10-10-0.4593').
        :param num_labels: Количество классов в задаче классификации.
        :param target_column_name: Имя столбца с целевой переменной.
        :param audio_path_column_name: Имя столбца с путями к аудиофайлам.
        :param audio_freq: Частота дискретизации, к которой будут приведены все аудио. По умолчанию 16000.
        :param use_augmentation: Использовать ли аугментацию данных при обучении. По умолчанию False.
        """
        self.feature_extractor = AutoFeatureExtractor.from_pretrained(checkpoint)
        self.model = AutoModelForAudioClassification.from_pretrained(
            checkpoint,
            num_labels=num_labels,
            ignore_mismatched_sizes=True  # Позволяет заменить "голову" классификатора
        )
        self.target_column_name = target_column_name
        self.audio_path_column_name = audio_path_column_name
        self.audio_freq = audio_freq
        self.num_labels = num_labels
        self.use_augmentation = use_augmentation

        self.trainer = None  # будет создан после обучения
        self.compute_metrics = None  # будет создана после выбора метрики

    def _apply_augmentations(self, samples, sample_rate):
        """
        Внутренний метод для применения серии аугментаций к одному аудиофайлу.
        Каждая аугментация применяется с вероятностью 20%.
        """
        # 1. AddGaussianNoise
        if np.random.rand() < 0.2:
            noise_amplitude = np.random.uniform(0.001, 0.015)
            noise = np.random.randn(len(samples))
            samples = samples + noise_amplitude * noise

        # 2. TimeStretch
        if np.random.rand() < 0.2:
            rate = np.random.uniform(0.8, 1.25)
            samples = librosa.effects.time_stretch(y=samples, rate=rate)

        # 3. PitchShift
        if np.random.rand() < 0.2:
            n_steps = np.random.uniform(-4, 4)
            samples = librosa.effects.pitch_shift(y=samples, sr=sample_rate, n_steps=n_steps)

        # 4. Shift (циклический сдвиг)
        if np.random.rand() < 0.2:
            shift_fraction = np.random.uniform(-0.5, 0.5)
            shift_samples = int(len(samples) * shift_fraction)
            samples = np.roll(samples, shift_samples)

        return samples

    def _prepare_dataset(self, df, augment=False):
        """Внутренний метод для подготовки датасета."""
        df_copy = df.copy()

        # Создаем Dataset из pandas DataFrame
        dataset = Dataset.from_pandas(df_copy[[self.target_column_name, self.audio_path_column_name]])

        # Используем магию datasets: указываем, что колонка - это аудио,
        # и оно будет автоматически загружено и передискретизировано
        dataset = dataset.cast_column(self.audio_path_column_name, Audio(sampling_rate=self.audio_freq))

        def preprocess_function(examples):
            # Извлекаем аудиоданные (уже загруженные и передискретизированные)
            audio_arrays = [x["array"] for x in examples[self.audio_path_column_name]]

            # Применяем аугментацию, если флаг установлен
            if augment:
                audio_arrays = [self._apply_augmentations(samples=audio, sample_rate=self.audio_freq) for audio in audio_arrays]

            # ВАЖНО: не возвращаем тензоры здесь — паддинг сделает коллатор
            inputs = self.feature_extractor(
                audio_arrays,
                sampling_rate=self.audio_freq
            )
            return inputs

        # Применяем предобработку
        processed_dataset = dataset.map(
            preprocess_function,
            remove_columns=[self.audio_path_column_name],
            batched=True
        )
        processed_dataset = processed_dataset.rename_column(self.target_column_name, 'labels')

        return processed_dataset

    def _setup_compute_metrics(self, metric_name):
        """Внутренний метод для настройки функции подсчета метрик."""
        if metric_name == 'f1':
            metric = evaluate.load('f1')
            def compute_f1(pred):
                if isinstance(pred, (tuple, list)):
                    logits, labels = pred
                else:
                    logits, labels = pred.predictions, pred.label_ids
                predictions = np.argmax(logits, axis=-1)
                return metric.compute(predictions=predictions, references=labels, average='weighted')
            self.compute_metrics = compute_f1

        elif metric_name == 'accuracy':
            metric = evaluate.load('accuracy')
            def compute_accuracy(pred):
                if isinstance(pred, (tuple, list)):
                    logits, labels = pred
                else:
                    logits, labels = pred.predictions, pred.label_ids
                predictions = np.argmax(logits, axis=-1)
                return metric.compute(predictions=predictions, references=labels)
            self.compute_metrics = compute_accuracy
        else:
            raise ValueError('Параметр "metric_name" может быть только "f1" или "accuracy".')

    def fit(self, train_data, epochs=3, test_size=0.2, per_device_train_batch_size=16,
            gradient_accumulation_steps=1, learning_rate=2e-5, metric_name='f1', fp16=True,
            logging_steps=50, eval_steps=100, output_dir='./result'):
        """
        Дообучает модель на предоставленных данных.
        """
        # Разделяем DataFrame на train и test до создания Dataset'ов.
        # Это необходимо, чтобы применять аугментацию только к обучающей выборке.
        train_df, test_df = train_test_split(
            train_data,
            test_size=test_size,
            stratify=train_data[self.target_column_name],
            random_state=42 # для воспроизводимости
        )

        train_dataset = self._prepare_dataset(train_df, augment=self.use_augmentation)
        eval_dataset = self._prepare_dataset(test_df, augment=False) # Аугментация на тесте не нужна

        # Приводим колонку с метками к ClassLabel для обеих выборок
        label_casting = ClassLabel(num_classes=self.num_labels)
        train_dataset = train_dataset.cast_column("labels", label_casting)
        eval_dataset = eval_dataset.cast_column("labels", label_casting)

        self._setup_compute_metrics(metric_name=metric_name.lower())

        training_arguments = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=epochs,
            per_device_train_batch_size=per_device_train_batch_size,
            per_device_eval_batch_size=per_device_train_batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            learning_rate=learning_rate,
            warmup_ratio=0.1,
            lr_scheduler_type='cosine',
            weight_decay=0.01,
            eval_strategy='steps',
            save_strategy='steps',
            load_best_model_at_end=True,
            metric_for_best_model=metric_name,
            save_total_limit=1,
            logging_strategy='steps',
            logging_steps=logging_steps,
            eval_steps=eval_steps,
            save_steps=eval_steps,
            report_to='none',
            fp16=fp16 and torch.cuda.is_available()
        )

        # Явный коллейтор для аудио: паддинг по input_values
        data_collator = DataCollatorWithPadding(tokenizer=self.feature_extractor, padding=True)

        # Используем WeightedCETrainer вместо стандартного Trainer
        self.trainer = WeightedCETrainer(
            model=self.model,
            args=training_arguments,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            processing_class=self.feature_extractor,  # вместо tokenizer
            compute_metrics=self.compute_metrics,
            data_collator=data_collator,
            # Доп. аргументы для вычисления весов классов:
            train_data_df=train_df,
            target_column_name=self.target_column_name,
            num_labels=self.num_labels,
        )

        self.trainer.train()

        return self

    def predict(self, df):
        """
        Делает предсказания для новых данных.
        """
        if self.trainer is None:
            raise RuntimeError("Модель еще не обучена. Вызовите метод .fit() перед предсказанием.")

        df_copy = df.copy()
        df_copy[self.target_column_name] = 0  # фиктивный таргет
        # При предсказании аугментация всегда выключена
        predict_dataset = self._prepare_dataset(df_copy, augment=False)

        predictions = self.trainer.predict(predict_dataset)
        return np.argmax(predictions.predictions, axis=-1)

    def get_embeddings(self, df, batch_size=32):
        """
        Извлекает эмбеддинги для каждого аудиофайла.
        Этот метод идеально подходит для создания признаков для других моделей (CatBoost, LightGBM).

        :param df: pd.DataFrame с путями к аудиофайлам.
        :param batch_size: Размер батча для обработки. Подбирайте для оптимальной скорости.
        :return: np.array размером (n_samples, hidden_size) с эмбеддингами.
        """
        if self.trainer is None:
            raise RuntimeError("Модель должна быть обучена. Вызовите .fit() перед извлечением эмбеддингов.")

        device = self.trainer.model.device
        self.trainer.model.eval()

        paths = df[self.audio_path_column_name].tolist()
        all_embeddings = []

        for i in tqdm(range(0, len(paths), batch_size), desc="Извлечение эмбеддингов"):
            batch_paths = paths[i:i + batch_size]
            batch_audio = [librosa.load(p, sr=self.audio_freq)[0] for p in batch_paths]

            inputs = self.feature_extractor(
                batch_audio,
                sampling_rate=self.audio_freq,
                return_tensors="pt"
            ).to(device)

            with torch.no_grad():
                # Получаем базовую модель без классификационной "головы"
                base_model_prefix = self.trainer.model.base_model_prefix
                base_model = getattr(self.trainer.model, base_model_prefix)
                outputs = base_model(**inputs)

            # Для AST (и ViT) эмбеддинг всего инпута - это скрытое состояние [CLS] токена
            cls_embeddings = outputs.last_hidden_state[:, 0, :]
            all_embeddings.append(cls_embeddings.cpu().numpy())

        return np.vstack(all_embeddings)

Пример использования.

In [ ]:
import os
import shutil
import numpy as np
import soundfile as sf
import pandas as pd

TEMP_AUDIO_DIR = 'temp_audio'

if os.path.exists(TEMP_AUDIO_DIR):
    shutil.rmtree(TEMP_AUDIO_DIR)
os.makedirs(TEMP_AUDIO_DIR)

SAMPLE_RATE = 16000
DURATION = 2
N_SAMPLES = SAMPLE_RATE * DURATION
AMPLITUDE = np.iinfo(np.int16).max * 0.3
t = np.linspace(0., DURATION, N_SAMPLES)

data_config = [
    {'label': 0, 'count': 5, 'base_freq': 400},
    {'label': 1, 'count': 7, 'base_freq': 1000},
    {'label': 2, 'count': 6, 'base_freq': 250},
]

all_files_data = []

for item in data_config:
    label = item['label']
    count = item['count']
    base_freq = item['base_freq']

    for i in range(1, count + 1):
        freq_variation = np.random.randint(-50, 50)
        current_freq = base_freq + freq_variation
        
        audio_data = (AMPLITUDE * np.sin(2. * np.pi * current_freq * t)).astype(np.int16)
        
        file_path = os.path.join(TEMP_AUDIO_DIR, f"{label}_{i}.wav")
        
        sf.write(file_path, audio_data, SAMPLE_RATE)
        
        all_files_data.append({'path': file_path, 'label': label})

final_df = pd.DataFrame(all_files_data).sample(frac=1)

In [ ]:
# 1. Подготовка данных в формате pd.DataFrame
# В реальном сценарии данные загружаются из CSV файла.
train_data = final_df[:12]
submission_data = final_df[12:]

# 2. Инициализация и обучение модели
model = AudioClassification(
    checkpoint='MIT/ast-finetuned-audioset-10-10-0.4593',
    num_labels=3,
    target_column_name='label',
    audio_path_column_name='path',
    use_augmentation=True
)

model.fit(
    train_data,
    epochs=1,
    test_size=0.25,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=1,
    learning_rate=3e-5,
    metric_name='accuracy',
    fp16=False,
    logging_steps=2,
    eval_steps=4
)

# 3. Прогнозирование и получение эмбеддингов
labels = model.predict(submission_data)
print(labels)

embeddings = model.get_embeddings(submission_data, batch_size=2)
print(f'Форма эмбеддингов: {embeddings.shape}')

# Дообучение ViT'а и AST'а для регрессии звуков.

In [ ]:
import pandas as pd
import numpy as np
import torch
import librosa
from transformers import AutoFeatureExtractor, AutoModelForAudioClassification, Trainer, TrainingArguments
import evaluate
from datasets import Dataset, Audio
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split # Для корректного разделения данных

class AudioRegression:
    """
    Pipeline, благодаря которому можно быстро и удобно загружать аудио-энкодер из transformers и дообучать его для
    задачи регрессии (предсказания непрерывного значения) по аудиофайлам.
    Добавлена возможность аугментации данных "на лету" с помощью librosa и numpy.

    Необходимые импорты:
    import pandas as pd
    import numpy as np
    import torch
    import librosa
    from transformers import AutoFeatureExtractor, AutoModelForAudioClassification, Trainer, TrainingArguments
    import evaluate
    from datasets import Dataset, Audio
    from tqdm.auto import tqdm
    from sklearn.model_selection import train_test_split
    """

    def __init__(self, checkpoint, target_column_name, audio_path_column_name, audio_freq=16000, use_augmentation=False):
        """
        :param checkpoint: Название предобученной модели из Hugging Face Hub (например, 'MIT/ast-finetuned-audioset-10-10-0.4593').
        :param target_column_name: Имя столбца с целевой переменной (непрерывное значение).
        :param audio_path_column_name: Имя столбца с путями к аудиофайлам.
        :param audio_freq: Частота дискретизации, к которой будут приведены все аудио. По умолчанию 16000.
        :param use_augmentation: Использовать ли аугментацию данных при обучении. По умолчанию False.
        """
        self.feature_extractor = AutoFeatureExtractor.from_pretrained(checkpoint)
        self.model = AutoModelForAudioClassification.from_pretrained(
            checkpoint,
            num_labels=1,  # Для регрессии предсказываем одно значение
            ignore_mismatched_sizes=True  # Позволяет заменить "голову" классификатора на регрессионную
        )
        self.target_column_name = target_column_name
        self.audio_path_column_name = audio_path_column_name
        self.audio_freq = audio_freq
        self.use_augmentation = use_augmentation

        self.trainer = None  # будет создан после обучения
        self.compute_metrics = None  # будет создана после выбора метрики

    def _apply_augmentations(self, samples, sample_rate):
        """
        Внутренний метод для применения серии аугментаций к одному аудиофайлу.
        Каждая аугментация применяется с вероятностью 20%.
        """
        # 1. AddGaussianNoise
        if np.random.rand() < 0.2:
            noise_amplitude = np.random.uniform(0.001, 0.015)
            noise = np.random.randn(len(samples))
            samples = samples + noise_amplitude * noise

        # 2. TimeStretch
        if np.random.rand() < 0.2:
            rate = np.random.uniform(0.8, 1.25)
            samples = librosa.effects.time_stretch(y=samples, rate=rate)

        # 3. PitchShift
        if np.random.rand() < 0.2:
            n_steps = np.random.uniform(-4, 4)
            samples = librosa.effects.pitch_shift(y=samples, sr=sample_rate, n_steps=n_steps)

        # 4. Shift (циклический сдвиг)
        if np.random.rand() < 0.2:
            shift_fraction = np.random.uniform(-0.5, 0.5)
            shift_samples = int(len(samples) * shift_fraction)
            samples = np.roll(samples, shift_samples)

        return samples

    def _prepare_dataset(self, df, augment=False):
        """Внутренний метод для подготовки датасета."""
        df_copy = df.copy()
        # Убедимся, что таргет имеет тип float32, что стандартно для регрессии в PyTorch
        df_copy[self.target_column_name] = df_copy[self.target_column_name].astype(np.float32)

        dataset = Dataset.from_pandas(df_copy[[self.target_column_name, self.audio_path_column_name]])
        dataset = dataset.cast_column(self.audio_path_column_name, Audio(sampling_rate=self.audio_freq))

        def preprocess_function(examples):
            audio_arrays = [x["array"] for x in examples[self.audio_path_column_name]]

            if augment:
                audio_arrays = [self._apply_augmentations(samples=audio, sample_rate=self.audio_freq) for audio in audio_arrays]

            inputs = self.feature_extractor(
                audio_arrays,
                sampling_rate=self.audio_freq,
                return_tensors="pt"
            )
            return inputs

        processed_dataset = dataset.map(preprocess_function, remove_columns=self.audio_path_column_name, batched=True)
        processed_dataset = processed_dataset.rename_column(self.target_column_name, 'labels')

        return processed_dataset

    def _setup_compute_metrics(self, metric_name):
        """Внутренний метод для настройки функции подсчета метрик регрессии."""
        metric_name = metric_name.lower()
        if metric_name in ['mse', 'rmse']:
            metric = evaluate.load('mse')
            def compute_mse(pred):
                logits, labels = pred
                predictions = logits.squeeze(-1)
                result = metric.compute(predictions=predictions, references=labels)
                # Добавляем RMSE, если нужно
                if metric_name == 'rmse':
                    result['rmse'] = np.sqrt(result['mse'])
                return result
            self.compute_metrics = compute_mse
        elif metric_name == 'mae':
            metric = evaluate.load('mae')
            def compute_mae(pred):
                logits, labels = pred
                predictions = logits.squeeze(-1)
                return metric.compute(predictions=predictions, references=labels)
            self.compute_metrics = compute_mae
        else:
            raise ValueError('Параметр "metric_name" может быть только "mse", "rmse" или "mae".')


    def fit(self, train_data, epochs=3, test_size=0.2, per_device_train_batch_size=16,
            gradient_accumulation_steps=1, learning_rate=2e-5, metric_name='mse', fp16=True,
            logging_steps=50, eval_steps=100, output_dir='./result'):
        """
        Дообучает модель на предоставленных данных.
        """
        # Для регрессии стратификация не используется
        train_df, test_df = train_test_split(
            train_data,
            test_size=test_size,
            random_state=42 # для воспроизводимости
        )

        train_dataset = self._prepare_dataset(train_df, augment=self.use_augmentation)
        eval_dataset = self._prepare_dataset(test_df, augment=False) # Аугментация на тесте не нужна

        self._setup_compute_metrics(metric_name=metric_name.lower())
        
        # Для метрик, где чем меньше, тем лучше, Trainer сам это определяет, но можно указать явно
        metric_to_track = 'rmse' if metric_name == 'rmse' else metric_name
        
        training_arguments = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=epochs,
            per_device_train_batch_size=per_device_train_batch_size,
            per_device_eval_batch_size=per_device_train_batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            learning_rate=learning_rate,
            warmup_ratio=0.1,
            lr_scheduler_type='cosine',
            weight_decay=0.01,
            eval_strategy='steps',
            save_strategy='steps',
            eval_steps=eval_steps,
            save_steps=eval_steps,
            load_best_model_at_end=True,
            metric_for_best_model=metric_to_track,
            save_total_limit=1,
            logging_strategy='steps',
            logging_steps=logging_steps,
            report_to='none',
            fp16=fp16,
            greater_is_better=False # Для MSE, RMSE, MAE чем меньше, тем лучше
        )

        self.trainer = Trainer(
            model=self.model,
            args=training_arguments,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            tokenizer=self.feature_extractor,
            compute_metrics=self.compute_metrics
        )

        self.trainer.train()

        return self

    def predict(self, df):
        """
        Делает предсказания (регрессию) для новых данных.
        """
        if self.trainer is None:
            raise RuntimeError("Модель еще не обучена. Вызовите метод .fit() перед предсказанием.")

        df_copy = df.copy()
        df_copy[self.target_column_name] = 0.0  # фиктивный таргет типа float

        predict_dataset = self._prepare_dataset(df_copy, augment=False)
        predictions = self.trainer.predict(predict_dataset)
        
        # Для регрессии возвращаем сами значения, убирая лишнюю размерность
        return predictions.predictions.squeeze(-1)

    def get_embeddings(self, df, batch_size=32):
        """
        Извлекает эмбеддинги для каждого аудиофайла.
        Этот метод идеально подходит для создания признаков для других моделей (CatBoost, LightGBM).
        Работает идентично, независимо от задачи (классификация или регрессия).
        """
        if self.trainer is None:
            raise RuntimeError("Модель должна быть обучена. Вызовите .fit() перед извлечением эмбеддингов.")

        device = self.trainer.model.device
        self.trainer.model.eval()

        paths = df[self.audio_path_column_name].tolist()
        all_embeddings = []

        for i in tqdm(range(0, len(paths), batch_size), desc="Извлечение эмбеддингов"):
            batch_paths = paths[i:i + batch_size]
            
            batch_audio = [librosa.load(p, sr=self.audio_freq)[0] for p in batch_paths]
            
            inputs = self.feature_extractor(
                batch_audio,
                sampling_rate=self.audio_freq,
                return_tensors="pt"
            ).to(device)

            with torch.no_grad():
                base_model_prefix = self.trainer.model.base_model_prefix
                base_model = getattr(self.trainer.model, base_model_prefix)
                outputs = base_model(**inputs)

            cls_embeddings = outputs.last_hidden_state[:, 0, :]
            all_embeddings.append(cls_embeddings.cpu().numpy())

        return np.vstack(all_embeddings)

Пример использования.

In [ ]:
import os
import shutil
import numpy as np
import soundfile as sf
import pandas as pd

TEMP_AUDIO_DIR = 'temp_audio_regression'

if os.path.exists(TEMP_AUDIO_DIR):
    shutil.rmtree(TEMP_AUDIO_DIR)
os.makedirs(TEMP_AUDIO_DIR)

SAMPLE_RATE = 16000
DURATION = 2
N_SAMPLES = SAMPLE_RATE * DURATION
AMPLITUDE = np.iinfo(np.int16).max * 0.3
t = np.linspace(0., DURATION, N_SAMPLES)

all_files_data = []
NUM_FILES_TOTAL = 31

for i in range(NUM_FILES_TOTAL):
    # Генерируем случайную частоту - это и будет наш таргет для регрессии
    current_freq = np.random.uniform(200.0, 1200.0)
    
    audio_data = (AMPLITUDE * np.sin(2. * np.pi * current_freq * t)).astype(np.int16)
    
    file_path = os.path.join(TEMP_AUDIO_DIR, f"freq_{current_freq:.2f}_{i}.wav")
    
    sf.write(file_path, audio_data, SAMPLE_RATE)
    
    # В DataFrame сохраняем путь и реальную частоту
    all_files_data.append({'path': file_path, 'frequency': current_freq})

final_df = pd.DataFrame(all_files_data).sample(frac=1).reset_index(drop=True)

In [ ]:
# 2. Подготовка данных в формате pd.DataFrame
# В реальном сценарии данные загружаются из CSV файла.
train_data = final_df[:25]
submission_data = final_df[25:]

# 3. Инициализация и обучение модели для РЕГРЕССИИ
model = AudioRegression(
    checkpoint='MIT/ast-finetuned-audioset-10-10-0.4593',
    target_column_name='frequency',  # Название колонки с таргетом
    audio_path_column_name='path',
    use_augmentation=True
)

model.fit(
    train_data,
    epochs=5,
    test_size=0.25,
    per_device_train_batch_size=8,
    gradient_accumulation_steps=1,
    learning_rate=3e-5,
    metric_name='mae',
    fp16=False
)
preds = model.predict(submission_data)
embeddings = model.get_embeddings(submission_data, batch_size=2)

print(preds)
print(embeddings)

# Дообучение классификатора картинок.

In [ ]:
!pip install evaluate
import pandas as pd
import numpy as np
import torch
import albumentations as A
from PIL import Image
from transformers import (
    AutoImageProcessor,
    AutoModelForImageClassification,
    Trainer,
    TrainingArguments
)
import evaluate
from datasets import Dataset, ClassLabel, Image as DSImage
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from transformers.trainer_utils import get_last_checkpoint
import os
import json

class WeightedCETrainer(Trainer):
    """
    Кастомный Trainer с автоматическим вычислением весов классов.
    weight_i = N / (K * n_i), где:
      N — число обучающих примеров,
      K — число классов,
      n_i — число примеров класса i.
    Отсутствующие классы получают вес 0.
    """

    def __init__(
        self,
        *args,
        train_data_df=None,
        target_column_name=None,
        num_labels=None,
        **kwargs
    ):
        # тихо переводим устаревший аргумент tokenizer в processing_class,
        # чтобы убрать FutureWarning и ничего не менять в вызывающем коде
        processing = kwargs.pop("tokenizer", None)
        if processing is not None and "processing_class" not in kwargs:
            kwargs["processing_class"] = processing

        super().__init__(*args, **kwargs)

        self.num_labels = num_labels or getattr(self.model.config, "num_labels", None)
        self.class_weights = None

        if train_data_df is not None and target_column_name is not None and self.num_labels is not None:
            labels = np.asarray(train_data_df[target_column_name].values)

            # на всякий: если метки не целочисленные — факторизуем
            if not np.issubdtype(labels.dtype, np.integer):
                _, labels = np.unique(labels, return_inverse=True)

            labels = labels.astype(int)
            counts = np.bincount(labels, minlength=self.num_labels)
            n_samples = counts.sum()

            weights = np.zeros(self.num_labels, dtype=np.float32)
            nonzero = counts > 0
            weights[nonzero] = n_samples / (self.num_labels * counts[nonzero].astype(np.float32))

            self.class_weights = torch.tensor(weights, dtype=torch.float32)

    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits if hasattr(outputs, "logits") else outputs["logits"]

        weight = self.class_weights.to(logits.device) if self.class_weights is not None else None
        loss_fct = torch.nn.CrossEntropyLoss(weight=weight)
        loss = loss_fct(logits, labels.long())

        return (loss, outputs) if return_outputs else loss

class ImageClassification:
    """
    Pipeline, благодаря которому можно быстро и удобно загружать энкодер изображений из transformers и дообучать его для
    классификации картинок. Проблема дисбаланса классов решается с помощью взвешенной функции потерь (автоматически),
    а также дополнительно оценивается метрикой F1-weighted. Добавлена возможность аугментации данных "на лету"
    с помощью albumentations и numpy.

    Необходимые импорты:
    import pandas as pd
    import numpy as np
    import torch
    import albumentations as A
    from PIL import Image
    from transformers import AutoImageProcessor, AutoModelForImageClassification, Trainer, TrainingArguments
    import evaluate
    from datasets import Dataset, ClassLabel, Image as DSImage
    from tqdm.auto import tqdm
    from sklearn.model_selection import train_test_split
    """

    def __init__(self, checkpoint, num_labels, target_column_name,
                 image_path_column_name, use_augmentation=False, target_size=(224, 224)):
        """
        :param checkpoint: Название предобученной модели из Hugging Face Hub.
        :param num_labels: Количество классов в задаче классификации.
        :param target_column_name: Имя столбца с целевой переменной.
        :param image_path_column_name: Имя столбца с путями к файлам изображений.
        :param use_augmentation: Использовать ли аугментацию данных при обучении. По умолчанию False.
        :param target_size: Размер, к которому будут приводится все картинки. По умолчанию (224, 224).
        """
        # стараемся использовать быстрый image processor, если доступен
        try:
            self.image_processor = AutoImageProcessor.from_pretrained(checkpoint, use_fast=True)
        except TypeError:
            self.image_processor = AutoImageProcessor.from_pretrained(checkpoint)

        self.model = AutoModelForImageClassification.from_pretrained(
            checkpoint,
            num_labels=num_labels,
            ignore_mismatched_sizes=True
        )
        self.target_column_name = target_column_name
        self.image_path_column_name = image_path_column_name
        self.num_labels = num_labels
        self.use_augmentation = use_augmentation
        self.target_size = target_size

        self.augment_transform = A.Compose([
            A.HorizontalFlip(p=0.5),
            A.ShiftScaleRotate(p=0.3, shift_limit=0.1, scale_limit=0.2, rotate_limit=20),
            A.RandomBrightnessContrast(p=0.3),
            A.CoarseDropout(p=0.3, max_holes=8, max_height=16, max_width=16),
            A.ToGray(p=0.2)
        ])

        self.trainer = None
        self.compute_metrics = None

    def _apply_augmentations(self, image):
        image_np = np.array(image)
        augmented_image_np = self.augment_transform(image=image_np)['image']
        return Image.fromarray(augmented_image_np)

    def _prepare_dataset(self, df):
        df_copy = df.copy()
        dataset = Dataset.from_pandas(
            df_copy[[self.target_column_name, self.image_path_column_name]],
            preserve_index=False
        )
        dataset = dataset.rename_columns({
            self.image_path_column_name: "image",
            self.target_column_name: "labels"
        })
        dataset = dataset.cast_column("image", DSImage(decode=True))
        return dataset

    def _make_transform(self, augment=False):
        def transform(batch):
            images = [img.convert("RGB").resize(self.target_size, Image.Resampling.BICUBIC)
                      for img in batch["image"]]
            if augment:
                images = [self._apply_augmentations(img) for img in images]
            inputs = self.image_processor(images, return_tensors="pt")
            # возвращаем список тензоров по одному на пример, чтобы collate_fn корректно склеил
            batch["pixel_values"] = [x for x in inputs["pixel_values"]]
            return batch
        return transform

    def _setup_compute_metrics(self, metric_name):
        if metric_name == 'f1':
            metric = evaluate.load('f1')
            def compute_f1(pred):
                if isinstance(pred, (tuple, list)):
                    logits, labels = pred
                else:
                    logits, labels = pred.predictions, pred.label_ids
                predictions = np.argmax(logits, axis=-1)
                return metric.compute(predictions=predictions, references=labels, average='weighted')
            self.compute_metrics = compute_f1

        elif metric_name == 'accuracy':
            metric = evaluate.load('accuracy')
            def compute_accuracy(pred):
                if isinstance(pred, (tuple, list)):
                    logits, labels = pred
                else:
                    logits, labels = pred.predictions, pred.label_ids
                predictions = np.argmax(logits, axis=-1)
                return metric.compute(predictions=predictions, references=labels)
            self.compute_metrics = compute_accuracy
        else:
            raise ValueError('Параметр "metric_name" может быть только "f1" или "accuracy".')

    def fit(self, train_data, epochs=3, test_size=0.2, per_device_train_batch_size=16,
            gradient_accumulation_steps=1, learning_rate=2e-5, metric_name='f1', fp16=True,
            logging_steps=50, eval_steps=100, output_dir='./result'):

        # стратифицированный сплит по исходным меткам
        train_df, test_df = train_test_split(
            train_data,
            test_size=test_size,
            stratify=train_data[self.target_column_name],
            random_state=42
        )

        # маппинг меток -> id (на случай строковых/не 0..K-1 меток)
        classes = sorted(train_df[self.target_column_name].unique().tolist())
        if self.num_labels != len(classes):
            print(f"Warning: num_labels={self.num_labels} != len(classes)={len(classes)}")

        label2id = {c: i for i, c in enumerate(classes)}
        id2label = {i: str(c) for c, i in label2id.items()}

        # применяем маппинг к train/test
        for df_ in (train_df, test_df):
            df_[self.target_column_name] = df_[self.target_column_name].map(label2id).astype(int)

        # сохраняем маппинги в конфиг модели
        self.model.config.label2id = {str(k): int(v) for k, v in label2id.items()}
        self.model.config.id2label = {int(k): str(v) for k, v in id2label.items()}

        train_dataset = self._prepare_dataset(train_df)
        eval_dataset = self._prepare_dataset(test_df)

        # приводим labels к ClassLabel с явными именами классов
        label_casting = ClassLabel(num_classes=self.num_labels, names=[id2label[i] for i in range(self.num_labels)])
        train_dataset = train_dataset.cast_column("labels", label_casting)
        eval_dataset = eval_dataset.cast_column("labels", label_casting)

        train_dataset = train_dataset.with_transform(self._make_transform(augment=self.use_augmentation))
        eval_dataset = eval_dataset.with_transform(self._make_transform(augment=False))

        def collate_fn(batch):
            return {
                'pixel_values': torch.stack([ex['pixel_values'] for ex in batch]),
                'labels': torch.tensor([int(ex['labels']) for ex in batch], dtype=torch.long)
            }

        self._setup_compute_metrics(metric_name=metric_name)

        def _safe_resume_ckpt(out_dir, expected_num_labels):
            if not os.path.isdir(out_dir):
                return None
            ckpt = get_last_checkpoint(out_dir)
            if ckpt is None:
                return None
            cfg_path = os.path.join(ckpt, "config.json")
            try:
                with open(cfg_path, "r") as f:
                    cfg = json.load(f)
                ckpt_labels = int(cfg.get("num_labels", -1))
            except Exception:
                ckpt_labels = -1
            if ckpt_labels == expected_num_labels:
                print(f"Resuming from checkpoint: {ckpt}")
                return ckpt
            print(f"Ignoring checkpoint {ckpt}: num_labels mismatch ({ckpt_labels} vs {expected_num_labels})")
            return None

        training_arguments = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=epochs,
            per_device_train_batch_size=per_device_train_batch_size,
            per_device_eval_batch_size=per_device_train_batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            learning_rate=learning_rate,
            warmup_ratio=0.1,
            lr_scheduler_type='cosine',
            weight_decay=0.01,
            eval_strategy='steps',
            save_strategy='steps',
            load_best_model_at_end=True,
            metric_for_best_model=metric_name,
            greater_is_better=True,
            save_total_limit=1,
            logging_strategy='steps',
            logging_steps=logging_steps,
            eval_steps=eval_steps,
            save_steps=eval_steps,
            report_to='none',
            fp16=fp16 and torch.cuda.is_available(),
            bf16=False,
            remove_unused_columns=False,
            dataloader_num_workers=1,
            dataloader_pin_memory=True
        )

        self.trainer = WeightedCETrainer(
            model=self.model,
            args=training_arguments,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            processing_class=self.image_processor,  # вместо tokenizer=...
            compute_metrics=self.compute_metrics,
            data_collator=collate_fn,
            train_data_df=train_df,                       # для вычисления весов классов
            target_column_name=self.target_column_name,
            num_labels=self.num_labels
        )

        last_ckpt = _safe_resume_ckpt(output_dir, self.num_labels)
        self.trainer.train(resume_from_checkpoint=last_ckpt)

        return self

    def predict(self, df):
        if self.trainer is None:
            raise RuntimeError("Модель еще не обучена. Вызовите .fit() перед предсказанием.")

        df_copy = df.copy()
        df_copy[self.target_column_name] = 0  # фиктивная метка для совместимости с коллатором
        predict_dataset = self._prepare_dataset(df_copy)
        predict_dataset = predict_dataset.with_transform(self._make_transform(augment=False))

        predictions = self.trainer.predict(predict_dataset)
        return np.argmax(predictions.predictions, axis=-1)

    def get_embeddings(self, df, batch_size=32):
        """
        Извлекает эмбеддинги для каждого изображения.
        Этот метод идеально подходит для создания признаков для других моделей (CatBoost, LightGBM).
        """
        if self.trainer is None:
            raise RuntimeError("Модель должна быть обучена. Вызовите .fit() перед извлечением эмбеддингов.")

        device = self.trainer.model.device
        self.trainer.model.eval()

        paths = df[self.image_path_column_name].tolist()
        all_embeddings = []

        for i in tqdm(range(0, len(paths), batch_size), desc="Извлечение эмбеддингов"):
            batch_paths = paths[i:i + batch_size]
            batch_images = [Image.open(p).convert("RGB") for p in batch_paths]

            inputs = self.image_processor(batch_images, return_tensors="pt").to(device)

            with torch.no_grad():
                base_model_prefix = self.trainer.model.base_model_prefix
                base_model = getattr(self.trainer.model, base_model_prefix)
                outputs = base_model(**inputs)

            cls_embeddings = outputs.last_hidden_state[:, 0, :]
            all_embeddings.append(cls_embeddings.cpu().numpy())

        return np.vstack(all_embeddings)

Пример использования.

In [ ]:
import os
import random

if not os.path.exists('dummy_images'):
    os.makedirs('dummy_images')

image_paths = []
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0)]
for i in range(20):
    path = f'dummy_images/img_{i}.png'
    color = colors[i % 4] 
    Image.new('RGB', (224, 224), color=color).save(path)
    image_paths.append(path)

data = {
    'image_path': image_paths,
    'label': [random.randint(0, 3) for i in range(20)]
}
full_df = pd.DataFrame(data)

In [ ]:
pipeline = ImageClassification(
    checkpoint='facebook/deit-tiny-patch16-224' ,
    num_labels=4,
    target_column_name='label',
    image_path_column_name='image_path',
    use_augmentation=True
)

pipeline.fit(
    train_data=full_df, 
    epochs=2,
    per_device_train_batch_size=4,
    fp16=torch.cuda.is_available(),
    logging_steps=2,
    eval_steps=4
)

test_df = full_df.head(5).copy()
predicted_labels = pipeline.predict(test_df)

print(f"Реальные метки: {test_df['label'].tolist()}")
print(f"Предсказанные метки: {predicted_labels.tolist()}")

embeddings = pipeline.get_embeddings(test_df, batch_size=5)

print(f"Форма массива эмбеддингов: {embeddings.shape}")

# Дообучение регрессора картинок.

In [ ]:
!pip install evaluate
import pandas as pd
import numpy as np
import torch
import albumentations as A
from PIL import Image
from transformers import (
    AutoImageProcessor,
    AutoModelForImageClassification,
    Trainer,
    TrainingArguments
)
import evaluate
from datasets import Dataset, Image as DSImage, Value, Sequence
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from transformers.trainer_utils import get_last_checkpoint
import os

class ImageRegression:
    """
    Класс-пайплайн для дообучения энкодеров изображений из Hugging Face Transformers
    на задачу регрессии. Поддерживает on-the-fly аугментации через Albumentations,
    автоматическую подготовку датасета и вычисление метрик (RMSE или MAE),
    а также извлечение эмбеддингов из базовой модели.

    Необходимые импорты:
    import pandas as pd
    import numpy as np
    import torch
    import albumentations as A
    from PIL import Image
    from transformers import (
        AutoImageProcessor,
        AutoModelForImageClassification,
        Trainer,
        TrainingArguments
    )
    import evaluate
    from datasets import Dataset, Image as DSImage, Value, Sequence
    from tqdm.auto import tqdm
    from sklearn.model_selection import train_test_split
    from transformers.trainer_utils import get_last_checkpoint
    import os
    """

    def __init__(
        self,
        checkpoint,
        target_columns_names,
        image_path_column_name,
        use_augmentation=False,
        target_size=(224, 224),
    ):
        """
        Инициализация пайплайна.

        :param checkpoint: Имя предобученной модели из Hugging Face Hub.
        :param target_columns_names: Имя столбца (str) или список имен столбцов (list[str])
                                     с целевыми переменными в DataFrame.
        :param image_path_column_name: Имя столбца с путями к изображениям в DataFrame.
        :param use_augmentation: Использовать ли аугментацию при обучении.
        :param target_size: Размер изображения (width, height), к которому приводятся все картинки.
        """
        if isinstance(target_columns_names, str):
            target_columns_names = [target_columns_names]
        self.target_columns_names = list(target_columns_names)
        self.image_path_column_name = image_path_column_name
        self.use_augmentation = use_augmentation
        self.target_size = target_size

        # Вычисляем число регрессионных таргетов
        self.num_labels = len(self.target_columns_names)

        self.image_processor = AutoImageProcessor.from_pretrained(checkpoint)

        self.model = AutoModelForImageClassification.from_pretrained(
            checkpoint,
            num_labels=self.num_labels,
            ignore_mismatched_sizes=True,
        )
        self.model.config.problem_type = "regression"

        self.augment_transform = A.Compose(
            [
                A.HorizontalFlip(p=0.5),
                A.ShiftScaleRotate(
                    p=0.3,
                    shift_limit=0.1,
                    scale_limit=0.2,
                    rotate_limit=20,
                ),
                A.RandomBrightnessContrast(p=0.3),
                A.CoarseDropout(
                    p=0.3,
                    max_holes=8,
                    max_height=16,
                    max_width=16,
                ),
                A.ToGray(p=0.2),
            ]
        )

        self.trainer = None
        self.compute_metrics = None

    def _apply_augmentations(self, image):
        """
        Применение аугментаций к одному изображению.

        :param image: PIL.Image.
        :return: Аугментированное PIL.Image.
        """
        image_np = np.array(image)
        augmented_image_np = self.augment_transform(image=image_np)["image"]
        return Image.fromarray(augmented_image_np)

    def _prepare_dataset(self, df):
        """
        Подготовка датасета Hugging Face Datasets из DataFrame.

        :param df: pandas.DataFrame с колонками таргетов и путей к изображениям.
        :return: datasets.Dataset с колонками image и labels (Sequence[float32] длины num_labels).
        """
        df_copy = df.copy()

        # Собираем labels как список float32 (мультитаргет поддерживается из коробки)
        labels = df_copy[self.target_columns_names].astype(np.float32).values.tolist()
        base_df = pd.DataFrame(
            {
                "image": df_copy[self.image_path_column_name].values,
                "labels": labels,
            }
        )

        dataset = Dataset.from_pandas(base_df, preserve_index=False)
        dataset = dataset.cast_column("image", DSImage(decode=True))

        # Приводим метки к Sequence(float32) фиксированной длины = num_labels
        dataset = dataset.cast_column(
            "labels",
            Sequence(feature=Value("float32"), length=self.num_labels),
        )

        return dataset

    def _make_transform(self, augment=False):
        """
        Создание функции трансформации для with_transform.

        :param augment: Применять ли аугментации.
        :return: Функция-трансформ для батча.
        """
        def transform(batch):
            images = [
                img.convert("RGB").resize(
                    self.target_size,
                    Image.Resampling.BICUBIC,
                )
                for img in batch["image"]
            ]

            if augment:
                images = [self._apply_augmentations(img) for img in images]

            inputs = self.image_processor(
                images=images,
                return_tensors="pt",
            )

            batch["pixel_values"] = [x for x in inputs["pixel_values"]]
            return batch

        return transform

    def _setup_compute_metrics(self, metric_name):
        """
        Настройка функции вычисления метрик.

        :param metric_name: 'rmse' или 'mae'.
        """
        def _extract(eval_pred):
            if isinstance(eval_pred, (tuple, list)):
                logits, labels = eval_pred
            else:
                logits = eval_pred.predictions
                labels = eval_pred.label_ids
            preds = np.array(logits)
            labels = np.array(labels)
            return preds, labels

        if metric_name == "rmse":
            def compute(eval_pred):
                preds, labels = _extract(eval_pred)
                rmse = float(np.sqrt(np.mean((preds - labels) ** 2)))
                return {"rmse": rmse}
            self.compute_metrics = compute

        elif metric_name == "mae":
            def compute(eval_pred):
                preds, labels = _extract(eval_pred)
                mae = float(np.mean(np.abs(preds - labels)))
                return {"mae": mae}
            self.compute_metrics = compute

        else:
            raise ValueError(
                'Параметр "metric_name" может быть только "rmse" или "mae".'
            )

    def fit(
        self,
        train_data,
        epochs=3,
        test_size=0.2,
        per_device_train_batch_size=16,
        gradient_accumulation_steps=1,
        learning_rate=2e-5,
        metric_name="rmse",
        fp16=True,
        logging_steps=50,
        eval_steps=100,
        output_dir='./result'
    ):
        """
        Обучение модели.

        :param train_data: pandas.DataFrame с колонками таргетов и путей к изображениям.
        :param epochs: Количество эпох.
        :param test_size: Размер валидационной выборки.
        :param per_device_train_batch_size: Размер батча на устройство для обучения.
        :param gradient_accumulation_steps: Шаги аккумуляции градиента.
        :param learning_rate: Скорость обучения.
        :param metric_name: Метрика для отслеживания лучшей модели: 'rmse' или 'mae'.
        :param fp16: Включить FP16 при наличии CUDA.
        :param logging_steps: Раз во сколько шагов записывать результаты модели.
        :param eval_steps: Раз во сколько шагов проверять модель на тестовой выборке.
        :return: self.
        """
        train_df, test_df = train_test_split(
            train_data,
            test_size=test_size,
            random_state=42,
            shuffle=True,
        )

        train_dataset = self._prepare_dataset(train_df)
        eval_dataset = self._prepare_dataset(test_df)

        train_dataset = train_dataset.with_transform(
            self._make_transform(augment=self.use_augmentation)
        )
        eval_dataset = eval_dataset.with_transform(
            self._make_transform(augment=False)
        )

        def collate_fn(batch):
            return {
                "pixel_values": torch.stack(
                    [ex["pixel_values"] for ex in batch]
                ),
                "labels": torch.tensor(
                    [ex["labels"] for ex in batch],
                    dtype=torch.float32,
                ),
            }

        self._setup_compute_metrics(metric_name=metric_name)

        greater_is_better = False if metric_name in {"rmse", "mae"} else True

        training_arguments = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=epochs,
            per_device_train_batch_size=per_device_train_batch_size,
            per_device_eval_batch_size=per_device_train_batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            learning_rate=learning_rate,
            warmup_ratio=0.1,
            lr_scheduler_type="cosine",
            weight_decay=0.01,
            eval_strategy="steps",
            save_strategy="steps",
            load_best_model_at_end=True,
            metric_for_best_model=metric_name,
            greater_is_better=greater_is_better,
            save_total_limit=1,
            logging_strategy="steps",
            logging_steps=logging_steps,
            eval_steps=eval_steps,
            save_steps=eval_steps,
            report_to="none",
            fp16=fp16 and torch.cuda.is_available(),
            bf16=False,
            remove_unused_columns=False,
            dataloader_num_workers=2,
            dataloader_pin_memory=True,
        )

        self.trainer = Trainer(
            model=self.model,
            args=training_arguments,
            train_dataset=train_dataset,
            eval_dataset=eval_dataset,
            tokenizer=self.image_processor,
            compute_metrics=self.compute_metrics,
            data_collator=collate_fn,
        )

        last_ckpt = (
            get_last_checkpoint(output_dir) if os.path.isdir(output_dir) else None
        )

        self.trainer.train(resume_from_checkpoint=last_ckpt)
        return self

    def predict(self, df):
        """
        Предсказание численных значений для набора изображений.

        :param df: pandas.DataFrame с колонкой путей к изображениям.
        :return: numpy.array с предсказаниями формы (n_samples, num_labels);
                 если num_labels == 1, вернется вектор формы (n_samples,).
        """
        if self.trainer is None:
            raise RuntimeError(
                "Модель еще не обучена. Вызовите .fit() перед предсказанием."
            )

        df_copy = df.copy()
        # Создаем фиктивные таргеты для унифицированной подготовки датасета
        for col in self.target_columns_names:
            df_copy[col] = 0.0

        predict_dataset = self._prepare_dataset(df_copy)
        predict_dataset = predict_dataset.with_transform(
            self._make_transform(augment=False)
        )

        predictions = self.trainer.predict(predict_dataset)
        preds = np.array(predictions.predictions)

        # Для единичного таргета вернем 1D-вектор
        if preds.ndim == 2 and preds.shape[1] == 1:
            preds = preds.squeeze(1)

        return preds

    def get_embeddings(self, df, batch_size=32):
        """
        Извлечение эмбеддингов базовой модели для каждого изображения.

        :param df: pandas.DataFrame с колонкой путей к изображениям.
        :param batch_size: Размер батча.
        :return: numpy.array формы (n_samples, hidden_size) с эмбеддингами.
        """
        if self.trainer is None:
            raise RuntimeError(
                "Модель должна быть обучена. Вызовите .fit() перед извлечением эмбеддингов."
            )

        device = self.trainer.model.device
        self.trainer.model.eval()

        paths = df[self.image_path_column_name].tolist()
        all_embeddings = []

        for i in tqdm(
            range(0, len(paths), batch_size),
            desc="Извлечение эмбеддингов",
        ):
            batch_paths = paths[i: i + batch_size]

            batch_images = [
                Image.open(p)
                .convert("RGB")
                .resize(self.target_size, Image.Resampling.BICUBIC)
                for p in batch_paths
            ]

            inputs = self.image_processor(
                batch_images,
                return_tensors="pt",
            ).to(device)

            with torch.no_grad():
                base_model_prefix = self.trainer.model.base_model_prefix
                base_model = getattr(self.trainer.model, base_model_prefix)

                outputs = base_model(
                    **inputs,
                    return_dict=True,
                )

                if (
                    hasattr(outputs, "pooler_output")
                    and outputs.pooler_output is not None
                ):
                    emb = outputs.pooler_output

                elif hasattr(outputs, "last_hidden_state"):
                    hs = outputs.last_hidden_state

                    if hs.ndim == 3:
                        emb = hs[:, 0, :]

                    elif hs.ndim == 4:
                        emb = hs.mean(dim=(2, 3))

                    else:
                        raise ValueError(
                            "Неизвестная форма last_hidden_state "
                            "для извлечения эмбеддингов."
                        )
                else:
                    raise ValueError(
                        "Модель не вернула подходящие тензоры "
                        "для эмбеддингов."
                    )

            all_embeddings.append(emb.cpu().numpy())

        return np.vstack(all_embeddings)

Пример использования.

In [ ]:
import os

if not os.path.exists('dummy_images'):
    os.makedirs('dummy_images')

image_paths = []
colors = [(255, 0, 0), (0, 255, 0), (0, 0, 255), (255, 255, 0)]
for i in range(20):
    path = f'dummy_images/img_{i}.png'
    color = colors[i % 4] 
    Image.new('RGB', (224, 224), color=color).save(path)
    image_paths.append(path)

data = {
    'image_path': image_paths,
    'target_1': [i for i in range(20)],
    'target_2': [i**2 for i in range(20)]
}
full_df = pd.DataFrame(data)

In [ ]:
pipeline = ImageRegression(
    checkpoint='facebook/deit-tiny-patch16-224',
    target_columns_names=['target_1', 'target_2'],  # можно передать и строку, и список — класс поддерживает оба варианта
    image_path_column_name='image_path',
    use_augmentation=True
)

pipeline.fit(
    train_data=full_df,
    epochs=2,
    per_device_train_batch_size=4,
    fp16=False,
    logging_steps=2,
    eval_steps=4
)

test_df = full_df.head(5).copy()
preds = pipeline.predict(test_df)

print(preds)

embeddings = pipeline.get_embeddings(test_df, batch_size=5)
print(f"Форма массива эмбеддингов: {embeddings.shape}")

# Дообучение классификатора, который работает с данными разной модальностью.

In [ ]:
!pip install wav2clip torchaudio transformers evaluate pillow

import math
from typing import List, Dict, Any, Optional, Union

import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import evaluate
from transformers import Trainer, TrainingArguments
from transformers.modeling_outputs import SequenceClassifierOutput


def set_seed(seed: int = 42):
    """
    Устанавливает фиксированное зерно для Python, NumPy и PyTorch.

    :param seed: Значение зерна (по умолчанию 42).
    """
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


def to_pil(x: Union[str, np.ndarray, Image.Image]) -> Image.Image:
    """
    Приводит вход к PIL.Image в формате RGB.

    :param x: Путь к изображению, массив NumPy (H,W[,C]) либо PIL.Image.
    :return: PIL.Image в RGB.
    :raises ValueError: Если тип входа не поддерживается.
    """
    if isinstance(x, Image.Image):
        return x.convert("RGB")
    if isinstance(x, str):
        return Image.open(x).convert("RGB")
    if isinstance(x, np.ndarray):
        return Image.fromarray(x).convert("RGB")
    raise ValueError("Ожидается путь/np.ndarray/PIL.Image")


def load_audio(path: str, target_sr: int) -> np.ndarray:
    """
    Загружает аудиофайл и при необходимости ресемплирует до target_sr.

    :param path: Путь к аудиофайлу (wav/flac и т.п.).
    :param target_sr: Целевая частота дискретизации.
    :return: Одноканальный сигнал формы [T] в float32.
    :raises RuntimeError: Если не установлен torchaudio.
    """
    try:
        import torchaudio
    except Exception as e:
        raise RuntimeError("Требуется torchaudio: pip install torchaudio") from e
    waveform, sr = torchaudio.load(path)  # [C, T]
    if waveform.size(0) > 1:
        waveform = waveform.mean(dim=0, keepdim=True)
    if sr != target_sr:
        waveform = torchaudio.functional.resample(waveform, orig_freq=sr, new_freq=target_sr)
    return waveform.squeeze(0).numpy().astype(np.float32)


class MultiComboDataset(Dataset):
    """
    Датасет для различных комбинаций модальностей (text/image/audio).
    Теперь поддерживает несколько изображений и аудио в одном сэмпле (через списки или несколько колонок).

    :param df: Исходный датафрейм.
    :param target_col: Имя колонки с таргетом.
    :param label2id: Маппинг "значение метки -> id".
    :param text_columns: Список текстовых колонок (склеиваются). Если None/[], модальность 'text' не используется.
    :param image_columns: Список колонок с изображениями; каждая ячейка может быть одиночным значением или списком.
    :param audio_columns: Список колонок с аудио; каждая ячейка может быть одиночным значением или списком.
    """
    def __init__(
        self,
        df: pd.DataFrame,
        target_col: str,
        label2id: Dict[Any, int],
        text_columns: Optional[List[str]] = None,
        image_columns: Optional[List[str]] = None,
        audio_columns: Optional[List[str]] = None
    ):
        self.df = df.reset_index(drop=True)
        self.target_col = target_col
        self.label2id = label2id
        self.text_columns = text_columns or []
        self.image_columns = image_columns or []
        self.audio_columns = audio_columns or []
        self.sep = " [SEP] "

    def __len__(self) -> int:
        """
        :return: Количество элементов в датасете.
        """
        return len(self.df)

    def __getitem__(self, idx: int) -> Dict[str, Any]:
        """
        Возвращает элемент датасета.

        :param idx: Индекс строки.
        :return: Словарь с ключами:
                 'labels' (int), а также при наличии модальностей 'text', 'images', 'audios'.
        """
        row = self.df.iloc[idx]
        item = {"labels": int(self.label2id[row[self.target_col]]) if self.target_col in row else 0}
        if self.text_columns:
            item["text"] = self.sep.join([str(row[c]) if pd.notnull(row[c]) else "" for c in self.text_columns])

        def _as_list(v):
            if v is None or (isinstance(v, float) and np.isnan(v)):
                return []
            if isinstance(v, (list, tuple)):
                return list(v)
            return [v]

        if self.image_columns:
            imgs = []
            for c in self.image_columns:
                if c in row:
                    imgs.extend(_as_list(row[c]))
            item["images"] = imgs
        if self.audio_columns:
            auds = []
            for c in self.audio_columns:
                if c in row:
                    auds.extend(_as_list(row[c]))
            item["audios"] = auds
        return item


class BaseBackend(nn.Module):
    """
    Базовый класс бэкенда для единой мультимодальной модели.

    Атрибуты:
      - name: Название бэкенда.
      - supported: Набор поддерживаемых модальностей, например {'text','image'}.
      - embed_dim: Базовая размерность эмбеддингов модальностей.
      - out_dim_per_modality: Реальные выходные размерности по модальностям (с учётом агрегации/concat).
    """
    name: str = "base"
    supported: set = set()
    embed_dim: int = 0
    out_dim_per_modality: Dict[str, int] = {}

    def collate(self, batch: List[Dict[str, Any]]) -> Dict[str, Any]:
        """
        Собирает батч из списка элементов MultiComboDataset для текущего бэкенда.

        :param batch: Список элементов датасета.
        :return: Содержит 'labels' (torch.LongTensor) и 'backend_inputs' (dict тензоров/сырья).
        """
        raise NotImplementedError

    def encode(self, backend_inputs: Dict[str, Any], device: torch.device) -> Dict[str, torch.Tensor]:
        """
        Кодирует модальности и возвращает эмбеддинги по доступным модальностям.

        :param backend_inputs: Предобработанные входы (из collate).
        :param device: Девайс для инференса.
        :return: Словарь {'text':[B,*], 'image':[B,*], 'audio':[B,*]} по имеющимся модальностям.
        """
        raise NotImplementedError

    def freeze_all(self):
        """
        Замораживает все параметры бэкенда (requires_grad=False).
        """
        for p in self.parameters():
            p.requires_grad = False

    def get_out_dim(self, modality: str) -> int:
        """
        Возвращает реальную размерность выходного эмбеддинга по модальности.

        :param modality: Имя модальности ('text','image','audio').
        :return: Размерность выходного вектора для заданной модальности.
        """
        return self.out_dim_per_modality.get(modality, self.embed_dim)


class ClipBackend(BaseBackend):
    """
    Бэкенд на основе CLIP (HF) для пары модальностей: текст + изображение.
    Поддерживает несколько изображений на сэмпл через агрегацию (concat или mean).

    :param checkpoint: Имя/путь чекпоинта CLIP (HF Hub). По умолчанию 'openai/clip-vit-base-patch32'.
    :param max_length: Максимальная длина текстовых токенов.
    :param freeze: Замораживать ли веса CLIP (linear probing).
    :param max_images: Максимальное число изображений на сэмпл (для concat/mean агрегации).
    :param image_agg: Тип агрегации изображений ('concat' или 'mean').
    """
    name = "clip"
    supported = {"text", "image"}

    def __init__(
        self,
        checkpoint: str = "openai/clip-vit-base-patch32",
        max_length: int = 77,
        freeze: bool = True,
        max_images: int = 1,
        image_agg: str = "concat"
    ):
        super().__init__()
        from transformers import CLIPModel, CLIPProcessor
        self.model = CLIPModel.from_pretrained(checkpoint)
        self.processor = CLIPProcessor.from_pretrained(checkpoint)
        self.embed_dim = int(self.model.config.projection_dim)
        self.max_length = max_length
        self.max_images = int(max_images)
        self.image_agg = image_agg
        if freeze:
            self.freeze_all()
        img_out = self.embed_dim * self.max_images if self.image_agg == "concat" else self.embed_dim
        self.out_dim_per_modality = {"text": self.embed_dim, "image": img_out}

    def collate(self, batch: List[Dict[str, Any]]) -> Dict[str, Any]:
        """
        Подготавливает батч для CLIP. Робастен к отсутствующим ключам 'text'/'images'.

        :param batch: Элементы с ключами 'text', 'images', 'labels'.
        :return: 'labels' и 'backend_inputs' с полями:
                 'text_inputs' (dict тензоров для текста),
                 'image_inputs' (dict с pixel_values или None),
                 'image_counts' (LongTensor [B]).
        """
        labels = torch.tensor([b.get("labels", 0) for b in batch], dtype=torch.long)
        texts = [b.get("text", "") for b in batch]

        # Списки картинок per-сэмпл -> плоский список + счётчики
        images_lists = [b.get("images", []) for b in batch]
        flat_images, counts = [], []
        for lst in images_lists:
            lst = lst if isinstance(lst, list) else ([] if lst is None else [lst])
            counts.append(len(lst))
            for img in lst:
                flat_images.append(to_pil(img))

        text_inputs = self.processor(
            text=texts, padding=True, truncation=True,
            max_length=self.max_length, return_tensors="pt"
        )
        text_inputs = {k: v for k, v in text_inputs.items()}

        if len(flat_images):
            img_proc = self.processor(images=flat_images, return_tensors="pt")
            image_inputs = {"pixel_values": img_proc["pixel_values"]}
        else:
            image_inputs = {"pixel_values": None}

        backend_inputs = {
            "text_inputs": text_inputs,
            "image_inputs": image_inputs,
            "image_counts": torch.tensor(counts, dtype=torch.long)
        }
        return {"labels": labels, "backend_inputs": backend_inputs}

    def _concat_padded(self, embs: torch.Tensor, counts: List[int], max_k: int) -> torch.Tensor:
        """
        Конкатенирует до max_k эмбеддингов на сэмпл с паддингом нулями.

        :param embs: Плоские эмбеддинги изображений [M, D], где M = сумма counts.
        :param counts: Список количеств изображений на сэмпл (длина B).
        :param max_k: Максимальное число изображений на сэмпл.
        :return: Нормализованный тензор [B, D*max_k].
        """
        device = embs.device if embs is not None else torch.device("cpu")
        B = len(counts); D = self.embed_dim
        out = torch.zeros((B, D * max_k), device=device, dtype=embs.dtype if embs is not None else torch.float32)
        if embs is None:
            return out
        offset = 0
        for i, c in enumerate(counts):
            if c > 0:
                chunk = embs[offset:offset + c]
                take = chunk[:max_k]
                if take.size(0) < max_k:
                    pad = torch.zeros((max_k - take.size(0), D), device=device, dtype=embs.dtype)
                    take = torch.cat([take, pad], dim=0)
                out[i] = take.reshape(-1)
            offset += c
        return F.normalize(out, dim=-1)

    def _mean_pool(self, embs: torch.Tensor, counts: List[int]) -> torch.Tensor:
        """
        Усредняет эмбеддинги изображений на сэмпл.

        :param embs: Плоские эмбеддинги [M, D].
        :param counts: Количества изображений на сэмпл (длина B).
        :return: Нормализованный тензор [B, D].
        """
        device = embs.device if embs is not None else torch.device("cpu")
        B = len(counts); D = self.embed_dim
        out = torch.zeros((B, D), device=device, dtype=embs.dtype if embs is not None else torch.float32)
        if embs is None:
            return out
        offset = 0
        for i, c in enumerate(counts):
            if c > 0:
                out[i] = embs[offset:offset + c].mean(dim=0)
            offset += c
        return F.normalize(out, dim=-1)

    def encode(self, backend_inputs: Dict[str, Any], device: torch.device) -> Dict[str, torch.Tensor]:
        """
        Вычисляет эмбеддинги текста и изображений (с агрегацией изображений).

        :param backend_inputs: Тензоры CLIPProcessor для текста и картинок.
        :param device: Девайс.
        :return: {'text':[B,D], 'image':[B,D*max_images] или [B,D]} — L2-нормированные эмбеддинги.
        """
        ti = {k: v.to(device) for k, v in backend_inputs["text_inputs"].items()}
        text_z = self.model.get_text_features(input_ids=ti["input_ids"], attention_mask=ti["attention_mask"])
        text_z = F.normalize(text_z, dim=-1)

        counts = backend_inputs["image_counts"].tolist()
        pi = backend_inputs["image_inputs"]["pixel_values"]
        if pi is not None:
            pi = pi.to(device)
            img_flat = self.model.get_image_features(pixel_values=pi)
            img_flat = F.normalize(img_flat, dim=-1)
            if self.image_agg == "concat":
                img_z = self._concat_padded(img_flat, counts, self.max_images)
            else:
                img_z = self._mean_pool(img_flat, counts)
        else:
            if self.image_agg == "concat":
                img_z = torch.zeros((len(counts), self.embed_dim * self.max_images), device=device)
            else:
                img_z = torch.zeros((len(counts), self.embed_dim), device=device)

        return {"text": text_z, "image": img_z}


class ClapBackend(BaseBackend):
    """
    Бэкенд на основе CLAP (HF) для пары модальностей: текст + аудио.
    Поддерживает несколько аудио на сэмпл через агрегацию (concat или mean).

    :param checkpoint: Имя/путь чекпоинта CLAP (HF Hub). По умолчанию 'laion/clap-htsat-unfused'.
    :param freeze: Замораживать ли веса CLAP.
    :param max_audios: Максимальное число аудио на сэмпл (для concat/mean агрегации).
    :param audio_agg: Тип агрегации аудио ('concat' или 'mean').
    """
    name = "clap"
    supported = {"text", "audio"}

    def __init__(
        self,
        checkpoint: str = "laion/clap-htsat-unfused",
        freeze: bool = True,
        max_audios: int = 1,
        audio_agg: str = "concat"
    ):
        super().__init__()
        from transformers import ClapModel, ClapProcessor
        self.model = ClapModel.from_pretrained(checkpoint)
        self.processor = ClapProcessor.from_pretrained(checkpoint)
        self.embed_dim = int(getattr(self.model.config, "projection_dim", 512))
        sr = getattr(self.processor, "sampling_rate", None)
        if sr is None:
            fe = getattr(self.processor, "feature_extractor", None)
            sr = getattr(fe, "sampling_rate", 48000) if fe is not None else 48000
        self.sr = int(sr)
        self.max_audios = int(max_audios)
        self.audio_agg = audio_agg
        if freeze:
            self.freeze_all()
        aud_out = self.embed_dim * self.max_audios if self.audio_agg == "concat" else self.embed_dim
        self.out_dim_per_modality = {"text": self.embed_dim, "audio": aud_out}

    def collate(self, batch: List[Dict[str, Any]]) -> Dict[str, Any]:
        """
        Подготавливает батч для CLAP (робастно к пропускам, поддерживает списки аудио per-сэмпл).

        :param batch: элементы с 'text','audios','labels'
        :return: 'labels' и 'backend_inputs' с полями:
                 'text_inputs' (dict для текста),
                 'audio_inputs' (dict с input_features или None),
                 'audio_counts' (LongTensor [B]).
        """
        labels = torch.tensor([b.get("labels", 0) for b in batch], dtype=torch.long)
        texts = [b.get("text", "") for b in batch]

        audios_lists = [b.get("audios", []) for b in batch]
        flat_audios, counts = [], []
        for lst in audios_lists:
            lst = lst if isinstance(lst, list) else ([] if lst is None else [lst])
            counts.append(len(lst))
            for a in lst:
                if isinstance(a, str):
                    flat_audios.append(load_audio(a, self.sr))
                elif isinstance(a, np.ndarray):
                    flat_audios.append(a.astype(np.float32))
                else:
                    raise ValueError("CLAP ожидает путь к аудио или numpy.ndarray")

        text_inputs = self.processor(text=texts, padding=True, truncation=True, return_tensors="pt")
        text_inputs = {k: v for k, v in text_inputs.items()}

        if len(flat_audios):
            aud_proc = self.processor(audios=flat_audios, sampling_rate=self.sr, padding=True, return_tensors="pt")
            audio_inputs = {"input_features": aud_proc["input_features"]}
        else:
            audio_inputs = {"input_features": None}

        backend_inputs = {
            "text_inputs": text_inputs,
            "audio_inputs": audio_inputs,
            "audio_counts": torch.tensor(counts, dtype=torch.long)
        }
        return {"labels": labels, "backend_inputs": backend_inputs}

    def _concat_padded(self, embs: torch.Tensor, counts: List[int], max_k: int) -> torch.Tensor:
        """
        Конкатенирует до max_k эмбеддингов аудио на сэмпл с паддингом нулями.

        :param embs: Плоские эмбеддинги аудио [M, D], где M = сумма counts.
        :param counts: Список количеств аудио на сэмпл (длина B).
        :param max_k: Максимальное число аудио на сэмпл.
        :return: Нормализованный тензор [B, D*max_k].
        """
        device = embs.device if embs is not None else torch.device("cpu")
        B = len(counts); D = self.embed_dim
        out = torch.zeros((B, D * max_k), device=device, dtype=embs.dtype if embs is not None else torch.float32)
        if embs is None:
            return out
        offset = 0
        for i, c in enumerate(counts):
            if c > 0:
                chunk = embs[offset:offset + c]
                take = chunk[:max_k]
                if take.size(0) < max_k:
                    pad = torch.zeros((max_k - take.size(0), D), device=device, dtype=embs.dtype)
                    take = torch.cat([take, pad], dim=0)
                out[i] = take.reshape(-1)
            offset += c
        return F.normalize(out, dim=-1)

    def _mean_pool(self, embs: torch.Tensor, counts: List[int]) -> torch.Tensor:
        """
        Усредняет эмбеддинги аудио на сэмпл.

        :param embs: Плоские эмбеддинги [M, D].
        :param counts: Количества аудио на сэмпл (длина B).
        :return: Нормализованный тензор [B, D].
        """
        device = embs.device if embs is not None else torch.device("cpu")
        B = len(counts); D = self.embed_dim
        out = torch.zeros((B, D), device=device, dtype=embs.dtype if embs is not None else torch.float32)
        if embs is None:
            return out
        offset = 0
        for i, c in enumerate(counts):
            if c > 0:
                out[i] = embs[offset:offset + c].mean(dim=0)
            offset += c
        return F.normalize(out, dim=-1)

    def encode(self, backend_inputs: Dict[str, Any], device: torch.device) -> Dict[str, torch.Tensor]:
        """
        Вычисляет эмбеддинги текста и аудио (с агрегацией аудио).

        :param backend_inputs: Тензоры ClapProcessor для текста и аудио.
        :param device: Девайс.
        :return: {'text':[B,D], 'audio':[B,D*max_audios] или [B,D]} — L2-нормированные эмбеддинги.
        """
        ti = {k: v.to(device) for k, v in backend_inputs["text_inputs"].items()}
        text_z = self.model.get_text_features(input_ids=ti["input_ids"], attention_mask=ti["attention_mask"])
        text_z = F.normalize(text_z, dim=-1)

        counts = backend_inputs["audio_counts"].tolist()
        af = backend_inputs["audio_inputs"]["input_features"]
        if af is not None:
            af = af.to(device)
            aud_flat = self.model.get_audio_features(input_features=af)
            aud_flat = F.normalize(aud_flat, dim=-1)
            if self.audio_agg == "concat":
                aud_z = self._concat_padded(aud_flat, counts, self.max_audios)
            else:
                aud_z = self._mean_pool(aud_flat, counts)
        else:
            if self.audio_agg == "concat":
                aud_z = torch.zeros((len(counts), self.embed_dim * self.max_audios), device=device)
            else:
                aud_z = torch.zeros((len(counts), self.embed_dim), device=device)

        return {"text": text_z, "audio": aud_z}


class ClipWav2CLIPBackend(BaseBackend):
    """
    Бэкенд: CLIP (text+image) + Wav2CLIP (audio->CLIP пространство). Поддерживает ['text','image','audio'].
    Поддержка нескольких изображений и аудио на сэмпл через агрегацию (concat или mean).

    :param checkpoint: Чекпоинт CLIP (HF), по умолчанию 'openai/clip-vit-base-patch32'.
    :param max_length: Максимальная длина текстовых токенов.
    :param freeze: Замораживать ли веса CLIP.
    :param audio_sr: Частота дискретизации для аудио (для Wav2CLIP).
    :param max_images: Максимум изображений на сэмпл (для агрегации).
    :param max_audios: Максимум аудио на сэмпл (для агрегации).
    :param image_agg: Агрегация изображений ('concat' или 'mean').
    :param audio_agg: Агрегация аудио ('concat' или 'mean').
    """
    name = "clip_wav2clip"
    supported = {"text", "image", "audio"}

    def __init__(
        self,
        checkpoint: str = "openai/clip-vit-base-patch32",
        max_length: int = 77,
        freeze: bool = True,
        audio_sr: int = 16000,
        max_images: int = 1,
        max_audios: int = 1,
        image_agg: str = "concat",
        audio_agg: str = "concat"
    ):
        super().__init__()
        from transformers import CLIPModel, CLIPProcessor
        self.model = CLIPModel.from_pretrained(checkpoint)
        self.processor = CLIPProcessor.from_pretrained(checkpoint)
        self.embed_dim = int(self.model.config.projection_dim)
        self.max_length = max_length
        self.audio_sr = int(audio_sr)
        self.max_images = int(max_images)
        self.max_audios = int(max_audios)
        self.image_agg = image_agg
        self.audio_agg = audio_agg
        if freeze:
            self.freeze_all()
        img_out = self.embed_dim * self.max_images if self.image_agg == "concat" else self.embed_dim
        aud_out = self.embed_dim * self.max_audios if self.audio_agg == "concat" else self.embed_dim
        self.out_dim_per_modality = {"text": self.embed_dim, "image": img_out, "audio": aud_out}
        # wav2clip lazy
        self._w2c_model = None
        self._w2c_mod = None
        self._w2c_api = None
        self._w2c_device = None

    def _ensure_w2c(self, device: torch.device):
        """
        Подгружает wav2clip и модель с учётом разных API версий.

        :param device: torch.device для инференса.
        :raises RuntimeError: Если пакет wav2clip не найден или не удалось загрузить модель.
        """
        if self._w2c_model is not None and self._w2c_device == str(device):
            return
        import importlib
        try:
            w2c = importlib.import_module("wav2clip")
        except Exception as e:
            raise RuntimeError("Не найден пакет 'wav2clip'. Установите: pip install wav2clip") from e
        dev_str = str(device) if device.type == "cuda" else "cpu"
        if hasattr(w2c, "load_model"):
            model = w2c.load_model(device=dev_str); api_kind = "func"
        elif hasattr(w2c, "get_model"):
            model = w2c.get_model(device=dev_str); api_kind = "func"
        elif hasattr(w2c, "Wav2CLIP"):
            try:
                model = w2c.Wav2CLIP(dev_str)
            except TypeError:
                model = w2c.Wav2CLIP(device=dev_str)
            api_kind = "method"
        else:
            raise RuntimeError("wav2clip установлен, но нет способов загрузки (load_model/get_model/Wav2CLIP)")
        self._w2c_mod = w2c
        self._w2c_model = model
        self._w2c_api = api_kind
        self._w2c_device = str(device)

    def collate(self, batch: List[Dict[str, Any]]) -> Dict[str, Any]:
        """
        Готовит батч: CLIPProcessor для текста/картинки + список аудио (пути/массивы).

        :param batch: элементы с 'text','images','audios','labels' (text можно опустить — подставится пустая строка).
        :return: 'labels' и 'backend_inputs' с полями для текста/изображений/аудио и счётчиками.
        """
        labels = torch.tensor([b.get("labels", 0) for b in batch], dtype=torch.long)
        texts = [b.get("text", "") for b in batch]

        # images (flatten + counts)
        images_lists = [b.get("images", []) for b in batch]
        flat_images, img_counts = [], []
        for lst in images_lists:
            lst = lst if isinstance(lst, list) else ([] if lst is None else [lst])
            img_counts.append(len(lst))
            for img in lst:
                flat_images.append(to_pil(img))
        text_inputs = self.processor(text=texts, padding=True, truncation=True, max_length=self.max_length, return_tensors="pt")
        text_inputs = {k: v for k, v in text_inputs.items()}
        if len(flat_images):
            img_proc = self.processor(images=flat_images, return_tensors="pt")
            image_inputs = {"pixel_values": img_proc["pixel_values"]}
        else:
            image_inputs = {"pixel_values": None}

        # audios (flatten + counts)
        audios_lists = [b.get("audios", []) for b in batch]
        flat_audios, aud_counts = [], []
        for lst in audios_lists:
            lst = lst if isinstance(lst, list) else ([] if lst is None else [lst])
            aud_counts.append(len(lst))
            for a in lst:
                if isinstance(a, str):
                    flat_audios.append(load_audio(a, self.audio_sr))
                elif isinstance(a, np.ndarray):
                    flat_audios.append(a.astype(np.float32))
                else:
                    raise ValueError("Ожидается путь к аудио или numpy.ndarray")

        if len(flat_audios):
            Lmax = max(len(a) for a in flat_audios)
            wav = np.zeros((len(flat_audios), Lmax), dtype=np.float32)
            lens = np.zeros((len(flat_audios),), dtype=np.int64)
            for i, a in enumerate(flat_audios):
                L = len(a)
                wav[i, :L] = a
                lens[i] = L
            audio_inputs = {
                "waveforms": torch.from_numpy(wav),   # [M, Lmax]
                "lengths": torch.from_numpy(lens)     # [M]
            }
        else:
            audio_inputs = {"waveforms": None, "lengths": None}

        backend_inputs = {
            "text_inputs": text_inputs,
            "image_inputs": image_inputs,
            "image_counts": torch.tensor(img_counts, dtype=torch.long),
            "audio_inputs": audio_inputs,
            "audio_counts": torch.tensor(aud_counts, dtype=torch.long),
        }
        return {"labels": labels, "backend_inputs": backend_inputs}

    def _agg_concat(self, embs: torch.Tensor, counts: List[int], max_k: int) -> torch.Tensor:
        """
        Конкатенирует до max_k эмбеддингов на сэмпл с паддингом нулями.

        :param embs: Плоские эмбеддинги [M, D].
        :param counts: Список количеств элементов на сэмпл (длина B).
        :param max_k: Максимальное число элементов на сэмпл.
        :return: Нормализованный тензор [B, D*max_k].
        """
        device = embs.device if embs is not None else torch.device("cpu")
        B = len(counts); D = self.embed_dim
        out = torch.zeros((B, D * max_k), device=device, dtype=embs.dtype if embs is not None else torch.float32)
        if embs is None:
            return out
        offset = 0
        for i, c in enumerate(counts):
            if c > 0:
                chunk = embs[offset:offset+c]
                take = chunk[:max_k]
                if take.size(0) < max_k:
                    pad = torch.zeros((max_k - take.size(0), D), device=device, dtype=embs.dtype)
                    take = torch.cat([take, pad], dim=0)
                out[i] = take.reshape(-1)
            offset += c
        return F.normalize(out, dim=-1)

    def _agg_mean(self, embs: torch.Tensor, counts: List[int]) -> torch.Tensor:
        """
        Усредняет эмбеддинги на сэмпл.

        :param embs: Плоские эмбеддинги [M, D].
        :param counts: Количества элементов на сэмпл (длина B).
        :return: Нормализованный тензор [B, D].
        """
        device = embs.device if embs is not None else torch.device("cpu")
        B = len(counts); D = self.embed_dim
        out = torch.zeros((B, D), device=device, dtype=embs.dtype if embs is not None else torch.float32)
        if embs is None:
            return out
        offset = 0
        for i, c in enumerate(counts):
            if c > 0:
                out[i] = embs[offset:offset+c].mean(dim=0)
            offset += c
        return F.normalize(out, dim=-1)

    def encode(self, backend_inputs: Dict[str, Any], device: torch.device) -> Dict[str, torch.Tensor]:
        """
        Кодирует модальности в CLIP‑пространство (текст/изображение через CLIP, аудио через Wav2CLIP).

        :param backend_inputs: Входы для бэкенда.
        :param device: torch.device.
        :return: {'text':[B,*], 'image':[B,*], 'audio':[B,*]} (L2-нормированные эмбеддинги).
        """
        # text
        ti = {k: v.to(device) for k, v in backend_inputs["text_inputs"].items()}
        text_z = self.model.get_text_features(input_ids=ti["input_ids"], attention_mask=ti["attention_mask"])
        text_z = F.normalize(text_z, dim=-1)

        # image
        img_counts = backend_inputs["image_counts"].tolist()
        pi = backend_inputs["image_inputs"]["pixel_values"]
        if pi is not None:
            pi = pi.to(device)
            img_flat = self.model.get_image_features(pixel_values=pi)
            img_flat = F.normalize(img_flat, dim=-1)
            if self.image_agg == "concat":
                image_z = self._agg_concat(img_flat, img_counts, self.max_images)
            else:
                image_z = self._agg_mean(img_flat, img_counts)
        else:
            if self.image_agg == "concat":
                image_z = torch.zeros((len(img_counts), self.embed_dim * self.max_images), device=device)
            else:
                image_z = torch.zeros((len(img_counts), self.embed_dim), device=device)

        # audio via wav2clip
        aud_counts = backend_inputs["audio_counts"].tolist()
        wav = backend_inputs["audio_inputs"]["waveforms"]
        lens = backend_inputs["audio_inputs"]["lengths"]
        if wav is not None and lens is not None and (lens.numel() if torch.is_tensor(lens) else len(lens)) > 0:
            self._ensure_w2c(device)
            w2c = self._w2c_mod
            waves = wav  # [M, Lmax] (CPU тензор — нормально)
            lens_np = lens.cpu().numpy()
            embs = []
            for i in range(waves.size(0)):
                L = int(lens_np[i])
                a_np = waves[i, :L].detach().cpu().numpy()
                e = None
                if self._w2c_api == "func":
                    if hasattr(w2c, "embed_audio"):
                        try:
                            e = w2c.embed_audio(a_np, self._w2c_model)
                        except TypeError:
                            e = w2c.embed_audio(a_np, self.audio_sr, self._w2c_model)
                    elif hasattr(w2c, "get_audio_embedding"):
                        try:
                            e = w2c.get_audio_embedding(a_np, self._w2c_model)
                        except TypeError:
                            e = w2c.get_audio_embedding(a_np, self.audio_sr, self._w2c_model)
                if e is None and self._w2c_api == "method" and hasattr(self._w2c_model, "embed_audio"):
                    try:
                        e = self._w2c_model.embed_audio(a_np)
                    except TypeError:
                        e = self._w2c_model.embed_audio(a_np, sr=self.audio_sr)
                if e is None:
                    raise RuntimeError("Не удалось получить аудио‑эмбеддинг через wav2clip.")
                e = np.asarray(e)
                if e.ndim == 2:
                    e = e.mean(axis=0)
                elif e.ndim > 2:
                    e = e.reshape(-1, e.shape[-1]).mean(axis=0)
                embs.append(e.astype(np.float32))
            aud_flat = torch.tensor(np.stack(embs, axis=0), dtype=torch.float32, device=device)
            aud_flat = F.normalize(aud_flat, dim=-1)
            if self.audio_agg == "concat":
                audio_z = self._agg_concat(aud_flat, aud_counts, self.max_audios)
            else:
                audio_z = self._agg_mean(aud_flat, aud_counts)
        else:
            if self.audio_agg == "concat":
                audio_z = torch.zeros((len(aud_counts), self.embed_dim * self.max_audios), device=device)
            else:
                audio_z = torch.zeros((len(aud_counts), self.embed_dim), device=device)

        return {"text": text_z, "image": image_z, "audio": audio_z}


class SingleBackboneClassifier(nn.Module):
    """
    Классификатор поверх одного мультимодального бэкенда: фьюжн эмбеддингов -> MLP.

    :param backend: Инициализированный бэкенд (CLIP/CLAP/ClipWav2CLIP).
    :param modalities: Список активных модальностей; порядок учитывается при concat ('image','text','audio').
    :param num_labels: Количество классов.
    :param fusion: Тип фьюжна — 'concat' или 'mean'.
    :param hidden: Размер скрытого слоя головы.
    :param dropout: Дропаут в голове.
    """
    def __init__(
        self,
        backend: BaseBackend,
        modalities: List[str],
        num_labels: int,
        fusion: str = "concat",
        hidden: int = 512,
        dropout: float = 0.1
    ):
        super().__init__()
        self.backend = backend
        self.modalities = modalities
        self.fusion = fusion
        self.num_labels = num_labels

        # Реальная входная размерность головы с учётом агрегации по модальностям
        order = [m for m in ["image", "text", "audio"] if m in self.modalities]
        if fusion == "concat":
            in_dim = sum(self.backend.get_out_dim(m) for m in order)
        elif fusion == "mean":
            dims = [self.backend.get_out_dim(m) for m in order]
            if len(set(dims)) != 1:
                raise ValueError(f'Для fusion="mean" размеры модальностей должны совпадать, а у нас: {dict(zip(order, dims))}')
            in_dim = dims[0]
        else:
            raise ValueError('fusion должен быть "concat" или "mean"')

        self.head = nn.Sequential(
            nn.LayerNorm(in_dim),
            nn.Linear(in_dim, hidden),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, num_labels)
        )

    def _infer_device_from_inputs(self, obj) -> torch.device:
        """
        Рекурсивно находит девайс по первому попавшемуся тензору во входах.
        Если тензоры не найдены — отдаёт доступный cuda/cpu.

        :param obj: Произвольная структура входов (тензоры/словари/списки).
        :return: torch.device для выполнения.
        """
        if isinstance(obj, torch.Tensor):
            return obj.device
        if isinstance(obj, dict):
            for v in obj.values():
                d = self._infer_device_from_inputs(v)
                if d is not None:
                    return d
        return torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def _fuse(self, z: Dict[str, torch.Tensor]) -> torch.Tensor:
        """
        Объединяет эмбеддинги модальностей согласно self.fusion.

        :param z: Эмбеддинги по модальностям.
        :return: Слитый эмбеддинг [B, D*|mods|] для concat или [B, D] для mean.
        """
        order = [m for m in ["image", "text", "audio"] if m in self.modalities]
        feats = []
        for m in order:
            t = z[m]
            if t.dim() == 3:
                t = t.mean(dim=1)
            elif t.dim() > 3:
                t = t.view(t.size(0), -1)
            feats.append(t)
        if self.fusion == "concat":
            return torch.cat(feats, dim=-1)
        elif self.fusion == "mean":
            sizes = [f.size(-1) for f in feats]
            if len(set(sizes)) != 1:
                raise ValueError(f'Для fusion="mean" размеры модальностей должны совпадать, а у нас: {sizes}')
            return torch.stack(feats, dim=0).mean(dim=0)

    def forward(self, backend_inputs: Dict[str, Any], labels: Optional[torch.Tensor] = None):
        """
        Прямой проход: кодирование модальностей -> фьюжн -> классификационная голова.

        :param backend_inputs: Входы для бэкенда (из его collate).
        :param labels: Игнорируется (loss считает Trainer).
        :return: SequenceClassifierOutput с logits [B, num_labels].
        """
        device = self._infer_device_from_inputs(backend_inputs)
        z = self.backend.encode(backend_inputs, device=device)
        fused = self._fuse(z)
        logits = self.head(fused)
        return SequenceClassifierOutput(logits=logits)

    @torch.no_grad()
    def get_embeddings(self, backend_inputs: Dict[str, Any], return_per_modality: bool = False):
        """
        Извлекает fused эмбеддинги (и опционально per-modality).

        :param backend_inputs: Входы для бэкенда.
        :param return_per_modality: Вернуть также словарь эмбеддингов по модальностям.
        :return: fused [B, *] или (fused, {'text':[B,*], 'image':[B,*], 'audio':[B,*]}).
        """
        device = self._infer_device_from_inputs(backend_inputs)
        z = self.backend.encode(backend_inputs, device=device)
        fused = self._fuse(z)
        if return_per_modality:
            return fused, z
        return fused


class WeightedCETrainer(Trainer):
    """
    Trainer с CrossEntropyLoss и поддержкой весов классов.
    Умеет автоматически вычислять class_weights по частотам классов, если они не переданы.

    :param num_labels: Количество классов.
    :param train_labels: Список/массив меток train (для авто-вычисления весов).
    :param class_weights: Веса классов (np.ndarray/список длины num_labels) или None.
    """
    def __init__(self, *args, num_labels=None, train_labels=None, class_weights=None, **kwargs):
        super().__init__(*args, **kwargs)
        self.num_labels = num_labels
        if class_weights is not None:
            w = torch.as_tensor(class_weights, dtype=torch.float32)
        else:
            w = None
            if train_labels is not None and num_labels is not None:
                train_labels = np.asarray(train_labels).astype(int)
                counts = np.bincount(train_labels, minlength=num_labels)
                n = counts.sum()
                weights = np.zeros(num_labels, dtype=np.float32)
                nonzero = counts > 0
                weights[nonzero] = n / (num_labels * counts[nonzero].astype(np.float32))
                w = torch.tensor(weights, dtype=torch.float32)
        self.class_weights = w
        self._warned_label_tiling = False

    def compute_loss(self, model, inputs, return_outputs: bool = False, num_items_in_batch: Optional[int] = None):
        """
        Считает CrossEntropyLoss с опциональными весами классов.

        :param model: Модель.
        :param inputs: Батч, содержащий 'labels' (LongTensor) и аргументы для model.forward.
        :param return_outputs: Возвращать ли также outputs.
        :param num_items_in_batch: Совместимость с Trainer API (не используется).
        :return: loss (и outputs, если return_outputs=True).
        :raises ValueError: Если размеры logits и labels не совпадают (с учётом DP).
        """
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        logits = outputs.logits

        # Переносим метки на тот же девайс, что и логиты
        labels = labels.to(logits.device)

        # Защита от DP-дублирования батча
        if logits.size(0) != labels.size(0):
            ngpu = torch.cuda.device_count()
            if ngpu > 1 and logits.size(0) == labels.size(0) * ngpu:
                # Дублируем метки по числу GPU
                labels = labels.repeat_interleave(ngpu)
                if not self._warned_label_tiling:
                    print(f"[Warning] DataParallel удвоил batch для logits. "
                          f"Повторяем labels x{ngpu} для выравнивания. "
                          f"logits: {tuple(logits.shape)}, labels: {tuple(labels.shape)}")
                    self._warned_label_tiling = True
            else:
                raise ValueError(f"Batch size mismatch: logits {tuple(logits.shape)} vs labels {tuple(labels.shape)}")

        weight = self.class_weights.to(logits.device) if self.class_weights is not None else None
        loss = nn.CrossEntropyLoss(weight=weight)(logits, labels.long())
        return (loss, outputs) if return_outputs else loss


class SingleModelMultiComboClassification:
    """
    Пайплайн с одной мультимодальной моделью (бэкендом) под заданную комбинацию модальностей.

    Поддерживаемые комбинации:
      - ['text','image']         -> CLIP (HF)
      - ['text','audio']         -> CLAP (HF)
      - ['image','audio']        -> ClipWav2CLIP
      - ['text','image','audio'] -> ClipWav2CLIP

    Поддерживает мульти-изображения/аудио с concat-агрегацией (паддинг до max_*).

    :param modalities: Активные модальности ('text','image','audio') в любом порядке.
    :param num_labels: Количество классов.
    :param target_column_name: Имя столбца таргета в DataFrame.
    :param text_columns: Имена текстовых колонок (склеиваются).
    :param image_columns: Список колонок с изображениями (ячейки — одиночные значения или списки).
    :param audio_columns: Список колонок с аудио (ячейки — одиночные значения или списки).
    :param backend: 'auto' | 'clip' | 'clap' | 'clip_wav2clip'. 'auto' подбирает по комбинации модальностей.
    :param clip_checkpoint: Чекпоинт CLIP (HF). По умолчанию 'openai/clip-vit-base-patch32'.
    :param clap_checkpoint: Чекпоинт CLAP (HF). По умолчанию 'laion/clap-htsat-unfused'.
    :param fusion: Тип фьюжна ('concat' или 'mean').
    :param freeze_backbone: Заморозить веса бэкенда (linear probing).
    :param clip_max_length: Максимальная длина токенов для CLIP.
    :param max_images_per_sample: Максимум изображений на сэмпл при агрегации.
    :param max_audios_per_sample: Максимум аудио на сэмпл при агрегации.
    """
    def __init__(
        self,
        modalities: List[str],
        num_labels: int,
        target_column_name: str,
        text_columns: Optional[List[str]] = None,
        image_columns: Optional[List[str]] = None,
        audio_columns: Optional[List[str]] = None,
        backend: str = "auto",
        clip_checkpoint: str = "openai/clip-vit-base-patch32",
        clap_checkpoint: str = "laion/clap-htsat-unfused",
        fusion: str = "concat",
        freeze_backbone: bool = True,
        clip_max_length: int = 77,
        max_images_per_sample: int = 1,
        max_audios_per_sample: int = 1
    ):
        self.modalities = sorted(list(set(modalities)))
        self.num_labels = num_labels
        self.target_column_name = target_column_name
        self.text_columns = text_columns or []
        self.image_columns = image_columns or []
        self.audio_columns = audio_columns or []
        self.backend_name = backend
        self.clip_checkpoint = clip_checkpoint
        self.clap_checkpoint = clap_checkpoint
        self.fusion = fusion
        self.freeze_backbone = freeze_backbone
        self.clip_max_length = clip_max_length
        self.max_images_per_sample = int(max_images_per_sample)
        self.max_audios_per_sample = int(max_audios_per_sample)

        self.label2id: Optional[Dict[Any, int]] = None
        self.id2label: Optional[Dict[int, str]] = None

        self.backend: Optional[BaseBackend] = None
        self.model: Optional[SingleBackboneClassifier] = None
        self.trainer: Optional[Trainer] = None
        self.compute_metrics = None

        self._build_backend()

    def _build_backend(self):
        """
        Инициализирует бэкенд согласно backend/auto и проверяет совместимость с выбранными модальностями.

        :raises ValueError: Если комбинация модальностей не поддерживается выбранным бэкендом.
        """
        mods = set(self.modalities)
        name = self.backend_name
        if name == "auto":
            if mods == {"text", "image"}:
                name = "clip"
            elif mods == {"text", "audio"}:
                name = "clap"
            elif mods in ({"image", "audio"}, {"text", "image", "audio"}):
                name = "clip_wav2clip"
            else:
                raise ValueError(f"Неподдерживаемая комбинация: {mods}")

        if name == "clip":
            self.backend = ClipBackend(
                checkpoint=self.clip_checkpoint,
                max_length=self.clip_max_length,
                freeze=self.freeze_backbone,
                max_images=self.max_images_per_sample,
                image_agg="concat"
            )
        elif name == "clap":
            self.backend = ClapBackend(
                checkpoint=self.clap_checkpoint,
                freeze=self.freeze_backbone,
                max_audios=self.max_audios_per_sample,
                audio_agg="concat"
            )
        elif name == "clip_wav2clip":
            self.backend = ClipWav2CLIPBackend(
                checkpoint=self.clip_checkpoint,
                max_length=self.clip_max_length,
                freeze=self.freeze_backbone,
                audio_sr=16000,
                max_images=self.max_images_per_sample,
                max_audios=self.max_audios_per_sample,
                image_agg="concat",
                audio_agg="concat"
            )
        else:
            raise ValueError(f"Неизвестный backend: {name}")

        if not set(self.modalities).issubset(self.backend.supported):
            raise ValueError(f"Бэкенд {self.backend.name} не поддерживает модальности {self.modalities}. Поддерживает: {self.backend.supported}")

    def _setup_metrics(self, metric_name: str):
        """
        Создаёт функцию подсчёта метрик для Trainer.

        :param metric_name: 'f1' или 'accuracy'.
        :raises ValueError: Если метрика не поддерживается.
        """
        metric_name = metric_name.lower()
        if metric_name == "f1":
            metric = evaluate.load("f1")
            def compute(p):
                preds = p.predictions.argmax(-1)
                return metric.compute(predictions=preds, references=p.label_ids, average="weighted")
        elif metric_name == "accuracy":
            metric = evaluate.load("accuracy")
            def compute(p):
                preds = p.predictions.argmax(-1)
                return metric.compute(predictions=preds, references=p.label_ids)
        else:
            raise ValueError('metric_name должен быть "f1" или "accuracy"')
        self.compute_metrics = compute

    def _split(self, df: pd.DataFrame, test_size: float = 0.2, seed: int = 42):
        """
        Перемешивает и делит DataFrame на train/eval.

        :param df: Исходный датафрейм.
        :param test_size: Доля валидации (0..1).
        :param seed: Зерно для перемешивания.
        :return: (df_train, df_eval).
        """
        df = df.sample(frac=1.0, random_state=seed).reset_index(drop=True)
        n_eval = int(math.ceil(len(df) * test_size))
        return df.iloc[n_eval:].reset_index(drop=True), df.iloc[:n_eval].reset_index(drop=True)

    def _validate_data(self, df: pd.DataFrame):
        """
        Проверяет соответствие колонок выбранным модальностям.
        Бросает ValueError с понятной подсказкой, если что-то не так.

        :param df: Исходный датафрейм.
        :raises ValueError: При отсутствии нужных колонок/настроек.
        """
        if "text" in self.modalities:
            if not self.text_columns:
                raise ValueError("Вы выбрали модальность 'text', но text_columns пустой.")
            missing = [c for c in self.text_columns if c not in df.columns]
            if missing:
                raise ValueError(f"В DataFrame отсутствуют текстовые колонки: {missing}")

        if "image" in self.modalities:
            if not self.image_columns:
                raise ValueError("Вы выбрали модальность 'image', но image_columns пуст.")
            missing = [c for c in self.image_columns if c not in df.columns]
            if missing:
                raise ValueError(f"В DataFrame отсутствуют колонки изображений: {missing}")

        if "audio" in self.modalities:
            if not self.audio_columns:
                raise ValueError("Вы выбрали модальность 'audio', но audio_columns пуст.")
            missing = [c for c in self.audio_columns if c not in df.columns]
            if missing:
                raise ValueError(f"В DataFrame отсутствуют колонки аудио: {missing}")

    def fit(
        self,
        train_data: pd.DataFrame,
        epochs: int = 3,
        test_size: float = 0.2,
        per_device_train_batch_size: int = 16,
        gradient_accumulation_steps: int = 1,
        learning_rate: float = 2e-4,
        metric_name: str = "f1",
        fp16: bool = True,
        logging_steps: int = 50,
        eval_steps: int = 200,
        output_dir: str = "./result",
        seed: int = 42,
        hidden: int = 512,
        dropout: float = 0.1
    ):
        """
        Обучает классификационную голову поверх выбранного бэкенда.

        :param train_data: Данные обучения с нужными колонками.
        :param epochs: Количество эпох.
        :param test_size: Доля валидации.
        :param per_device_train_batch_size: Размер батча на устройство.
        :param gradient_accumulation_steps: Градиентная аккумуляция.
        :param learning_rate: Learning rate для AdamW.
        :param metric_name: 'f1' или 'accuracy' для ранжирования чекпоинтов.
        :param fp16: Использовать ли fp16 при наличии CUDA.
        :param logging_steps: Шаг логирования.
        :param eval_steps: Шаг валидации/сохранения.
        :param output_dir: Папка для логов/чекпоинтов.
        :param seed: Зерно.
        :param hidden: Размер скрытого слоя головы.
        :param dropout: Дропаут в голове.
        :return: self.
        :raises ValueError: При проблемах с данными/параметрами.
        """
        self._validate_data(train_data)
        set_seed(seed)

        classes = sorted(train_data[self.target_column_name].unique().tolist())
        if self.num_labels != len(classes):
            print(f"Warning: num_labels={self.num_labels} != len(classes)={len(classes)}")
        self.label2id = {c: i for i, c in enumerate(classes)}
        self.id2label = {i: str(c) for c, i in self.label2id.items()}

        df_train, df_eval = self._split(train_data, test_size=test_size, seed=seed)
        ds_train = MultiComboDataset(df_train, self.target_column_name, self.label2id, self.text_columns, self.image_columns, self.audio_columns)
        ds_eval  = MultiComboDataset(df_eval,  self.target_column_name, self.label2id, self.text_columns, self.image_columns, self.audio_columns)

        y_train = np.array([self.label2id[y] for y in df_train[self.target_column_name].tolist()], dtype=int)
        counts = np.bincount(y_train, minlength=self.num_labels)
        n = counts.sum()
        class_weights = np.zeros(self.num_labels, dtype=np.float32)
        nonzero = counts > 0
        class_weights[nonzero] = n / (self.num_labels * counts[nonzero].astype(np.float32))

        self.model = SingleBackboneClassifier(
            backend=self.backend,
            modalities=self.modalities,
            num_labels=self.num_labels,
            fusion=self.fusion,
            hidden=hidden,
            dropout=dropout
        )

        self._setup_metrics(metric_name)

        args = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=epochs,
            per_device_train_batch_size=per_device_train_batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            learning_rate=learning_rate,
            warmup_ratio=0.1,
            lr_scheduler_type="cosine",
            weight_decay=0.01,
            eval_strategy="steps",
            eval_steps=eval_steps,
            save_strategy="steps",
            save_steps=eval_steps,
            load_best_model_at_end=True,
            metric_for_best_model=f"eval_{metric_name}",
            save_total_limit=1,
            logging_strategy="steps",
            logging_steps=logging_steps,
            report_to="none",
            fp16=fp16 and torch.cuda.is_available(),
            dataloader_num_workers=0,
            seed=seed,
            remove_unused_columns=False
        )

        def data_collator(batch_list):
            """
            Хук для Trainer: вызывает backend.collate.

            :param batch_list: Элементы из датасета.
            :return: Батч для model.forward() (labels, backend_inputs).
            """
            return self.backend.collate(batch_list)

        self.trainer = WeightedCETrainer(
            model=self.model,
            args=args,
            train_dataset=ds_train,
            eval_dataset=ds_eval,
            data_collator=data_collator,
            compute_metrics=self.compute_metrics,
            num_labels=self.num_labels,
            train_labels=y_train,
            class_weights=class_weights
        )

        self.trainer.train()
        return self

    def predict(self, df: pd.DataFrame, return_label_str: bool = False) -> np.ndarray:
        """
        Делает предсказания классов на новых данных.

        :param df: Датафрейм с нужными колонками модальностей.
        :param return_label_str: Вернуть строковые метки (True) или id (False).
        :return: Вектор предсказанных меток (id или строки).
        :raises RuntimeError: Если модель ещё не обучена.
        """
        if self.trainer is None:
            raise RuntimeError("Модель не обучена. Вызовите .fit().")
        df_c = df.copy()
        if self.target_column_name not in df_c.columns:
            df_c[self.target_column_name] = 0
        ds = MultiComboDataset(df_c, self.target_column_name, self.label2id, self.text_columns, self.image_columns, self.audio_columns)
        preds = self.trainer.predict(test_dataset=ds)
        y_pred = np.argmax(preds.predictions, axis=-1)
        if return_label_str:
            return np.array([self.id2label[int(i)] for i in y_pred])
        return y_pred

    def get_embeddings(self, df: pd.DataFrame, batch_size: int = 32, return_per_modality: bool = False):
        """
        Извлекает эмбеддинги для новых данных.

        :param df: Датафрейм с нужными колонками модальностей.
        :param batch_size: Размер батча для извлечения эмбеддингов.
        :param return_per_modality: Вернуть также эмбеддинги по модальностям.
        :return: fused эмбеддинги [N, D_fused] или (fused, {'text':[N,*], 'image':[N,*], 'audio':[N,*]}).
        :raises RuntimeError: Если модель ещё не обучена.
        """
        if self.trainer is None or self.model is None:
            raise RuntimeError("Модель не обучена. Вызовите .fit().")

        device = getattr(self.trainer.args, "device", None)
        if device is None:
            try:
                device = next(self.trainer.model.parameters()).device
            except StopIteration:
                device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.model.to(device)
        self.model.eval()

        df_c = df.copy()
        if self.target_column_name not in df_c.columns:
            df_c[self.target_column_name] = 0

        ds = MultiComboDataset(
            df_c,
            self.target_column_name,
            self.label2id,
            self.text_columns,
            self.image_columns,
            self.audio_columns
        )

        def collate(batch_list):
            """
            Collate для DataLoader при извлечении эмбеддингов.

            :param batch_list: Элементы из датасета.
            :return: Батч (labels, backend_inputs).
            """
            return self.backend.collate(batch_list)

        def move_to_device(obj, device):
            """
            Рекурсивно переносит тензоры на нужный девайс.

            :param obj: Тензор/словарь/список/кортеж/прочее.
            :param device: torch.device.
            :return: Объект с перенесёнными тензорами.
            """
            if torch.is_tensor(obj):
                return obj.to(device)
            if isinstance(obj, dict):
                return {k: move_to_device(v, device) for k, v in obj.items()}
            if isinstance(obj, (list, tuple)):
                t = [move_to_device(v, device) for v in obj]
                return type(obj)(t) if not isinstance(obj, list) else t
            return obj

        loader = DataLoader(ds, batch_size=batch_size, shuffle=False, collate_fn=collate)
        fused_list = []
        per_mod_lists = {m: [] for m in self.modalities} if return_per_modality else None

        with torch.no_grad():
            for batch in loader:
                bi = move_to_device(batch["backend_inputs"], device)
                fused, per = self.model.get_embeddings(backend_inputs=bi, return_per_modality=True)
                fused_list.append(fused.cpu().numpy())
                if return_per_modality:
                    for m in per_mod_lists.keys():
                        if m in per:
                            per_mod_lists[m].append(per[m].cpu().numpy())

        fused_arr = np.vstack(fused_list)
        if not return_per_modality:
            return fused_arr
        per_mod = {m: np.vstack(chunks) for m, chunks in per_mod_lists.items()}
        return fused_arr, per_mod

In [ ]:
import os, random
import numpy as np
import pandas as pd
from PIL import Image

import torch
import torchaudio

HAVE_TORCHAUDIO = True

random.seed(42); np.random.seed(42); torch.manual_seed(42)

BASE_DIR = "./dummy_data"
IMG_DIR  = os.path.join(BASE_DIR, "images")
AUD_DIR  = os.path.join(BASE_DIR, "audio")
os.makedirs(IMG_DIR, exist_ok=True)
os.makedirs(AUD_DIR, exist_ok=True)

# Сколько элементов на модальность в каждой строке
K_PER_MODALITY = 3

def make_dummy_images(n=12, size=(256, 256)):
    paths = []
    for i in range(n):
        color = tuple(np.random.randint(0, 255, size=3).tolist())
        img = Image.new("RGB", size, color=color)
        path = os.path.join(IMG_DIR, f"img_{i:02d}.png")
        img.save(path)
        paths.append(path)
    return paths

def make_dummy_audios(n=12, sr=48000, duration_sec=0.6):
    if not HAVE_TORCHAUDIO:
        raise RuntimeError("Нельзя сгенерировать аудио без torchaudio. Установите 'pip install torchaudio'.")
    paths = []
    t = torch.linspace(0, duration_sec, int(sr * duration_sec))
    for i in range(n):
        freq = random.choice([220, 330, 440, 550, 660, 880])
        wave = 0.2 * torch.sin(2 * np.pi * freq * t)  # амплитуда 0.2
        wave = wave.unsqueeze(0)  # [1, T] mono
        path = os.path.join(AUD_DIR, f"tone_{i:02d}.wav")
        torchaudio.save(path, wave, sample_rate=sr)
        paths.append(path)
    return paths

img_paths = make_dummy_images(n=12)
audio_paths = make_dummy_audios(n=12) if HAVE_TORCHAUDIO else []

# Вспомогательные тексты
TITLES = ["Red fox", "Blue sky", "Green field", "Yellow sun", "Purple rain", "Silver line"]
BODIES = ["quick brown", "lazy dog", "jumps high", "runs fast", "stays calm", "shines bright"]
QUERIES = ["find tone", "classify sound", "describe image", "retrieve pair", "detect event"]

def rand_title(): return random.choice(TITLES)
def rand_body(): return random.choice(BODIES)
def rand_query(): return random.choice(QUERIES)

# Универсальные хелперы
def sample_k(seq, k):
    if len(seq) >= k:
        return random.sample(seq, k)  # без повторов
    else:
        return [random.choice(seq) for _ in range(k)]  # с повторами, если мало исходников

def as_cols(prefix, values):
    # {"prefix_1": values[0], ..., "prefix_k": values[k-1]}
    return {f"{prefix}_{i+1}": v for i, v in enumerate(values)}

def pick_text_desc(k=K_PER_MODALITY):
    # Текстовое описание: "Title | body"
    vals = [f"{rand_title()} | {rand_body()}" for _ in range(k)]
    return as_cols("text", vals)

def pick_text_query(k=K_PER_MODALITY):
    vals = [rand_query() for _ in range(k)]
    return as_cols("text", vals)

def pick_images(k=K_PER_MODALITY):
    vals = sample_k(img_paths, k)
    return as_cols("image_path", vals)

def pick_audios(k=K_PER_MODALITY):
    vals = sample_k(audio_paths, k)
    return as_cols("audio_path", vals)

# 1) Текст + Картинка -> по 3 текстовых и 3 картинок
def build_df_text_image(n=24, k=K_PER_MODALITY):
    rows = []
    for _ in range(n):
        row = {}
        row.update(pick_text_desc(k))
        row.update(pick_images(k))
        row["label"] = random.choice(["class_a", "class_b", "class_c"])
        rows.append(row)
    return pd.DataFrame(rows)

# 2) Текст + Звук -> по 3 текста (query) и 3 аудио
def build_df_text_audio(n=24, k=K_PER_MODALITY):
    if not HAVE_TORCHAUDIO:
        raise RuntimeError("Для text+audio нужен torchaudio.")
    rows = []
    for _ in range(n):
        row = {}
        row.update(pick_text_query(k))
        row.update(pick_audios(k))
        row["label"] = random.choice(["ok", "ng"])
        rows.append(row)
    return pd.DataFrame(rows)

# 3) Картинка + Звук -> по 3 картинки и 3 аудио
def build_df_image_audio(n=24, k=K_PER_MODALITY):
    if not HAVE_TORCHAUDIO:
        raise RuntimeError("Для image+audio нужен torchaudio.")
    rows = []
    for _ in range(n):
        row = {}
        row.update(pick_images(k))
        row.update(pick_audios(k))
        row["label"] = random.choice(["dog", "cat", "bird"])
        rows.append(row)
    return pd.DataFrame(rows)

# 4) Текст + Картинка + Звук -> по 3 на каждую модальность
def build_df_text_image_audio(n=24, k=K_PER_MODALITY):
    if not HAVE_TORCHAUDIO:
        raise RuntimeError("Для text+image+audio нужен torchaudio.")
    rows = []
    for _ in range(n):
        row = {}
        row.update(pick_text_desc(k))
        row.update(pick_images(k))
        row.update(pick_audios(k))
        row["label"] = random.choice(["A", "B", "C", "D"])
        rows.append(row)
    return pd.DataFrame(rows)

# Собираем 4 датасета
df_text_image = build_df_text_image(24, K_PER_MODALITY)
df_text_audio = build_df_text_audio(24, K_PER_MODALITY) if HAVE_TORCHAUDIO else None
df_image_audio = build_df_image_audio(24, K_PER_MODALITY) if HAVE_TORCHAUDIO else None
df_text_image_audio  = build_df_text_image_audio(24, K_PER_MODALITY) if HAVE_TORCHAUDIO else None

print("df_text_image columns:", list(df_text_image.columns))
if HAVE_TORCHAUDIO:
    print("df_text_audio columns:", list(df_text_audio.columns))
    print("df_image_audio columns:", list(df_image_audio.columns))
    print("df_text_image_audio columns:", list(df_text_image_audio.columns))

Пример использования на данных текст + картинка.

In [ ]:
pipeline = SingleModelMultiComboClassification(
    modalities=["text", "image"],
    num_labels=3,
    target_column_name="label",
    text_columns=["text_1", "text_2"],
    image_columns=["image_path_1", "image_path_2", "image_path_3"],
    backend="auto",
    fusion="concat",
    freeze_backbone=True,
    max_images_per_sample=2
)
pipeline.fit(df_text_image, epochs=2, per_device_train_batch_size=8, logging_steps=2, eval_steps=4)
preds = pipeline.predict(df_text_image.head(5), return_label_str=True)
emb = pipeline.get_embeddings(df_text_image.head(8), batch_size=8)
print(preds)
print(emb.shape)

Пример использования на данных текст + звук.

In [ ]:
pipeline = SingleModelMultiComboClassification(
    modalities=["text", "audio"],
    num_labels=2,
    target_column_name="label",
    text_columns=["text_1"],
    audio_columns=["audio_path_1"],
    backend="auto",
    fusion="concat",
    freeze_backbone=True,
    max_audios_per_sample=1
)
pipeline.fit(df_text_audio, epochs=1, per_device_train_batch_size=8)
preds = pipeline.predict(df_text_audio.head(5), return_label_str=True)
emb = pipeline.get_embeddings(df_text_audio.head(8), batch_size=8)
print(preds)
print(emb.shape)

Пример использования на данных картинка + звук.

In [ ]:
pipeline = SingleModelMultiComboClassification(
    modalities=["image", "audio"],
    num_labels=3,
    target_column_name="label",
    image_columns=["image_path_1"],
    audio_columns=["audio_path_1", "audio_path_2"],
    backend="auto",
    fusion="concat",
    freeze_backbone=True,
    max_images_per_sample=1,
    max_audios_per_sample=1
)
pipeline.fit(df_image_audio, epochs=1, per_device_train_batch_size=8)
preds = pipeline.predict(df_image_audio.head(5), return_label_str=True)
emb = pipeline.get_embeddings(df_image_audio.head(8), batch_size=8)
print(preds)
print(emb)

Пример использования на данных текст + картинка + звук.

In [ ]:
pipeline = SingleModelMultiComboClassification(
    modalities=["text", "image", "audio"],
    num_labels=4,
    target_column_name="label",
    text_columns=["text_1"],
    image_columns=["image_path_1", "image_path_2"],
    audio_columns=["audio_path_1"],
    backend="auto",
    fusion="concat",
    freeze_backbone=True,
    max_images_per_sample=2,
    max_audios_per_sample=2
)
pipeline.fit(df_text_image_audio, epochs=2, per_device_train_batch_size=8, logging_steps=3, eval_steps=6)
preds = pipeline.predict(df_text_image_audio.head(5), return_label_str=True)
emb = pipeline.get_embeddings(df_text_image_audio.head(8), batch_size=8)
print(preds)
print(emb)

# Дообучение регрессора, который работает с данными разной модальностью.

In [ ]:
!pip install -q wav2clip torchaudio transformers evaluate pillow

import math
from typing import List, Dict, Any, Optional, Union

import numpy as np
import pandas as pd
from PIL import Image

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader

import evaluate
from transformers import Trainer, TrainingArguments
from transformers.modeling_outputs import ModelOutput


def set_seed(seed: int = 42):
    """
    Устанавливает фиксированное зерно для Python, NumPy и PyTorch (включая все доступные CUDA-устройства).

    :param seed: Значение зерна.
    """
    import random
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)


def to_pil(x: Union[str, np.ndarray, Image.Image]) -> Image.Image:
    """
    Приводит входное изображение к PIL.Image в формате RGB.

    Поддерживаемые форматы:
      - путь к файлу (str);
      - NumPy массив (H, W[, C]) со значениями uint8;
      - PIL.Image (любого режима).

    :param x: Источник изображения.
    :return: Изображение PIL.Image в RGB.
    :raises ValueError: Если тип входа не поддерживается.
    """
    if isinstance(x, Image.Image):
        return x.convert("RGB")
    if isinstance(x, str):
        return Image.open(x).convert("RGB")
    if isinstance(x, np.ndarray):
        return Image.fromarray(x).convert("RGB")
    raise ValueError("Ожидается путь/np.ndarray/PIL.Image")


def load_audio(path: str, target_sr: int) -> np.ndarray:
    """
    Загружает аудио и (при необходимости) ресемплирует до target_sr.

    - Монофонизирует вход через усреднение каналов.
    - Возвращает float32 массив формы [T].

    :param path: Путь к аудиофайлу (wav/flac/…).
    :param target_sr: Целевая частота дискретизации.
    :return: Сигнал формы [T] float32.
    :raises RuntimeError: Если torchaudio недоступен.
    """
    try:
        import torchaudio
    except Exception as e:
        raise RuntimeError("Требуется torchaudio: pip install torchaudio") from e
    waveform, sr = torchaudio.load(path)  # [C, T]
    if waveform.size(0) > 1:
        waveform = waveform.mean(dim=0, keepdim=True)
    if sr != target_sr:
        waveform = torchaudio.functional.resample(waveform, orig_freq=sr, new_freq=target_sr)
    return waveform.squeeze(0).numpy().astype(np.float32)


class MultiComboRegDataset(Dataset):
    """
    Регрессионный датасет с поддержкой нескольких изображений и аудио на сэмпл.

    Ячейки колонок изображений/аудио могут содержать:
      - одиночное значение (путь/np.ndarray/PIL для изображения; путь/np.ndarray для аудио);
      - список таких значений.

    :param df: Исходный DataFrame.
    :param target_cols: Список целевых колонок (поддерживается многомерная регрессия).
    :param text_columns: Список текстовых колонок. Их значения конкатенируются через [SEP].
    :param image_columns: Список колонок изображений.
    :param audio_columns: Список колонок аудио.
    """
    def __init__(
        self,
        df: pd.DataFrame,
        target_cols: List[str],
        text_columns: Optional[List[str]] = None,
        image_columns: Optional[List[str]] = None,
        audio_columns: Optional[List[str]] = None
    ):
        """
        Инициализация датасета.

        :param df: Исходный DataFrame.
        :param target_cols: Названия колонок целей.
        :param text_columns: Текстовые колонки (склеиваются).
        :param image_columns: Колонки изображений (значение или список значений).
        :param audio_columns: Колонки аудио (значение или список значений).
        """
        self.df = df.reset_index(drop=True)
        self.target_cols = target_cols
        self.text_columns = text_columns or []
        self.image_columns = image_columns or []
        self.audio_columns = audio_columns or []
        self.sep = " [SEP] "

    def __len__(self) -> int:
        """
        Количество элементов датасета.

        :return: Длина датасета.
        """
        return len(self.df)

    def __getitem__(self, idx: int) -> Dict[str, Any]:
        """
        Возвращает один элемент датасета.

        Ключи в словаре:
          - 'labels': np.ndarray формы [T], float32;
          - 'text': строка (если заданы text_columns);
          - 'images': список изображений (пути/np.ndarray/PIL) или пустой список;
          - 'audios': список аудио (пути/np.ndarray) или пустой список.

        :param idx: Индекс строки.
        :return: Словарь для последующего collate бэкенда.
        """
        row = self.df.iloc[idx]
        y = np.array([float(row[c]) for c in self.target_cols], dtype=np.float32)

        def _as_list(v):
            if v is None or (isinstance(v, float) and np.isnan(v)):
                return []
            if isinstance(v, (list, tuple)):
                return list(v)
            return [v]

        item = {"labels": y}
        if self.text_columns:
            item["text"] = self.sep.join([str(row[c]) if pd.notnull(row[c]) else "" for c in self.text_columns])

        if self.image_columns:
            imgs = []
            for c in self.image_columns:
                if c in row:
                    imgs.extend(_as_list(row[c]))
            item["images"] = imgs

        if self.audio_columns:
            auds = []
            for c in self.audio_columns:
                if c in row:
                    auds.extend(_as_list(row[c]))
            item["audios"] = auds

        return item


class BaseBackend(nn.Module):
    """
    Базовый класс бэкенда для единой мультимодальной модели.

    Атрибуты:
      - name: Название бэкенда.
      - supported: Набор поддерживаемых модальностей (например, {'text','image'}).
      - embed_dim: Базовая размерность эмбеддингов модальностей.
      - out_dim_per_modality: Словарь фактических размерностей выходных эмбеддингов по модальностям
                              (с учётом агрегации/concat).
    """
    name: str = "base"
    supported: set = set()
    embed_dim: int = 0
    out_dim_per_modality: Dict[str, int] = {}

    def collate(self, batch: List[Dict[str, Any]]) -> Dict[str, Any]:
        """
        Собирает батч для текущего бэкенда.

        :param batch: Список элементов MultiComboRegDataset.
        :return: Словарь вида:
                 {
                   'labels': torch.FloatTensor [B, T],
                   'backend_inputs': dict(...)
                 }
        """
        raise NotImplementedError

    def encode(self, backend_inputs: Dict[str, Any], device: torch.device) -> Dict[str, torch.Tensor]:
        """
        Кодирует модальности и возвращает эмбеддинги по доступным модальностям.

        :param backend_inputs: Предобработанные входы (из collate).
        :param device: Целевой девайс.
        :return: Словарь {'text':[B,*], 'image':[B,*], 'audio':[B,*]} по имеющимся модальностям.
        """
        raise NotImplementedError

    def freeze_all(self):
        """
        Замораживает все параметры бэкенда (requires_grad=False).
        """
        for p in self.parameters():
            p.requires_grad = False

    @staticmethod
    def _stack_labels_float(batch: List[Dict[str, Any]]) -> torch.Tensor:
        """
        Преобразует список 'labels' из элементов батча в тензор float32 [B, T] с выравниванием по T.

        Если длины таргетов различны, добивает меньшие нулями справа до максимальной длины.

        :param batch: Список элементов датасета.
        :return: torch.FloatTensor [B, T].
        """
        ys = []
        for b in batch:
            y = b.get("labels", None)
            if y is None:
                ys.append(np.array([0.0], dtype=np.float32))
            else:
                arr = np.array(y, dtype=np.float32).reshape(-1)
                ys.append(arr)
        max_t = max(a.shape[0] for a in ys)
        ys_padded = []
        for a in ys:
            if a.shape[0] == max_t:
                ys_padded.append(a)
            else:
                pad = np.zeros(max_t, dtype=np.float32)
                pad[:a.shape[0]] = a
                ys_padded.append(pad)
        return torch.from_numpy(np.stack(ys_padded, axis=0))

    def get_out_dim(self, modality: str) -> int:
        """
        Возвращает итоговую размерность выходного вектора по модальности
        (с учётом агрегации и параметров max_*).

        :param modality: Имя модальности: 'text' | 'image' | 'audio'.
        :return: Размерность эмбеддинга.
        """
        return self.out_dim_per_modality.get(modality, self.embed_dim)


class ClipBackend(BaseBackend):
    """
    CLIP-бэкенд для текста и изображений с поддержкой нескольких изображений на сэмпл.

    Агрегация изображений:
      - 'concat': конкатенация эмбеддингов N изображений в один вектор (с паддингом нулями до max_images);
      - 'mean': усреднение эмбеддингов изображений.

    :param checkpoint: Имя/путь чекпоинта CLIP (HF Hub).
    :param max_length: Максимальная длина токенов текста.
    :param freeze: Замораживать ли веса CLIP (linear probing).
    :param max_images: Максимальное число изображений на сэмпл для агрегации.
    :param image_agg: Тип агрегации изображений ('concat' или 'mean').
    """
    name = "clip"
    supported = {"text", "image"}

    def __init__(
        self,
        checkpoint: str = "openai/clip-vit-base-patch32",
        max_length: int = 77,
        freeze: bool = True,
        max_images: int = 1,
        image_agg: str = "concat"
    ):
        """
        Инициализация CLIP-бэкенда.

        :param checkpoint: Чекпоинт CLIP.
        :param max_length: Максимальная длина текста (токенов).
        :param freeze: Замораживать ли веса.
        :param max_images: Максимум изображений на сэмпл.
        :param image_agg: Агрегация изображений.
        """
        super().__init__()
        from transformers import CLIPModel, CLIPProcessor
        self.model = CLIPModel.from_pretrained(checkpoint)
        self.processor = CLIPProcessor.from_pretrained(checkpoint)
        self.embed_dim = int(self.model.config.projection_dim)
        self.max_length = max_length
        self.max_images = int(max_images)
        self.image_agg = image_agg
        if freeze:
            self.freeze_all()
        img_out = self.embed_dim * self.max_images if self.image_agg == "concat" else self.embed_dim
        self.out_dim_per_modality = {"text": self.embed_dim, "image": img_out}

    def collate(self, batch: List[Dict[str, Any]]) -> Dict[str, Any]:
        """
        Готовит батч для CLIP.

        Формирует:
          - text_inputs: токены текста;
          - image_inputs: pixel_values (возможен None);
          - image_counts: количество изображений на каждый сэмпл (для агрегации).

        :param batch: Элементы с ключами 'text', 'images', 'labels'.
        :return: {'labels': FloatTensor [B,T], 'backend_inputs': dict}.
        """
        labels = self._stack_labels_float(batch)
        texts = [b.get("text", "") for b in batch]

        images_lists = [b.get("images", []) for b in batch]
        flat_images, counts = [], []
        for lst in images_lists:
            lst = lst if isinstance(lst, list) else ([] if lst is None else [lst])
            counts.append(len(lst))
            for img in lst:
                flat_images.append(to_pil(img))

        text_inputs = self.processor(
            text=texts, padding=True, truncation=True, max_length=self.max_length, return_tensors="pt"
        )
        text_inputs = {k: v for k, v in text_inputs.items()}

        if len(flat_images):
            img_proc = self.processor(images=flat_images, return_tensors="pt")
            image_inputs = {"pixel_values": img_proc["pixel_values"]}
        else:
            image_inputs = {"pixel_values": None}

        backend_inputs = {
            "text_inputs": text_inputs,
            "image_inputs": image_inputs,
            "image_counts": torch.tensor(counts, dtype=torch.long),
        }
        return {"labels": labels, "backend_inputs": backend_inputs}

    def _concat_padded(self, embs: torch.Tensor, counts: List[int], max_k: int) -> torch.Tensor:
        """
        Конкатенирует до max_k эмбеддингов изображений на сэмпл с паддингом нулями.

        :param embs: Плоские эмбеддинги изображений [M, D], где M = сумма(counts).
        :param counts: Список количеств изображений на сэмпл.
        :param max_k: Максимум изображений на сэмпл (для паддинга/отрезания).
        :return: Нормализованный тензор [B, D*max_k].
        """
        device = embs.device if embs is not None else torch.device("cpu")
        B = len(counts); D = self.embed_dim
        out = torch.zeros((B, D * max_k), device=device, dtype=embs.dtype if embs is not None else torch.float32)
        if embs is None:
            return out
        offset = 0
        for i, c in enumerate(counts):
            if c > 0:
                chunk = embs[offset:offset + c]
                take = chunk[:max_k]
                if take.size(0) < max_k:
                    pad = torch.zeros((max_k - take.size(0), D), device=device, dtype=embs.dtype)
                    take = torch.cat([take, pad], dim=0)
                out[i] = take.reshape(-1)
            offset += c
        return F.normalize(out, dim=-1)

    def _mean_pool(self, embs: torch.Tensor, counts: List[int]) -> torch.Tensor:
        """
        Усредняет эмбеддинги изображений на сэмпл.

        :param embs: Плоские эмбеддинги [M, D].
        :param counts: Количества изображений на сэмпл.
        :return: Нормализованный тензор [B, D].
        """
        device = embs.device if embs is not None else torch.device("cpu")
        B = len(counts); D = self.embed_dim
        out = torch.zeros((B, D), device=device, dtype=embs.dtype if embs is not None else torch.float32)
        if embs is None:
            return out
        offset = 0
        for i, c in enumerate(counts):
            if c > 0:
                out[i] = embs[offset:offset + c].mean(dim=0)
            offset += c
        return F.normalize(out, dim=-1)

    def encode(self, backend_inputs: Dict[str, Any], device: torch.device) -> Dict[str, torch.Tensor]:
        """
        Вычисляет эмбеддинги текста и изображений (с агрегацией изображений).

        :param backend_inputs: Тензоры CLIPProcessor для текста и картинок.
        :param device: Девайс.
        :return: {'text':[B,D], 'image':[B,D*max_images] или [B,D]} — L2-нормированные эмбеддинги.
        """
        ti = {k: v.to(device) for k, v in backend_inputs["text_inputs"].items()}
        text_z = self.model.get_text_features(input_ids=ti["input_ids"], attention_mask=ti["attention_mask"])
        text_z = F.normalize(text_z, dim=-1)

        counts = backend_inputs["image_counts"].tolist()
        pi = backend_inputs["image_inputs"]["pixel_values"]
        if pi is not None:
            pi = pi.to(device)
            img_flat = self.model.get_image_features(pixel_values=pi)
            img_flat = F.normalize(img_flat, dim=-1)
            if self.image_agg == "concat":
                img_z = self._concat_padded(img_flat, counts, self.max_images)
            else:
                img_z = self._mean_pool(img_flat, counts)
        else:
            img_z = torch.zeros(
                (len(counts), self.embed_dim * self.max_images if self.image_agg == "concat" else self.embed_dim),
                device=device
            )

        return {"text": text_z, "image": img_z}


class ClapBackend(BaseBackend):
    """
    CLAP-бэкенд для текста и аудио с поддержкой нескольких аудио на сэмпл.

    Агрегация аудио:
      - 'concat': конкатенация эмбеддингов N аудиоклипов в один вектор (с паддингом нулями до max_audios);
      - 'mean': усреднение эмбеддингов аудио.

    :param checkpoint: Имя/путь чекпоинта CLAP (HF Hub).
    :param freeze: Замораживать ли веса CLAP.
    :param max_audios: Максимальное число аудио на сэмпл для агрегации.
    :param audio_agg: Тип агрегации аудио ('concat' или 'mean').
    """
    name = "clap"
    supported = {"text", "audio"}

    def __init__(
        self,
        checkpoint: str = "laion/clap-htsat-unfused",
        freeze: bool = True,
        max_audios: int = 1,
        audio_agg: str = "concat"
    ):
        """
        Инициализация CLAP-бэкенда.

        :param checkpoint: Чекпоинт CLAP.
        :param freeze: Замораживать ли веса.
        :param max_audios: Максимум аудио на сэмпл.
        :param audio_agg: Агрегация аудио.
        """
        super().__init__()
        from transformers import ClapModel, ClapProcessor
        self.model = ClapModel.from_pretrained(checkpoint)
        self.processor = ClapProcessor.from_pretrained(checkpoint)
        self.embed_dim = int(getattr(self.model.config, "projection_dim", 512))
        sr = getattr(self.processor, "sampling_rate", None)
        if sr is None:
            fe = getattr(self.processor, "feature_extractor", None)
            sr = getattr(fe, "sampling_rate", 48000) if fe is not None else 48000
        self.sr = int(sr)
        self.max_audios = int(max_audios)
        self.audio_agg = audio_agg
        if freeze:
            self.freeze_all()
        aud_out = self.embed_dim * self.max_audios if self.audio_agg == "concat" else self.embed_dim
        self.out_dim_per_modality = {"text": self.embed_dim, "audio": aud_out}

    def collate(self, batch: List[Dict[str, Any]]) -> Dict[str, Any]:
        """
        Готовит батч для CLAP.

        Формирует:
          - text_inputs: токены текста;
          - audio_inputs: input_features (или None);
          - audio_counts: число аудио на сэмпл.

        :param batch: Элементы с ключами 'text','audios','labels'.
        :return: {'labels': FloatTensor [B,T], 'backend_inputs': dict}.
        """
        labels = self._stack_labels_float(batch)
        texts = [b.get("text", "") for b in batch]

        audios_lists = [b.get("audios", []) for b in batch]
        flat_audios, counts = [], []
        for lst in audios_lists:
            lst = lst if isinstance(lst, list) else ([] if lst is None else [lst])
            counts.append(len(lst))
            for a in lst:
                if isinstance(a, str):
                    flat_audios.append(load_audio(a, self.sr))
                elif isinstance(a, np.ndarray):
                    flat_audios.append(a.astype(np.float32))
                else:
                    raise ValueError("Ожидается путь к аудио или numpy.ndarray")

        text_inputs = self.processor(text=texts, padding=True, truncation=True, return_tensors="pt")
        text_inputs = {k: v for k, v in text_inputs.items()}

        if len(flat_audios):
            aud_proc = self.processor(audios=flat_audios, sampling_rate=self.sr, padding=True, return_tensors="pt")
            audio_inputs = {"input_features": aud_proc["input_features"]}
        else:
            audio_inputs = {"input_features": None}

        backend_inputs = {
            "text_inputs": text_inputs,
            "audio_inputs": audio_inputs,
            "audio_counts": torch.tensor(counts, dtype=torch.long)
        }
        return {"labels": labels, "backend_inputs": backend_inputs}

    def _concat_padded(self, embs: torch.Tensor, counts: List[int], max_k: int) -> torch.Tensor:
        """
        Конкатенирует до max_k эмбеддингов аудио на сэмпл с паддингом нулями.

        :param embs: Плоские эмбеддинги аудио [M, D].
        :param counts: Список количеств аудио на сэмпл.
        :param max_k: Максимум аудио на сэмпл.
        :return: Нормализованный тензор [B, D*max_k].
        """
        device = embs.device if embs is not None else torch.device("cpu")
        B = len(counts); D = self.embed_dim
        out = torch.zeros((B, D * max_k), device=device, dtype=embs.dtype if embs is not None else torch.float32)
        if embs is None:
            return out
        offset = 0
        for i, c in enumerate(counts):
            if c > 0:
                chunk = embs[offset:offset + c]
                take = chunk[:max_k]
                if take.size(0) < max_k:
                    pad = torch.zeros((max_k - take.size(0), D), device=device, dtype=embs.dtype)
                    take = torch.cat([take, pad], dim=0)
                out[i] = take.reshape(-1)
            offset += c
        return F.normalize(out, dim=-1)

    def _mean_pool(self, embs: torch.Tensor, counts: List[int]) -> torch.Tensor:
        """
        Усредняет эмбеддинги аудио на сэмпл.

        :param embs: Плоские эмбеддинги [M, D].
        :param counts: Количества аудио на сэмпл.
        :return: Нормализованный тензор [B, D].
        """
        device = embs.device if embs is not None else torch.device("cpu")
        B = len(counts); D = self.embed_dim
        out = torch.zeros((B, D), device=device, dtype=embs.dtype if embs is not None else torch.float32)
        if embs is None:
            return out
        offset = 0
        for i, c in enumerate(counts):
            if c > 0:
                out[i] = embs[offset:offset + c].mean(dim=0)
            offset += c
        return F.normalize(out, dim=-1)

    def encode(self, backend_inputs: Dict[str, Any], device: torch.device) -> Dict[str, torch.Tensor]:
        """
        Вычисляет эмбеддинги текста и аудио (с агрегацией аудио).

        :param backend_inputs: Тензоры ClapProcessor для текста и аудио.
        :param device: Девайс.
        :return: {'text':[B,D], 'audio':[B,D*max_audios] или [B,D]} — L2-нормированные эмбеддинги.
        """
        ti = {k: v.to(device) for k, v in backend_inputs["text_inputs"].items()}
        if hasattr(self.model, "get_text_features") and hasattr(self.model, "get_audio_features"):
            text_z = self.model.get_text_features(input_ids=ti["input_ids"], attention_mask=ti["attention_mask"])
        else:
            # совместимость со старыми версиями transformers
            out = self.model(**ti, output_attentions=False, output_hidden_states=False, return_dict=True)
            text_z = out.text_embeds
        text_z = F.normalize(text_z, dim=-1)

        counts = backend_inputs["audio_counts"].tolist()
        af = backend_inputs["audio_inputs"]["input_features"]
        if af is not None:
            af = af.to(device)
            if hasattr(self.model, "get_audio_features"):
                aud_flat = self.model.get_audio_features(input_features=af)
            else:
                out = self.model(input_features=af, output_attentions=False, output_hidden_states=False, return_dict=True)
                aud_flat = out.audio_embeds
            aud_flat = F.normalize(aud_flat, dim=-1)
            if self.audio_agg == "concat":
                aud_z = self._concat_padded(aud_flat, counts, self.max_audios)
            else:
                aud_z = self._mean_pool(aud_flat, counts)
        else:
            aud_z = torch.zeros(
                (len(counts), self.embed_dim * self.max_audios if self.audio_agg == "concat" else self.embed_dim),
                device=device
            )

        return {"text": text_z, "audio": aud_z}


class ClipWav2CLIPBackend(BaseBackend):
    """
    Комбинированный бэкенд: CLIP для текста/изображения + Wav2CLIP для аудио (в CLIP-пространство).

    Поддержка нескольких изображений и аудио на сэмпл с агрегацией 'concat' или 'mean'.

    :param checkpoint: Чекпоинт CLIP (HF).
    :param max_length: Максимальная длина токенов для CLIP.
    :param freeze: Замораживать ли веса CLIP.
    :param audio_sr: Частота дискретизации для Wav2CLIP.
    :param max_images: Максимум изображений на сэмпл.
    :param max_audios: Максимум аудио на сэмпл.
    :param image_agg: Тип агрегации изображений ('concat' | 'mean').
    :param audio_agg: Тип агрегации аудио ('concat' | 'mean').
    """
    name = "clip_wav2clip"
    supported = {"text", "image", "audio"}

    def __init__(
        self,
        checkpoint: str = "openai/clip-vit-base-patch32",
        max_length: int = 77,
        freeze: bool = True,
        audio_sr: int = 16000,
        max_images: int = 1,
        max_audios: int = 1,
        image_agg: str = "concat",
        audio_agg: str = "concat"
    ):
        """
        Инициализация ClipWav2CLIP-бэкенда.

        :param checkpoint: Чекпоинт CLIP.
        :param max_length: Максимальная длина текста.
        :param freeze: Замораживать ли веса CLIP.
        :param audio_sr: Частота дискретизации для Wav2CLIP.
        :param max_images: Максимум изображений на сэмпл.
        :param max_audios: Максимум аудио на сэмпл.
        :param image_agg: Агрегация изображений.
        :param audio_agg: Агрегация аудио.
        """
        super().__init__()
        from transformers import CLIPModel, CLIPProcessor
        self.model = CLIPModel.from_pretrained(checkpoint)
        self.processor = CLIPProcessor.from_pretrained(checkpoint)
        self.embed_dim = int(self.model.config.projection_dim)
        self.max_length = max_length
        self.audio_sr = int(audio_sr)
        self.max_images = int(max_images)
        self.max_audios = int(max_audios)
        self.image_agg = image_agg
        self.audio_agg = audio_agg
        if freeze:
            self.freeze_all()
        img_out = self.embed_dim * self.max_images if self.image_agg == "concat" else self.embed_dim
        aud_out = self.embed_dim * self.max_audios if self.audio_agg == "concat" else self.embed_dim
        self.out_dim_per_modality = {"text": self.embed_dim, "image": img_out, "audio": aud_out}
        # ленивое подключение Wav2CLIP
        self._w2c_model = None
        self._w2c_mod = None
        self._w2c_api = None
        self._w2c_device = None

    def _ensure_w2c(self, device: torch.device):
        """
        Ленивая загрузка wav2clip и его модели для текущего устройства.

        Поддерживаются разные версии API:
          - функции load_model/get_model + embed_audio/get_audio_embedding;
          - класс Wav2CLIP с методом embed_audio.

        :param device: Текущее устройство.
        :raises RuntimeError: Если wav2clip недоступен или не удалось инициализировать модель.
        """
        if self._w2c_model is not None and self._w2c_device == str(device):
            return
        import importlib
        try:
            w2c = importlib.import_module("wav2clip")
        except Exception as e:
            raise RuntimeError("Не найден пакет 'wav2clip'. Установите: pip install wav2clip") from e
        dev_str = str(device) if device.type == "cuda" else "cpu"
        if hasattr(w2c, "load_model"):
            model = w2c.load_model(device=dev_str); api_kind = "func"
        elif hasattr(w2c, "get_model"):
            model = w2c.get_model(device=dev_str); api_kind = "func"
        elif hasattr(w2c, "Wav2CLIP"):
            try:
                model = w2c.Wav2CLIP(dev_str)
            except TypeError:
                model = w2c.Wav2CLIP(device=dev_str)
            api_kind = "method"
        else:
            raise RuntimeError("wav2clip установлен, но нет способов загрузки (load_model/get_model/Wav2CLIP)")
        self._w2c_mod = w2c
        self._w2c_model = model
        self._w2c_api = api_kind
        self._w2c_device = str(device)

    def collate(self, batch: List[Dict[str, Any]]) -> Dict[str, Any]:
        """
        Готовит батч для текста/изображений (через CLIPProcessor) и аудио (через паддинг numpy->Tensor).

        Возвращает:
          - text_inputs: словарь тензоров для текста;
          - image_inputs: словарь с pixel_values или None;
          - image_counts: LongTensor [B] — количество изображений на сэмпл;
          - audio_inputs: {'waveforms': Tensor [M,Lmax] или None, 'lengths': LongTensor [M] или None};
          - audio_counts: LongTensor [B] — количество аудио на сэмпл;
          - labels: FloatTensor [B, T].

        :param batch: Элементы с 'text','images','audios','labels'.
        :return: Словарь {'labels', 'backend_inputs'}.
        """
        labels = self._stack_labels_float(batch)
        texts = [b.get("text", "") for b in batch]

        # images (flatten + counts)
        images_lists = [b.get("images", []) for b in batch]
        flat_images, img_counts = [], []
        for lst in images_lists:
            lst = lst if isinstance(lst, list) else ([] if lst is None else [lst])
            img_counts.append(len(lst))
            for img in lst:
                flat_images.append(to_pil(img))
        text_inputs = self.processor(text=texts, padding=True, truncation=True, max_length=self.max_length, return_tensors="pt")
        text_inputs = {k: v for k, v in text_inputs.items()}
        if len(flat_images):
            img_proc = self.processor(images=flat_images, return_tensors="pt")
            image_inputs = {"pixel_values": img_proc["pixel_values"]}
        else:
            image_inputs = {"pixel_values": None}

        # audios (flatten + counts)
        audios_lists = [b.get("audios", []) for b in batch]
        flat_audios, aud_counts = [], []
        for lst in audios_lists:
            lst = lst if isinstance(lst, list) else ([] if lst is None else [lst])
            aud_counts.append(len(lst))
            for a in lst:
                if isinstance(a, str):
                    flat_audios.append(load_audio(a, self.audio_sr))
                elif isinstance(a, np.ndarray):
                    flat_audios.append(a.astype(np.float32))
                else:
                    raise ValueError("Ожидается путь к аудио или numpy.ndarray")

        if len(flat_audios):
            Lmax = max(len(a) for a in flat_audios)
            wav = np.zeros((len(flat_audios), Lmax), dtype=np.float32)
            lens = np.zeros((len(flat_audios),), dtype=np.int64)
            for i, a in enumerate(flat_audios):
                L = len(a)
                wav[i, :L] = a
                lens[i] = L
            audio_inputs = {"waveforms": torch.from_numpy(wav), "lengths": torch.from_numpy(lens)}
        else:
            audio_inputs = {"waveforms": None, "lengths": None}

        backend_inputs = {
            "text_inputs": text_inputs,
            "image_inputs": image_inputs,
            "image_counts": torch.tensor(img_counts, dtype=torch.long),
            "audio_inputs": audio_inputs,
            "audio_counts": torch.tensor(aud_counts, dtype=torch.long),
        }
        return {"labels": labels, "backend_inputs": backend_inputs}

    def _agg_concat(self, embs: torch.Tensor, counts: List[int], max_k: int) -> torch.Tensor:
        """
        Конкатенирует до max_k эмбеддингов на сэмпл с паддингом нулями.

        :param embs: Плоские эмбеддинги [M, D].
        :param counts: Количества элементов на сэмпл.
        :param max_k: Максимум элементов на сэмпл (для паддинга).
        :return: Нормализованный тензор [B, D*max_k].
        """
        device = embs.device if embs is not None else torch.device("cpu")
        B = len(counts); D = self.embed_dim
        out = torch.zeros((B, D * max_k), device=device, dtype=embs.dtype if embs is not None else torch.float32)
        if embs is None:
            return out
        offset = 0
        for i, c in enumerate(counts):
            if c > 0:
                chunk = embs[offset:offset+c]
                take = chunk[:max_k]
                if take.size(0) < max_k:
                    pad = torch.zeros((max_k - take.size(0), D), device=device, dtype=embs.dtype)
                    take = torch.cat([take, pad], dim=0)
                out[i] = take.reshape(-1)
            offset += c
        return F.normalize(out, dim=-1)

    def _agg_mean(self, embs: torch.Tensor, counts: List[int]) -> torch.Tensor:
        """
        Усредняет эмбеддинги на сэмпл.

        :param embs: Плоские эмбеддинги [M, D].
        :param counts: Количества элементов на сэмпл.
        :return: Нормализованный тензор [B, D].
        """
        device = embs.device if embs is not None else torch.device("cpu")
        B = len(counts); D = self.embed_dim
        out = torch.zeros((B, D), device=device, dtype=embs.dtype if embs is not None else torch.float32)
        if embs is None:
            return out
        offset = 0
        for i, c in enumerate(counts):
            if c > 0:
                out[i] = embs[offset:offset+c].mean(dim=0)
            offset += c
        return F.normalize(out, dim=-1)

    def encode(self, backend_inputs: Dict[str, Any], device: torch.device) -> Dict[str, torch.Tensor]:
        """
        Кодирует модальности: текст/изображение — через CLIP; аудио — через Wav2CLIP.

        :param backend_inputs: Входы из collate().
        :param device: Целевой девайс.
        :return: Словарь {'text',[B,D]; 'image',[B,*]; 'audio',[B,*]} в CLIP-пространстве.
        """
        # text + image via CLIP
        ti = {k: v.to(device) for k, v in backend_inputs["text_inputs"].items()}
        text_z = self.model.get_text_features(input_ids=ti["input_ids"], attention_mask=ti["attention_mask"])
        text_z = F.normalize(text_z, dim=-1)

        img_counts = backend_inputs["image_counts"].tolist()
        pi = backend_inputs["image_inputs"]["pixel_values"]
        if pi is not None:
            pi = pi.to(device)
            img_flat = self.model.get_image_features(pixel_values=pi)
            img_flat = F.normalize(img_flat, dim=-1)
            if self.image_agg == "concat":
                image_z = self._agg_concat(img_flat, img_counts, self.max_images)
            else:
                image_z = self._agg_mean(img_flat, img_counts)
        else:
            image_z = torch.zeros(
                (len(img_counts), self.embed_dim * self.max_images if self.image_agg == "concat" else self.embed_dim),
                device=device
            )

        # audio via wav2clip
        aud_counts = backend_inputs["audio_counts"].tolist()
        wav = backend_inputs["audio_inputs"]["waveforms"]
        lens = backend_inputs["audio_inputs"]["lengths"]
        if wav is not None and lens is not None and (lens.numel() if torch.is_tensor(lens) else len(lens)) > 0:
            self._ensure_w2c(device)
            w2c = self._w2c_mod
            embs = []
            for i in range(wav.size(0)):
                L = int(lens[i].item())
                a_np = wav[i, :L].detach().cpu().numpy()
                e = None
                if self._w2c_api == "func":
                    if hasattr(w2c, "embed_audio"):
                        try:
                            e = w2c.embed_audio(a_np, self._w2c_model)
                        except TypeError:
                            e = w2c.embed_audio(a_np, self.audio_sr, self._w2c_model)
                    elif hasattr(w2c, "get_audio_embedding"):
                        try:
                            e = w2c.get_audio_embedding(a_np, self._w2c_model)
                        except TypeError:
                            e = w2c.get_audio_embedding(a_np, self.audio_sr, self._w2c_model)
                if e is None and self._w2c_api == "method" and hasattr(self._w2c_model, "embed_audio"):
                    try:
                        e = self._w2c_model.embed_audio(a_np)
                    except TypeError:
                        e = self._w2c_model.embed_audio(a_np, sr=self.audio_sr)
                if e is None:
                    raise RuntimeError("Не удалось получить аудио‑эмбеддинг через wav2clip.")
                e = np.asarray(e)
                if e.ndim == 2:
                    e = e.mean(axis=0)
                elif e.ndim > 2:
                    e = e.reshape(-1, e.shape[-1]).mean(axis=0)
                embs.append(e.astype(np.float32))
            aud_flat = torch.tensor(np.stack(embs, axis=0), dtype=torch.float32, device=device)
            aud_flat = F.normalize(aud_flat, dim=-1)
            if self.audio_agg == "concat":
                audio_z = self._agg_concat(aud_flat, aud_counts, self.max_audios)
            else:
                audio_z = self._agg_mean(aud_flat, aud_counts)
        else:
            audio_z = torch.zeros(
                (len(aud_counts), self.embed_dim * self.max_audios if self.audio_agg == "concat" else self.embed_dim),
                device=device
            )

        return {"text": text_z, "image": image_z, "audio": audio_z}


class SingleBackboneRegressor(nn.Module):
    """
    Регрессор поверх одного мультимодального бэкенда:
    фьюжн эмбеддингов модальностей -> MLP-голова -> предсказание R^T.
    """
    def __init__(
        self,
        backend: BaseBackend,
        modalities: List[str],
        num_targets: int,
        fusion: str = "concat",
        hidden: int = 512,
        dropout: float = 0.1
    ):
        """
        Инициализирует регрессионную голову.

        :param backend: Инициализированный бэкенд (CLIP/CLAP/ClipWav2CLIP).
        :param modalities: Список активных модальностей (учитывается порядок: image, text, audio).
        :param num_targets: Число целевых признаков (T).
        :param fusion: Тип фьюжна — 'concat' или 'mean'.
        :param hidden: Размер скрытого слоя головы.
        :param dropout: Дропаут в голове.
        :raises ValueError: Если fusion='mean' при несовпадающих размерах модальностей.
        """
        super().__init__()
        self.backend = backend
        self.modalities = modalities
        self.fusion = fusion
        self.num_targets = num_targets

        order = [m for m in ["image", "text", "audio"] if m in self.modalities]
        if fusion == "concat":
            in_dim = sum(self.backend.get_out_dim(m) for m in order)
        elif fusion == "mean":
            dims = [self.backend.get_out_dim(m) for m in order]
            if len(set(dims)) != 1:
                raise ValueError(f'Для fusion="mean" размеры модальностей должны совпадать: {dict(zip(order, dims))}')
            in_dim = dims[0]
        else:
            raise ValueError('fusion должен быть "concat" или "mean"')

        self.head = nn.Sequential(
            nn.LayerNorm(in_dim),
            nn.Linear(in_dim, hidden),
            nn.GELU(),
            nn.Dropout(dropout),
            nn.Linear(hidden, num_targets)
        )

    def _infer_device_from_inputs(self, obj) -> torch.device:
        """
        Пытается определить целевой девайс по первому найденному тензору во входах.

        :param obj: Произвольная вложенная структура (тензоры/словари/списки).
        :return: torch.device (cuda при наличии, иначе cpu).
        """
        if isinstance(obj, torch.Tensor):
            return obj.device
        if isinstance(obj, dict):
            for v in obj.values():
                d = self._infer_device_from_inputs(v)
                if d is not None:
                    return d
        return torch.device("cuda" if torch.cuda.is_available() else "cpu")

    def _fuse(self, z: Dict[str, torch.Tensor]) -> torch.Tensor:
        """
        Объединяет эмбеддинги модальностей согласно self.fusion.

        - concat: конкатенация по последней оси;
        - mean: среднее по модальностям (требует совпадения размерностей).

        :param z: Словарь эмбеддингов по модальностям.
        :return: Слитый эмбеддинг [B, D*|mods|] (concat) или [B, D] (mean).
        """
        order = [m for m in ["image", "text", "audio"] if m in self.modalities]
        feats = []
        for m in order:
            t = z[m]
            if t.dim() == 3:
                t = t.mean(dim=1)
            elif t.dim() > 3:
                t = t.view(t.size(0), -1)
            feats.append(t)
        if self.fusion == "concat":
            return torch.cat(feats, dim=-1)
        elif self.fusion == "mean":
            sizes = [f.size(-1) for f in feats]
            if len(set(sizes)) != 1:
                raise ValueError(f'Для fusion="mean" размеры модальностей должны совпадать: {sizes}')
            return torch.stack(feats, dim=0).mean(dim=0)

    def forward(self, backend_inputs: Dict[str, Any], labels: Optional[torch.Tensor] = None) -> ModelOutput:
        """
        Прямой проход: кодирование модальностей -> фьюжн -> регрессионная голова.

        :param backend_inputs: Входы для бэкенда (из его collate()).
        :param labels: Не используется (Trainer читает labels отдельно).
        :return: ModelOutput с полем logits [B, T].
        """
        device = self._infer_device_from_inputs(backend_inputs)
        z = self.backend.encode(backend_inputs, device=device)
        fused = self._fuse(z)
        preds = self.head(fused)
        return ModelOutput(logits=preds)

    @torch.no_grad()
    def get_embeddings(self, backend_inputs: Dict[str, Any], return_per_modality: bool = False):
        """
        Извлекает fused эмбеддинги (и, опционально, эмбеддинги по модальностям).

        :param backend_inputs: Входы для бэкенда.
        :param return_per_modality: Вернуть также словарь эмбеддингов по модальностям.
        :return: fused [B, *] или (fused, {'text':[B,*], 'image':[B,*], 'audio':[B,*]}).
        """
        device = self._infer_device_from_inputs(backend_inputs)
        z = self.backend.encode(backend_inputs, device=device)
        fused = self._fuse(z)
        if return_per_modality:
            return fused, z
        return fused


class MSETrainer(Trainer):
    """
    Trainer для регрессии на основе MSE loss.

    :param reduction: Редукция лосса ('mean' по умолчанию).
    """
    def __init__(self, *args, reduction: str = "mean", **kwargs):
        """
        Инициализация MSETrainer.

        :param args: Аргументы Trainer.
        :param reduction: Тип редукции MSELoss ('mean' | 'sum' | 'none').
        :param kwargs: Аргументы Trainer.
        """
        super().__init__(*args, **kwargs)
        self._reduction = reduction

    def compute_loss(self, model, inputs, return_outputs: bool = False, num_items_in_batch: Optional[int] = None):
        """
        Вычисляет MSE лосс между предсказаниями и целевыми значениями.

        :param model: Модель.
        :param inputs: Батч, содержащий 'labels' (float [B,T]) и аргументы для model.forward().
        :param return_outputs: Возвращать ли также outputs.
        :param num_items_in_batch: Совместимость с API Trainer (не используется).
        :return: loss (и outputs, если return_outputs=True).
        """
        labels = inputs.pop("labels")
        outputs = model(**inputs)
        preds = outputs.logits
        labels = labels.to(preds.device)
        loss = nn.MSELoss(reduction=self._reduction)(preds, labels)
        return (loss, outputs) if return_outputs else loss


class SingleModelMultiComboRegression:
    """
    Регрессионный пайплайн на одной мультимодальной модели (бэкенде) под заданные комбинации модальностей.

    Поддерживаемые комбинации (auto-подбор бэкенда):
      - ['text','image']         -> CLIP (HF)
      - ['text','audio']         -> CLAP (HF)
      - ['image','audio']        -> ClipWav2CLIP
      - ['text','image','audio'] -> ClipWav2CLIP

    Поддержка нескольких изображений/аудио на сэмпл с агрегацией 'concat' или 'mean'.
    """
    def __init__(
        self,
        modalities: List[str],
        target_columns_names: List[str],
        text_columns: Optional[List[str]] = None,
        image_columns: Optional[List[str]] = None,
        audio_columns: Optional[List[str]] = None,
        backend: str = "auto",
        clip_checkpoint: str = "openai/clip-vit-base-patch32",
        clap_checkpoint: str = "laion/clap-htsat-unfused",
        fusion: str = "concat",
        freeze_backbone: bool = True,
        clip_max_length: int = 77,
        max_images_per_sample: int = 1,
        max_audios_per_sample: int = 1,
        image_agg: str = "concat",
        audio_agg: str = "concat"
    ):
        """
        Инициализация пайплайна регрессии.

        :param modalities: Активные модальности ('text','image','audio') в любом порядке.
        :param target_columns_names: Имена целевых колонок (num_targets = len(target_columns_names)).
        :param text_columns: Имена текстовых колонок (склеиваются).
        :param image_columns: Список колонок изображений (ячейки — значение или список значений).
        :param audio_columns: Список колонок аудио (ячейки — значение или список значений).
        :param backend: 'auto' | 'clip' | 'clap' | 'clip_wav2clip'.
        :param clip_checkpoint: Чекпоинт CLIP (HF).
        :param clap_checkpoint: Чекпоинт CLAP (HF).
        :param fusion: Тип фьюжна ('concat' или 'mean').
        :param freeze_backbone: Заморозить веса бэкенда (linear probing).
        :param clip_max_length: Максимальная длина токенов для CLIP.
        :param max_images_per_sample: Максимум изображений на сэмпл при агрегации.
        :param max_audios_per_sample: Максимум аудио на сэмпл при агрегации.
        :param image_agg: Агрегация изображений ('concat' | 'mean').
        :param audio_agg: Агрегация аудио ('concat' | 'mean').
        """
        self.modalities = sorted(list(set(modalities)))
        self.target_columns_names = target_columns_names
        self.num_targets = len(target_columns_names)
        self.text_columns = text_columns or []
        self.image_columns = image_columns or []
        self.audio_columns = audio_columns or []
        self.backend_name = backend
        self.clip_checkpoint = clip_checkpoint
        self.clap_checkpoint = clap_checkpoint
        self.fusion = fusion
        self.freeze_backbone = freeze_backbone
        self.clip_max_length = clip_max_length
        self.max_images_per_sample = int(max_images_per_sample)
        self.max_audios_per_sample = int(max_audios_per_sample)
        self.image_agg = image_agg
        self.audio_agg = audio_agg

        self.backend: Optional[BaseBackend] = None
        self.model: Optional[SingleBackboneRegressor] = None
        self.trainer: Optional[Trainer] = None
        self.compute_metrics = None

        self._build_backend()

    def _build_backend(self):
        """
        Инициализирует бэкенд согласно настройке backend/auto и проверяет совместимость с модальностями.

        :raises ValueError: Если комбинация модальностей не поддерживается выбранным бэкендом.
        """
        mods = set(self.modalities)
        name = self.backend_name
        if name == "auto":
            if mods == {"text", "image"}:
                name = "clip"
            elif mods == {"text", "audio"}:
                name = "clap"
            elif mods in ({"image", "audio"}, {"text", "image", "audio"}):
                name = "clip_wav2clip"
            else:
                raise ValueError(f"Неподдерживаемая комбинация: {mods}")

        if name == "clip":
            self.backend = ClipBackend(
                checkpoint=self.clip_checkpoint,
                max_length=self.clip_max_length,
                freeze=self.freeze_backbone,
                max_images=self.max_images_per_sample,
                image_agg=self.image_agg
            )
        elif name == "clap":
            self.backend = ClapBackend(
                checkpoint=self.clap_checkpoint,
                freeze=self.freeze_backbone,
                max_audios=self.max_audios_per_sample,
                audio_agg=self.audio_agg
            )
        elif name == "clip_wav2clip":
            self.backend = ClipWav2CLIPBackend(
                checkpoint=self.clip_checkpoint,
                max_length=self.clip_max_length,
                freeze=self.freeze_backbone,
                audio_sr=16000,
                max_images=self.max_images_per_sample,
                max_audios=self.max_audios_per_sample,
                image_agg=self.image_agg,
                audio_agg=self.audio_agg
            )
        else:
            raise ValueError(f"Неизвестный backend: {name}")

        if not set(self.modalities).issubset(self.backend.supported):
            raise ValueError(
                f"Бэкенд {self.backend.name} не поддерживает модальности {self.modalities}. "
                f"Поддерживает: {self.backend.supported}"
            )

    def _validate_data(self, df: pd.DataFrame):
        """
        Проверяет соответствие колонок в DataFrame выбранным модальностям и целям.

        :param df: Исходный датафрейм.
        :raises ValueError: При отсутствии обязательных колонок.
        """
        missing_targets = [c for c in self.target_columns_names if c not in df.columns]
        if missing_targets:
            raise ValueError(f"В DataFrame отсутствуют целевые колонки: {missing_targets}")

        if "text" in self.modalities:
            if not self.text_columns:
                raise ValueError("Вы выбрали модальность 'text', но text_columns пустой.")
            missing = [c for c in self.text_columns if c not in df.columns]
            if missing:
                raise ValueError(f"В DataFrame отсутствуют текстовые колонки: {missing}")

        if "image" in self.modalities:
            if not self.image_columns:
                raise ValueError("Вы выбрали модальность 'image', но image_columns пуст.")
            missing = [c for c in self.image_columns if c not in df.columns]
            if missing:
                raise ValueError(f"В DataFrame отсутствуют колонки изображений: {missing}")

        if "audio" in self.modalities:
            if not self.audio_columns:
                raise ValueError("Вы выбрали модальность 'audio', но audio_columns пуст.")
            missing = [c for c in self.audio_columns if c not in df.columns]
            if missing:
                raise ValueError(f"В DataFrame отсутствуют колонки аудио: {missing}")

    def _split(self, df: pd.DataFrame, test_size: float = 0.2, seed: int = 42):
        """
        Перемешивает и делит DataFrame на обучающую и валидационную части.

        :param df: Исходный датафрейм.
        :param test_size: Доля валидации (0..1).
        :param seed: Зерно для перемешивания.
        :return: (df_train, df_eval).
        """
        df = df.sample(frac=1.0, random_state=seed).reset_index(drop=True)
        n_eval = int(math.ceil(len(df) * test_size))
        return df.iloc[n_eval:].reset_index(drop=True), df.iloc[:n_eval].reset_index(drop=True)

    def _setup_compute_metrics(self, metric_name: str):
        """
        Создаёт функцию подсчёта метрик для Trainer.

        Поддерживаемые итоговые метрики:
          - 'rmse' (минимизируется),
          - 'mae'  (минимизируется),
          - 'mse'  (минимизируется),
          - 'r2'   (максимизируется).

        :param metric_name: Название основной метрики.
        :raises RuntimeError: Если недоступны r_squared и sklearn для R2.
        """
        name = metric_name.lower()
    
        def compute(p):
            preds = np.asarray(p.predictions)
            refs  = np.asarray(p.label_ids)
    
            # гарантируем форму [N, T]
            if preds.ndim == 1: preds = preds[:, None]
            if refs.ndim  == 1: refs  = refs[:,  None]
    
            T = min(preds.shape[1], refs.shape[1])
            mse_list, mae_list, r2_list = [], [], []
    
            for t in range(T):
                y_true = refs[:, t].astype(np.float64)
                y_pred = preds[:, t].astype(np.float64)
    
                err = y_pred - y_true
                mse = float(np.mean(err**2))
                mae = float(np.mean(np.abs(err)))
    
                # R^2: 1 - SS_res/SS_tot; если дисперсия нулевая, даём 0.0 (как безопасный фолбэк)
                var = float(np.var(y_true))
                if var == 0.0:
                    r2 = 0.0
                else:
                    ss_res = float(np.sum(err**2))
                    ss_tot = float(np.sum((y_true - np.mean(y_true))**2))
                    r2 = 1.0 - (ss_res / ss_tot if ss_tot > 0 else 0.0)
    
                mse_list.append(mse)
                mae_list.append(mae)
                r2_list.append(r2)
    
            mse_avg = float(np.mean(mse_list))
            rmse_avg = float(np.sqrt(mse_avg))
            mae_avg = float(np.mean(mae_list))
            r2_avg = float(np.mean(r2_list))
            return {"rmse": rmse_avg, "mse": mse_avg, "mae": mae_avg, "r2": r2_avg}
    
        self.compute_metrics = compute
        self._primary_metric = name
        self._greater_is_better = True if name == "r2" else False

    def fit(
        self,
        train_data: pd.DataFrame,
        epochs: int = 3,
        test_size: float = 0.2,
        per_device_train_batch_size: int = 16,
        gradient_accumulation_steps: int = 1,
        learning_rate: float = 2e-4,
        metric_name: str = "rmse",
        fp16: bool = True,
        logging_steps: int = 50,
        eval_steps: int = 200,
        output_dir: str = "./reg_result",
        seed: int = 42,
        hidden: int = 512,
        dropout: float = 0.1
    ):
        """
        Обучает регрессионную голову поверх выбранного бэкенда.

        :param train_data: Данные обучения с необходимыми колонками модальностей и целями.
        :param epochs: Количество эпох обучения.
        :param test_size: Доля валидации.
        :param per_device_train_batch_size: Размер батча на устройство.
        :param gradient_accumulation_steps: Шаги аккумуляции градиентов.
        :param learning_rate: Learning rate для AdamW.
        :param metric_name: Основная метрика ('rmse' | 'mae' | 'mse' | 'r2') для выбора лучшей модели.
        :param fp16: Использовать ли fp16 при наличии CUDA.
        :param logging_steps: Периодичность логирования.
        :param eval_steps: Периодичность валидации/сохранения чекпоинтов.
        :param output_dir: Папка для логов/чекпоинтов.
        :param seed: Зерно.
        :param hidden: Размер скрытого слоя головы.
        :param dropout: Дропаут в голове.
        :return: self.
        :raises ValueError: При проблемах с данными или параметрами.
        """
        self._validate_data(train_data)
        set_seed(seed)

        df_train, df_eval = self._split(train_data, test_size=test_size, seed=seed)
        ds_train = MultiComboRegDataset(df_train, self.target_columns_names, self.text_columns, self.image_columns, self.audio_columns)
        ds_eval  = MultiComboRegDataset(df_eval,  self.target_columns_names, self.text_columns, self.image_columns, self.audio_columns)

        self.model = SingleBackboneRegressor(
            backend=self.backend,
            modalities=self.modalities,
            num_targets=self.num_targets,
            fusion=self.fusion,
            hidden=hidden,
            dropout=dropout
        )

        self._setup_compute_metrics(metric_name)

        args = TrainingArguments(
            output_dir=output_dir,
            num_train_epochs=epochs,
            per_device_train_batch_size=per_device_train_batch_size,
            gradient_accumulation_steps=gradient_accumulation_steps,
            learning_rate=learning_rate,
            warmup_ratio=0.1,
            lr_scheduler_type="cosine",
            weight_decay=0.01,
            eval_strategy="steps",
            eval_steps=eval_steps,
            save_strategy="steps",
            save_steps=eval_steps,
            load_best_model_at_end=True,
            metric_for_best_model=f"eval_{self._primary_metric}",
            greater_is_better=self._greater_is_better,
            save_total_limit=1,
            logging_strategy="steps",
            logging_steps=logging_steps,
            report_to="none",
            fp16=fp16 and torch.cuda.is_available(),
            dataloader_num_workers=0,
            seed=seed,
            remove_unused_columns=False
        )

        def data_collator(batch_list):
            """
            Collate-хук для Trainer: делегирует сборку батча в backend.collate().

            :param batch_list: Список элементов датасета.
            :return: Батч словаря {'labels', 'backend_inputs'}.
            """
            return self.backend.collate(batch_list)

        self.trainer = MSETrainer(
            model=self.model,
            args=args,
            train_dataset=ds_train,
            eval_dataset=ds_eval,
            data_collator=data_collator,
            compute_metrics=self.compute_metrics
        )
        self.trainer.train()
        return self

    def predict(self, df: pd.DataFrame) -> np.ndarray:
        """
        Делает предсказания регрессионных таргетов на новых данных.

        :param df: Датафрейм с нужными колонками модальностей (целевые колонки могут отсутствовать).
        :return: Массив предсказаний формы [N, T].
        :raises RuntimeError: Если модель ещё не обучена.
        """
        if self.trainer is None:
            raise RuntimeError("Модель не обучена. Вызовите .fit().")
        df_c = df.copy()
        for c in self.target_columns_names:
            if c not in df_c.columns:
                df_c[c] = 0.0
        ds = MultiComboRegDataset(df_c, self.target_columns_names, self.text_columns, self.image_columns, self.audio_columns)
        preds = self.trainer.predict(test_dataset=ds)
        return preds.predictions

    def get_embeddings(self, df: pd.DataFrame, batch_size: int = 32, return_per_modality: bool = False):
        """
        Извлекает эмбеддинги для новых данных (fused и, опционально, по модальностям).

        :param df: Датафрейм с нужными колонками модальностей.
        :param batch_size: Размер батча для извлечения эмбеддингов.
        :param return_per_modality: Вернуть также по-модальные эмбеддинги.
        :return: fused [N, D_fused] или (fused, {'text':[N,*], 'image':[N,*], 'audio':[N,*]}).
        :raises RuntimeError: Если модель ещё не обучена.
        """
        if self.trainer is None or self.model is None:
            raise RuntimeError("Модель не обучена. Вызовите .fit().")

        device = getattr(self.trainer.args, "device", None)
        if device is None:
            try:
                device = next(self.trainer.model.parameters()).device
            except StopIteration:
                device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

        self.model.to(device)
        self.model.eval()

        df_c = df.copy()
        for c in self.target_columns_names:
            if c not in df_c.columns:
                df_c[c] = 0.0

        ds = MultiComboRegDataset(
            df_c,
            self.target_columns_names,
            self.text_columns,
            self.image_columns,
            self.audio_columns
        )

        def collate(batch_list):
            """
            Collate для DataLoader при извлечении эмбеддингов (использует backend.collate()).

            :param batch_list: Элементы датасета.
            :return: Батч {'labels', 'backend_inputs'}.
            """
            return self.backend.collate(batch_list)

        def move_to_device(obj, device):
            """
            Рекурсивно переносит тензоры в структуре на заданный девайс.

            :param obj: Тензор/словарь/список/кортеж/прочее.
            :param device: torch.device ('cpu' или 'cuda').
            :return: Объект с перенесёнными тензорами.
            """
            if torch.is_tensor(obj):
                return obj.to(device)
            if isinstance(obj, dict):
                return {k: move_to_device(v, device) for k, v in obj.items()}
            if isinstance(obj, (list, tuple)):
                t = [move_to_device(v, device) for v in obj]
                return type(obj)(t) if not isinstance(obj, list) else t
            return obj

        loader = DataLoader(ds, batch_size=batch_size, shuffle=False, collate_fn=collate)
        fused_list = []
        per_mod_lists = {m: [] for m in self.modalities} if return_per_modality else None

        with torch.no_grad():
            for batch in loader:
                bi = move_to_device(batch["backend_inputs"], device)
                fused, per = self.model.get_embeddings(backend_inputs=bi, return_per_modality=True)
                fused_list.append(fused.cpu().numpy())
                if return_per_modality:
                    for m in per_mod_lists.keys():
                        if m in per:
                            per_mod_lists[m].append(per[m].cpu().numpy())

        fused_arr = np.vstack(fused_list)
        if not return_per_modality:
            return fused_arr
        per_mod = {m: np.vstack(chunks) for m, chunks in per_mod_lists.items()}
        return fused_arr, per_mod

In [ ]:
import os, random
import numpy as np
import pandas as pd
from PIL import Image

import torch
import torchaudio

HAVE_TORCHAUDIO = True

random.seed(42); np.random.seed(42); torch.manual_seed(42)

BASE_DIR = "./dummy_data"
IMG_DIR  = os.path.join(BASE_DIR, "images")
AUD_DIR  = os.path.join(BASE_DIR, "audio")
os.makedirs(IMG_DIR, exist_ok=True)
os.makedirs(AUD_DIR, exist_ok=True)

# Сколько элементов на модальность в каждой строке
K_PER_MODALITY = 3

def make_dummy_images(n=12, size=(256, 256)):
    paths = []
    for i in range(n):
        color = tuple(np.random.randint(0, 255, size=3).tolist())
        img = Image.new("RGB", size, color=color)
        path = os.path.join(IMG_DIR, f"img_{i:02d}.png")
        img.save(path)
        paths.append(path)
    return paths

def make_dummy_audios(n=12, sr=48000, duration_sec=0.6):
    if not HAVE_TORCHAUDIO:
        raise RuntimeError("Нельзя сгенерировать аудио без torchaudio. Установите 'pip install torchaudio'.")
    paths = []
    t = torch.linspace(0, duration_sec, int(sr * duration_sec))
    for i in range(n):
        freq = random.choice([220, 330, 440, 550, 660, 880])
        wave = 0.2 * torch.sin(2 * np.pi * freq * t)  # амплитуда 0.2
        wave = wave.unsqueeze(0)  # [1, T] mono
        path = os.path.join(AUD_DIR, f"tone_{i:02d}.wav")
        torchaudio.save(path, wave, sample_rate=sr)
        paths.append(path)
    return paths

img_paths = make_dummy_images(n=12)
audio_paths = make_dummy_audios(n=12) if HAVE_TORCHAUDIO else []

# Вспомогательные тексты
TITLES = ["Red fox", "Blue sky", "Green field", "Yellow sun", "Purple rain", "Silver line"]
BODIES = ["quick brown", "lazy dog", "jumps high", "runs fast", "stays calm", "shines bright"]
QUERIES = ["find tone", "classify sound", "describe image", "retrieve pair", "detect event"]

def rand_title(): return random.choice(TITLES)
def rand_body(): return random.choice(BODIES)
def rand_query(): return random.choice(QUERIES)

# Универсальные хелперы
def sample_k(seq, k):
    if len(seq) >= k:
        return random.sample(seq, k)  # без повторов
    else:
        return [random.choice(seq) for _ in range(k)]  # с повторами, если мало исходников

def as_cols(prefix, values):
    # {"prefix_1": values[0], ..., "prefix_k": values[k-1]}
    return {f"{prefix}_{i+1}": v for i, v in enumerate(values)}

def pick_text_desc(k=K_PER_MODALITY):
    # Текстовое описание: "Title | body"
    vals = [f"{rand_title()} | {rand_body()}" for _ in range(k)]
    return as_cols("text", vals)

def pick_text_query(k=K_PER_MODALITY):
    vals = [rand_query() for _ in range(k)]
    return as_cols("text", vals)

def pick_images(k=K_PER_MODALITY):
    vals = sample_k(img_paths, k)
    return as_cols("image_path", vals)

def pick_audios(k=K_PER_MODALITY):
    vals = sample_k(audio_paths, k)
    return as_cols("audio_path", vals)

# 1) Текст + Картинка -> по 3 текстовых и 3 картинок
def build_df_text_image(n=24, k=K_PER_MODALITY):
    rows = []
    for _ in range(n):
        row = {}
        row.update(pick_text_desc(k))
        row.update(pick_images(k))
        row["y1"] = random.uniform(-10, 10)
        row["y2"] = random.uniform(-10, 10)
        row["y3"] = random.uniform(-10, 10)
        rows.append(row)
    return pd.DataFrame(rows)

# 2) Текст + Звук -> по 3 текста (query) и 3 аудио
def build_df_text_audio(n=24, k=K_PER_MODALITY):
    if not HAVE_TORCHAUDIO:
        raise RuntimeError("Для text+audio нужен torchaudio.")
    rows = []
    for _ in range(n):
        row = {}
        row.update(pick_text_query(k))
        row.update(pick_audios(k))
        row["y1"] = random.uniform(-10, 10)
        row["y2"] = random.uniform(-10, 10)
        row["y3"] = random.uniform(-10, 10)
        rows.append(row)
    return pd.DataFrame(rows)

# 3) Картинка + Звук -> по 3 картинки и 3 аудио
def build_df_image_audio(n=24, k=K_PER_MODALITY):
    if not HAVE_TORCHAUDIO:
        raise RuntimeError("Для image+audio нужен torchaudio.")
    rows = []
    for _ in range(n):
        row = {}
        row.update(pick_images(k))
        row.update(pick_audios(k))
        row["y1"] = random.uniform(-10, 10)
        row["y2"] = random.uniform(-10, 10)
        row["y3"] = random.uniform(-10, 10)
        rows.append(row)
    return pd.DataFrame(rows)

# 4) Текст + Картинка + Звук -> по 3 на каждую модальность
def build_df_text_image_audio(n=24, k=K_PER_MODALITY):
    if not HAVE_TORCHAUDIO:
        raise RuntimeError("Для text+image+audio нужен torchaudio.")
    rows = []
    for _ in range(n):
        row = {}
        row.update(pick_text_desc(k))
        row.update(pick_images(k))
        row.update(pick_audios(k))
        row["y1"] = random.uniform(-10, 10)
        row["y2"] = random.uniform(-10, 10)
        row["y3"] = random.uniform(-10, 10)
        rows.append(row)
    return pd.DataFrame(rows)

# Собираем 4 датасета
df_text_image = build_df_text_image(24, K_PER_MODALITY)
df_text_audio = build_df_text_audio(24, K_PER_MODALITY) if HAVE_TORCHAUDIO else None
df_image_audio = build_df_image_audio(24, K_PER_MODALITY) if HAVE_TORCHAUDIO else None
df_text_image_audio  = build_df_text_image_audio(24, K_PER_MODALITY) if HAVE_TORCHAUDIO else None

print("df_text_image columns:", list(df_text_image.columns))
if HAVE_TORCHAUDIO:
    print("df_text_audio columns:", list(df_text_audio.columns))
    print("df_image_audio columns:", list(df_image_audio.columns))
    print("df_text_image_audio columns:", list(df_text_image_audio.columns))

Пример использования на данных текст + картинка.

In [ ]:
pipeline = SingleModelMultiComboRegression(
    modalities=["text", "image"],
    target_columns_names=["y1", "y2"],
    text_columns=["text_1", "text_2"],
    image_columns=["image_path_1"],
    backend="auto",
    clip_checkpoint="openai/clip-vit-base-patch32",  # необязательно
    fusion="concat",  # можно другое сделать
    freeze_backbone=True
)

pipeline.fit(df_text_image, epochs=1, per_device_train_batch_size=8)
preds = pipeline.predict(df_text_image.head(5))
embeddings = pipeline.get_embeddings(df_text_image.head(8))

print(preds)
print(embeddings)

Пример использования на данных текст + звук.

In [ ]:
pipeline = SingleModelMultiComboRegression(
    modalities=["text", "audio"],
    target_columns_names=["y1"],
    text_columns=["text_1"],
    audio_columns=["audio_path_1", "audio_path_2"],
    backend="auto",  # выберет CLAP
    clap_checkpoint="laion/clap-htsat-unfused",  # необязательно
    fusion="concat",
    freeze_backbone=True
)

pipeline.fit(df_text_audio, epochs=1, per_device_train_batch_size=8)
preds = pipeline.predict(df_text_audio.head(5))
embeddings = pipeline.get_embeddings(df_text_audio.head(8))

print(preds)
print(embeddings)

Пример использования на данных картинка + звук.

In [ ]:
pipeline = SingleModelMultiComboRegression(
    modalities=["image", "audio"],
    target_columns_names=["y1", "y2", "y3"],
    image_columns=["image_path_1", "image_path_2", "image_path_3"],
    audio_columns=["audio_path_1", "audio_path_2", "audio_path_3"],
    backend="auto",  # выберет ClipWav2CLIP
    clip_checkpoint="openai/clip-vit-base-patch32",  # необязательно
    fusion="concat",
    freeze_backbone=True
)

pipeline.fit(df_image_audio, epochs=1, per_device_train_batch_size=8)
preds = pipeline.predict(df_image_audio.head(5))
embeddings = pipeline.get_embeddings(df_image_audio.head(8))

print(preds)
print(embeddings)

Пример использования на данных текст + картинка + звук.

In [ ]:
pipeline = SingleModelMultiComboRegression(
    modalities=["text", "image", "audio"],
    target_columns_names=["y1", "y2"],
    text_columns=["text_1"],
    image_columns=["image_path_1"],
    audio_columns=["audio_path_1"],
    backend="auto",  # выберет ClipWav2CLIP
    clip_checkpoint="openai/clip-vit-base-patch32",
    fusion="concat",
    freeze_backbone=True
)

pipeline.fit(df_text_image_audio, epochs=4, per_device_train_batch_size=8,
             logging_steps=3, eval_steps=6)
preds = pipeline.predict(df_text_image_audio.head(5))
embeddings = pipeline.get_embeddings(df_text_image_audio.head(8))

print(preds)
print(embeddings)